# Aufgabenstellung

Diese Arbeit behandelt Aufgabenteil 2 des Projektes der Veranstaltung Roboterprogrammierung und beschäftigt sich mit der Erweiterung und systematischen Evaluation eines Lazy Probabilistic Roadmaps (Lazy-PRM) Verfahrens für die Pfadplanung. Der Fokus liegt auf der Entwicklung, Implementierung und Analyse verschiedener Node-Enhancementstrategien innerhalb des Lazy-PRM-Rahmens. Die Aufgabenstellung umfasst die folgenden AUfgaben:

- 2a: Entwicklung und Implementierung von mindestens drei unterschiedlichen Node-Enhancementstrategien zur gezielten Erweiterung der Roadmap.
- 2b: Anwendung, Vergleich und Animation der entwickelten Strategien anhand mehrerer Benchmark-Umgebungen für einen 2-DOF Punktroboter und einem 2DoF Planarroboter.
- 2c: Grafische Darstellung der Anzahl der Kollisonsberechnungen, Planungszeit, Roadmapgröße und Länge des Lösungspfades sowie Diskussion der Ergebnisse. 

Ziel ist es, zu untersuchen, wie unterschiedliche Node-Enhancementstrategien die Qualität, Effizienz und Robustheit von Lazy-PRM beeinflussen.


**Lazy-PRM & Enhancementstrategien**

Lazy-PRM basiert auf der klassischen Probabilistic Roadmap, unterscheidet sich jedoch durch den verzögerten Umgang mit Kollisionen.
Knoten und Kanten werden zunächst ohne aufwändige Kollisionsprüfung in die Roadmap aufgenommen. Erst wenn ein konkreter Pfad abgefragt wird, werden die benötigten Kanten schrittweise auf Kollision überprüft und bei Bedarf verworfen. Dieses „lazy“ Vorgehen reduziert initiale Rechenkosten und verschiebt Kollisionsprüfungen gezielt auf relevante Teile des Suchraums. Im Rahmen dieser Arbeit wurden fünf verschiedene Node-Enhancemethoden untersucht:

- **Baseline:** Gleichverteiltes Sampling im Konfigurationsraum
- **Mode 1:** Seed-basiertes Gauß-Sampling um bestehende Roadmap-Knoten
- **Mode 2:** Distanz-basiertes Sampling zur Förderung der Exploration
- **Mode 3:** Max-Min-Sampling zur gezielten Erweiterung der Roadmap in bislang schwach abgedeckten Bereichen des Konfigurationsraums.
- **Mode 4:** Start-/Ziel-biasiertes Sampling zur Erhöhung der Konnektivität in pfadrelevanten Regionen des Konfigurationsraums.

Jede Strategie wird im Notebook separat erläutert, implementiert und experimentell bewertet

Im folgendem Codeabschnitt werden alle benötigten Bibliotheken sowie die projektinternen Module für Benchmarking, Kollisionsprüfung und Lazy-PRM geladen. Anschließend werden die globalen Roadmapparameter und darauf aufbauend verschiedene Node-Enhancementstrategien (Baseline und Mode 1–4) definiert. Abschließend wird eine Hilfsfunktion bereitgestellt, um selbst erstellte Benchmark-Umgebungen inklusive Hindernissen sowie Start- und Zielkonfiguration zu visualisieren.

In [ ]:
# Imports
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from shapely.geometry import Polygon, LineString, Point
from IPBenchmark import Benchmark
from IPEnvironment import CollisionChecker

import IPTestSuite as ts
from IPVISLazyPRM import lazyPRMVisualize
from IPLazyPRM_Task2 import EnhancedLazyPRM

# Globale Roadmapparameter 
base_config = {
    "initialRoadmapSize": 2,    # Startgröße der Roadmap (wie viele Nodes zu Beginn)
    "updateRoadmapSize": 3,     # wie viele neue Nodes pro “Enhancement/Update”-Iteration hinzugefügt werden
    "kNearest": 4,              # wie viele Nachbarn pro Node für Kantenbildung
    "maxIterations": 90,        # maximale Iterationen (Roadmap-Erweiterungen / Lazy-PRM-Loops)

    # Robotparameter zentral in der Config
    "robotRadius": 0.25,        # Roboterradius
    "safetyMargin": 0.05,       # Sicherheitsabstand
}

# Modespezifische Konfiguration
cfg_base = dict(base_config, enhanceMode="baseline_uniform")
cfg_m1 = dict(base_config, enhanceMode="mode1_seed_gauss", seedSigma=4.0, seedTries=5)
cfg_m2 = dict(base_config, enhanceMode="mode2_seed_dist", seedMaxStep=4.0, seedBeta=0.9, seedTries=5)
cfg_m3 = dict(base_config, enhanceMode="mode3_max_min", dispersionCandidates=30)
cfg_m4 = dict(base_config, enhanceMode="mode4_start_goal_corr", corridorSigma=3.0, corridorAlongSigma=0.6, corridorTries=6)# 1.8 und 0.45

configs = {
    "baseline_uniform":       cfg_base,
    "mode1_seed_gauss":       cfg_m1,
    "mode2_seed_dist":        cfg_m2,
    "mode3_max_min":          cfg_m3,   
    "mode4_start_goal_corr":  cfg_m4,  
}

MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

ROBOT_RADIUS = base_config["robotRadius"]
SAFETY = base_config["safetyMargin"]
CLEARANCE = ROBOT_RADIUS + SAFETY   # Mindestabstand zum Hindernis

def plot_benchmark(bench, bounds=(0, 22, 0, 22), ax=None,
                   obstacle_alpha=0.25, obstacle_edge="white",
                   start_color="green", goal_color="red"):
    """
    Visualisiert einen IPBenchmark.Benchmark:
    - Hindernisse aus bench.collisionChecker.scene (Shapely-Geometrien)
    - Start / Goal aus startList/goalList
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))

    xmin, xmax, ymin, ymax = bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, alpha=0.3)
    ax.set_title(bench.name)

    # Hindernisse zeichnen
    scene = getattr(bench.collisionChecker, "scene", {})
    for _, geom in scene.items():
        if geom.is_empty:
            continue

        # Polygon / MultiPolygon
        if geom.geom_type == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=obstacle_alpha, edgecolor=obstacle_edge, linewidth=2)
        elif geom.geom_type == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=obstacle_alpha, edgecolor=obstacle_edge, linewidth=2)
        else:
            # Für Circles (buffer) ist es meist Polygon, aber falls nicht:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=obstacle_alpha, edgecolor=obstacle_edge, linewidth=2)
            except Exception:
                pass

    # Start / Goal
    s = bench.startList[0]
    g = bench.goalList[0]
    ax.scatter([s[0]], [s[1]], s=120, c=start_color, label="Start", zorder=5)
    ax.scatter([g[0]], [g[1]], s=120, c=goal_color, label="Goal", zorder=5)

    ax.legend(loc="upper right")
    return ax


# Benchmarks

Für die Evaluation der Node-Enhancementstrategien werden ausschließlich eigens entwickelte Benchmark-Umgebungen verwendet. Vorgegebene oder externe Benchmarks kommen in dieser Arbeit nicht zum Einsatz. Die selbst generierten Benchmarks sind gezielt so gestaltet, dass sie typische Herausforderungen wie Engstellen und schmale Passagen, Bereiche mit hoher Hindernisdichte oder Strukturierte Korridore und komplexe Geometrien abbilden. Durch die kontrollierte Gestaltung der Benchmarks ist es möglich, die Vor- und Nachteile der einzelnen Node-Enhancementstrategien systematisch und reproduzierbar zu untersuchen. Insbesondere kann analysiert werden, wie sich die Strategien unter steigender Umgebungs- und Konfigurationsraumkomplexität verhalten.

## Benchmark 1: Circle Field

Bei dem Benchmark "CirckeField" sind viele runde Hindernisse gleichmäßig über den Arbeitsraum verteilt, teilweise auch randnah. Zwischen den Hindernissen existieren zahlreiche alternative Durchgänge unterschiedlicher Breite. Dieser Benchmark testet die globale Abdeckung und Homogenität der Roadmap. Erfolgreiche Strategien müssen den Raum gleichmäßig explorieren, ohne sich zu stark auf lokale Bereiche zu konzentrieren. Besonders geeignet zur Bewertung von uniformem Sampling (Baseline-Strategie) und Max-Min-Strategien (Mode 3).

In [ ]:
import numpy as np
from shapely.geometry import Point

def make_circle_field_benchmark(
    name="CircleField",
    bounds=(0.0, 22.0, 0.0, 22.0),        # (xmin, xmax, ymin, ymax)
    start=(2.0, 20.0),
    goal=(20.0, 2.0),
    n_circles=18,
    r_min=1.2,
    r_max=2.2,
    min_center_dist=1.0,                  # zusätzlicher Abstand zwischen Kreismittelpunkten
    inner_keepout=2.2,                    # Abstand Start/Ziel (und optional “Luft” um sie herum)
    edge_overshoot=2.5,                   # Mittelpunkte dürfen so weit über den Rand hinaus liegen
    seed=42,
    max_tries=20000
):
    """
    Viele runde Hindernisse (Kreise) zufällig, so dass Start/Ziel frei bleiben.
    Anders als vorher: Kreise dürfen bis in den Rand ragen bzw. über den Rand hinausgehen.
    """
    rng = np.random.default_rng(seed)
    xmin, xmax, ymin, ymax = bounds

    scene = {}
    centers = []
    radii = []

    start = np.array(start, dtype=float)
    goal  = np.array(goal, dtype=float)

    # Sampling-Bereich für Mittelpunkte: erweitert, damit Kreise in den Rand ragen können
    x_lo = xmin - float(edge_overshoot)
    x_hi = xmax + float(edge_overshoot)
    y_lo = ymin - float(edge_overshoot)
    y_hi = ymax + float(edge_overshoot)

    tries = 0
    while len(centers) < n_circles and tries < max_tries:
        tries += 1

        r = float(rng.uniform(r_min, r_max))
        x = float(rng.uniform(x_lo, x_hi))
        y = float(rng.uniform(y_lo, y_hi))
        c = np.array([x, y], dtype=float)

        # Start/Ziel frei halten (hier bewusst unabhängig vom Rand)
        if np.linalg.norm(c - start) < (r + inner_keepout):
            continue
        if np.linalg.norm(c - goal) < (r + inner_keepout):
            continue

        # Abstand zwischen Hindernissen (Mittelpunktabstand + Radii + min_center_dist)
        ok = True
        for (c2, r2) in zip(centers, radii):
            if np.linalg.norm(c - c2) < (r + r2 + min_center_dist):
                ok = False
                break
        if not ok:
            continue

        centers.append(c)
        radii.append(r)
        scene[f"c{len(centers):02d}"] = Point(x, y).buffer(r)

    if len(centers) < n_circles:
        print(f"[WARN] Nur {len(centers)}/{n_circles} Kreise platziert (Parameter evtl. zu streng).")

    bench = Benchmark(
        name=name,
        collisionChecker=CollisionChecker(scene, limits=[[xmin, xmax], [ymin, ymax]], min_clearance=CLEARANCE),
        startList=[[float(start[0]), float(start[1])]],
        goalList=[[float(goal[0]), float(goal[1])]],
        description=f"Viele runde Hindernisse (n={len(centers)}), Rand-Überlappung erlaubt.",
        level=2
    )

    # Sanity: Start/Ziel kollisionsfrei
    assert not bench.collisionChecker.pointInCollision(bench.startList[0]), f"{name}: Start kollidiert"
    assert not bench.collisionChecker.pointInCollision(bench.goalList[0]),  f"{name}: Ziel kollidiert"

    return bench


# Anzeigen
bench_baseline = make_circle_field_benchmark(
    name="CircleField",
    bounds=(0.0, 22.0, 0.0, 22.0),
    start=(2, 20),
    goal=(20, 2),
    n_circles=40,
    r_min=1.3,
    r_max=2.2,
    inner_keepout=2.2,
    edge_overshoot=2.8,   # <<< hier “mehr Randüberhang” einstellen
    seed=7
)

plot_benchmark(bench_baseline)
plt.show()


## Benchmark 2: WallTinyDoor

Bei dem Benchmark "WallTinyDoor" teilt ein große trennende Wand den Raum nahezu vollständig. Start und Ziel sind nur durch eine sehr schmale Öffnung miteinander verbunden. Dieser Benchmark prüft die Fähigkeit der Strategien, schmale Passagen zuverlässig zu finden und zu durchqueren. Er ist besonders sensitiv gegenüber Strategien, die lokale Exploration (Mode 1) oder gerichtetes Sampling (Mode 4 bzw. Mode 2 als Zwischenform) fördern, und stellt eine klassische Schwäche rein uniformer Verfahren dar.

In [ ]:

def _find_free_point(cc, candidates):
    """Gibt den ersten kollisionsfreien Kandidaten zurück, sonst Exception."""
    for p in candidates:
        if not cc.pointInCollision(list(p)):
            return [float(p[0]), float(p[1])]
    raise Exception("Kein kollisionsfreier Punkt in candidates gefunden. Hindernisse/Bounds prüfen.")

def make_mode1_wall_tiny_door_closed22(extra_block=True):
    scene = {}

    # Raum
    xmin, xmax, ymin, ymax = 0.0, 22.0, 0.0, 22.0

    # Zentrale Wand (blockiert oben/unten komplett) – einzige Passage: Door
    x1, x2 = 9.6, 13.4
    door_y1, door_y2 = 10.2, 11.8

    scene["wall_bottom"] = Polygon([(x1, ymin), (x2, ymin), (x2, door_y1), (x1, door_y1)])
    scene["wall_top"]    = Polygon([(x1, door_y2), (x2, door_y2), (x2, ymax), (x1, ymax)])

    # Optionaler Zusatzblock links oben (kann Start kollidieren lassen -> deshalb Start später automatisch)
    if extra_block:
        scene["block_ul"] = Polygon([(2.0, 15.0), (6.0, 15.0), (6.0, 18.5), (2.0, 18.5)])

    cc = CollisionChecker(scene, min_clearance=CLEARANCE)

    # Kandidaten: Start links oben außerhalb vom block_ul (deutlich weiter links/oben),
    # Ziel rechts unten
    start_candidates = [
        (1.0, 21.0), (1.5, 20.5), (1.0, 19.5), (2.0, 21.0), (3.0, 21.0), (1.0, 18.0)
    ]
    goal_candidates = [
        (21.0, 1.0), (20.5, 1.5), (21.0, 2.0), (19.5, 1.0), (20.0, 3.0)
    ]

    start = _find_free_point(cc, start_candidates)
    goal  = _find_free_point(cc, goal_candidates)

    bench = Benchmark(
        name="WallTinyDoor",
        collisionChecker=cc,
        startList=[start],
        goalList=[goal],
        description="Zentrale Wand blockiert oben/unten komplett, nur eine kleine Tür offen.",
        level=3
    )
    return bench

#Anzeigen
bench_m1 = make_mode1_wall_tiny_door_closed22(extra_block=True)

ax = plot_benchmark(bench_m1)
ax.set_xlim(0, 22)   # x-Ausschnitt
ax.set_ylim(0, 22)  # y-Ausschnitt
plt.show()

## Benchmark 3: WallTinyWideDoor

Der Benchmark "WallTinyWideDoor" ist ähnlich zu "WallTinyDoor", jedoch ist die Öffnung in der Wand deutlich breiter. Die Passage ist vorhanden, aber weniger kritisch. Dieser Benchmark dient als Vergleichsfall zur schmalen Tür und untersucht, wie stark Strategien auf Engstellen reagieren, wenn diese nicht extrem ausgeprägt sind. Er erlaubt Rückschlüsse darauf, ob bestimmte Methoden übermäßig stark auf Engstellen optimiert sind oder auch bei moderaten Durchgängen stabil funktionieren.

In [ ]:
def _find_free_point(cc, candidates):
    """Gibt den ersten kollisionsfreien Kandidaten zurück, sonst Exception."""
    for p in candidates:
        if not cc.pointInCollision(list(p)):
            return [float(p[0]), float(p[1])]
    raise Exception("Kein kollisionsfreier Punkt in candidates gefunden. Hindernisse/Bounds prüfen.")

def make_mode2_wall_tiny_wide_door(extra_block=True):
    scene = {}

    # Raum
    xmin, xmax, ymin, ymax = 0.0, 22.0, 0.0, 22.0

    # Zentrale Wand (blockiert oben/unten komplett) – einzige Passage: Door
    x1, x2 = 6.6, 18.4
    door_y1, door_y2 = 10.2, 11.8

    scene["wall_bottom"] = Polygon([(x1, ymin), (x2, ymin), (x2, door_y1), (x1, door_y1)])
    scene["wall_top"]    = Polygon([(x1, door_y2), (x2, door_y2), (x2, ymax), (x1, ymax)])

    # Optionaler Zusatzblock links oben (kann Start kollidieren lassen -> deshalb Start später automatisch)
    if extra_block:
        scene["block_ul"] = Polygon([(2.0, 15.0), (6.0, 15.0), (6.0, 18.5), (2.0, 18.5)])

    cc = CollisionChecker(scene, min_clearance=CLEARANCE)

    # Kandidaten: Start links oben außerhalb vom block_ul (deutlich weiter links/oben),
    # Ziel rechts unten
    start_candidates = [
        (1.0, 21.0), (1.5, 20.5), (1.0, 19.5), (2.0, 21.0), (3.0, 21.0), (1.0, 18.0)
    ]
    goal_candidates = [
        (21.0, 1.0), (20.5, 1.5), (21.0, 2.0), (19.5, 1.0), (20.0, 3.0)
    ]

    start = _find_free_point(cc, start_candidates)
    goal  = _find_free_point(cc, goal_candidates)

    bench = Benchmark(
        name="WallTinyWideDoor",
        collisionChecker=cc,
        startList=[start],
        goalList=[goal],
        description="Zentrale Wand blockiert oben/unten komplett, nur ein kleine, breite Tür offen.",
        level=3
    )
    return bench

# Anzeigen
bench_m2 = make_mode2_wall_tiny_wide_door(extra_block=True)

ax = plot_benchmark(bench_m2)
ax.set_xlim(0, 22)   # x-Ausschnitt
ax.set_ylim(0, 22)  # y-Ausschnitt
plt.show()

## Benchmark 4: U-Shape

Bei Benchmark "U-Shape" umschließt ein U-förmiges Hindernis das Ziel teilweise. Der direkte Weg ist blockiert, der gültige Pfad erfordert ein gezieltes Umfahren der Struktur. Getestet wird hier die Fähigkeit zur nicht-lokalen Planung. Strategien müssen erkennen, dass ein direkter, zielgerichteter Ansatz scheitert, und stattdessen alternative, längere Wege erkunden. Besonders relevant für die Bewertung von Start–Ziel-biasierten Verfahren (Mode 4).

In [ ]:
def make_mode3_s_corridor_benchmark(
    name="U-Shape",
    bounds=(0.0, 22.0, 0.0, 22.0),
):

    xmin, xmax, ymin, ymax = bounds

    scene = {}

    wall = 4.0
    gap  = 2.4


    # --- Unterer großer Block (macht unteren Teil "zu", lässt aber links unten Start und Korridor offen) ---
    scene["blk_bottom_orange"] = Polygon([
        (xmin + 3.5, ymin + 5.0), # linkes unteres Eck
        (xmax - 5.5, ymin + 5.0),  # rechtes unteres Eck
        (xmax - 5.5, ymin + 15.0), # rechtes oberes Eck
        (xmax - 7.5, ymin + 15.0),
        (xmax - 7.5, ymin + 7.0),
        (xmin + 5.5, ymin + 7.0),
        (xmin + 5.5, ymin + 15.0),
        (xmin + 3.5, ymin + 15.0),# linkes oberes Eck
    ])

    # Start / Ziel
    start = [[xmin + 1.2, ymin + 1.2]]   # links unten
    goal  = [[xmax - 12.0, ymax - 12.0]]   # mittig

    bench = Benchmark(
        name=name,
        collisionChecker=CollisionChecker(scene, min_clearance=CLEARANCE),
        startList=start,
        goalList=goal,
        description="U-Shape.",
        level=4
    )

    # Sanity
    assert not bench.collisionChecker.pointInCollision(start[0]), "Start liegt in Kollision!"
    assert not bench.collisionChecker.pointInCollision(goal[0]),  "Ziel liegt in Kollision!"

    return bench

# Anzeigen
bench_m3 = make_mode3_s_corridor_benchmark()
ax = plot_benchmark(bench_m3)
ax.set_aspect("equal")
ax.set_xlim(0, 22)   # x-Ausschnitt
ax.set_ylim(0, 22)  # y-Ausschnitt
plt.show()

## Benchmark 5: Snail

Bei dem Benchmark "Snail" zwingt ein spiralförmiger Korridor den Roboter, einem langen, schmalen und verschlungenen Pfad zu folgen, um das Ziel im Inneren zu erreichen. Dieser Benchmark kombiniert lange schmale Passagen mit mehrfachen Richtungswechseln und stellt damit eine hohe Anforderung an Roadmap-Konnektivität und Pfadkontinuität. Er eignet sich besonders zur Analyse, ob Strategien nur lokal gute Lösungen finden oder auch komplexe, tief verschachtelte Strukturen zuverlässig erschließen.

In [ ]:
def make_mode4_snail(
    name="Snail",
    bounds=(0.0, 22.0, 0.0, 22.0),
):
    """
    Schneckenförmiges Hindernisfeld
    """
    xmin, xmax, ymin, ymax = bounds

    scene = {}

    # Parameter (einfach zu tunen)
    # "Wand"-Dicke (Hindernisse) und Korridorbreite (frei)
    wall = 4.0
    gap  = 2.4


    # --- Blauer Block  ---
    scene["blk_upper_blue"] = Polygon([
        (xmin + 2.5, ymin + 0.0),  # links unteres Eck
        (xmin + 5.0, ymin + 0.0), 
        (xmin + 5.0, ymin + 17.5),
        (xmin + 17.5, ymin + 17.5),
        (xmin + 17.5, ymin + 5.0), 
        (xmin + 10.0, ymin + 5.0), 
        (xmin + 10.0, ymin + 12.5),
        (xmin + 12.5, ymin + 12.5),
        (xmin + 12.5, ymin + 7.5), #innen
        (xmin + 15.0, ymin + 7.5), # außen
        (xmin + 15.0, ymin + 15.0),
        (xmin + 7.5, ymin + 15.0),
        (xmin + 7.5, ymin + 2.5),
        (xmin + 20.0, ymin + 2.5), # rechtes unteres Eck
        (xmin + 20.0, ymin + 20.0), # rechtes oberes Eck
        
        (xmin + 2.5, ymin + 20.0), # linkes oberes Eck
    
        
    ])

    # Start / Ziel
    start = [[xmin + 1.2, ymin + 1.2]]   # links unten
    goal  = [[xmax - 11.0, ymax - 11.0]]   # mittig

    bench = Benchmark(
        name=name,
        collisionChecker=CollisionChecker(scene, min_clearance=CLEARANCE),
        startList=start,
        goalList=goal,
        description="Schneckenförmiger enger Korridor durch große Hindernisfläche.",
        level=4
    )

    # Sanity
    assert not bench.collisionChecker.pointInCollision(start[0]), "Start liegt in Kollision!"
    assert not bench.collisionChecker.pointInCollision(goal[0]),  "Ziel liegt in Kollision!"

    return bench

# Anzeigen
bench_m4 = make_mode4_snail()
ax = plot_benchmark(bench_m4)
ax.set_aspect("equal")
ax.set_xlim(0, 22)   # x-Ausschnitt
ax.set_ylim(0, 22)  # y-Ausschnitt
plt.show()

# Pipeline für Parametertuning

Für jede Node-Enhancementstrategie (Baseline sowie Mode 1–4) wird ein zweistufiges Parametertuning durchgeführt, basierend ausschließlich auf den selbst entwickelten Benchmarks. Zunächst werden zufällige Parameterkombinationen aus vordefinierten Suchräumen erzeugt und grob evaluiert (Stage A). Anschließend werden die besten Kandidaten erneut mit höherer statistischer Aussagekraft getestet (Stage B), um robuste Parameter zu bestimmen.

Im Rahmen des Parametertunings werden ausschließlich die Mode-spezifischen Parameter der jeweiligen Node-Enhancementstrategien optimiert, beispielsweise seedSigma, dispersionCandidates oder corridorSigma. Die globalen Roadmap-Parameter bleiben dabei bewusst fixiert (tune_globals = False), um die Auswirkungen der einzelnen Strategien isoliert betrachten und ihre Unterschiede klar herausarbeiten zu können. Die Bewertung der Parameterkombinationen erfolgt anhand mehrerer Metriken, darunter die Erfolgsrate der Pfadplanung, die benötigte Laufzeit, die Anzahl der durchgeführten Kollisionsprüfungen, die resultierende Pfadlänge sowie die Größe der erzeugten Roadmap. Die Auswahl der jeweils besten Parameterkonfiguration erfolgt lexikographisch, wobei zunächst eine möglichst hohe Erfolgsrate priorisiert wird und bei gleicher Erfolgsrate sukzessive geringere Laufzeiten, weniger Kollisionsprüfungen und kürzere Pfade bevorzugt werden. Das Ergebnis des Parametertunings ist für jeden Modus eine robuste, empirisch bestimmte Parameterkonfiguration, die in den nachfolgenden Evaluationen verwendet wird.

In [ ]:
"""
# ============================================================
# PARAMETER TUNING PIPELINE (Updated for your CURRENT benchmarks + modes + parameters)
# ============================================================
# Drop-in section for Projekt_Aufgabe2_LazyPRM.ipynb
#
# Assumptions (matching your current notebook setup):
# - You already have: run_suite, EnhancedLazyPRM, and your benchmark factory functions defined.
# - Current modes + parameters:
#   baseline_uniform: (no extra params)
#   mode1_seed_gauss: seedSigma, seedTries
#   mode2_seed_dist:  seedMaxStep, seedBeta, seedTries
#   mode3_max_min:    dispersionCandidates
#   mode4_start_goal_corr: corridorSigma, corridorAlongSigma, corridorTries
# - You use ONLY your own benchmarks.
# ============================================================

import math
import numpy as np
import pandas as pd


# -----------------------------
# 1) Benchmarks (robust factory lookup)
# -----------------------------
def _first_existing_factory(factory_name_candidates):
    
    #Returns a callable from globals() if any name exists, else raises a clear error.
    #This makes the tuning block resilient to different function names in your notebook.
    
    for fn_name in factory_name_candidates:
        fn = globals().get(fn_name, None)
        if callable(fn):
            return fn
    raise NameError(
        "No benchmark factory found. Tried: "
        + ", ".join(factory_name_candidates)
        + "\nPlease adjust the candidate names to match your notebook."
    )


def build_all_benchmarks():
    benchmarks = [
        ("CircleField",      make_circle_field_benchmark()),
        ("TinyDoorClosed22", make_mode1_wall_tiny_door_closed22()),
        ("TinyWideDoor",     make_mode2_wall_tiny_wide_door()),
        ("U-Shape",          make_mode3_s_corridor_benchmark()),  # <- das IST dein U-Shape
        ("Snail",            make_mode4_snail()),
    ]

    out = []
    for bname, b in benchmarks:
        if hasattr(b, "name"):
            b.name = bname
        out.append(b)

    print("[build_all_benchmarks] loaded:", [b.name for b in out])
    return out



# -----------------------------
# 2) Sampling helpers
# -----------------------------
def _rand_int(rng, lo, hi):
    return int(rng.integers(lo, hi + 1))

def _rand_float(rng, lo, hi):
    return float(rng.uniform(lo, hi))

def _rand_loguniform(rng, lo, hi):
    lo = float(lo); hi = float(hi)
    if lo <= 0 or hi <= 0:
        raise ValueError("loguniform bounds must be > 0")
    return float(math.exp(rng.uniform(math.log(lo), math.log(hi))))


# -----------------------------
# 3) Search spaces
# -----------------------------
def global_space():
    
    #Global roadmap parameters. Keep these ranges moderate; your current setup
    #already uses small-ish roadmaps to make enhancement differences visible.
    
    return {
        "initialRoadmapSize": lambda rng: _rand_int(rng, 2, 12),
        "updateRoadmapSize":  lambda rng: _rand_int(rng, 2, 10),
        "kNearest":           lambda rng: _rand_int(rng, 2, 12),
        "maxIterations":      lambda rng: _rand_int(rng, 30, 120),
    }


def param_space_for_mode(mode_name):
    if mode_name == "baseline_uniform":
        return {}

    if mode_name == "mode1_seed_gauss":
        return {
            # Larger sigma => broader local exploration around seed nodes
            "seedSigma": lambda rng: _rand_loguniform(rng, 0.2, 8.0),
            # How many attempts to get a valid sample
            "seedTries": lambda rng: _rand_int(rng, 2, 20),
        }

    if mode_name == "mode2_seed_dist":
        return {
            # Max step size from a seed/anchor
            "seedMaxStep": lambda rng: _rand_loguniform(rng, 0.3, 8.0),
            # Bias/weighting parameter (clipped to [0,1])
            "seedBeta":    lambda rng: _rand_float(rng, 0.2, 1.0),
            # Attempts to obtain valid samples
            "seedTries":   lambda rng: _rand_int(rng, 2, 20),
        }

    if mode_name == "mode3_max_min":
        return {
            # Number of candidate samples for Max-Min selection per iteration
            "dispersionCandidates": lambda rng: _rand_int(rng, 5, 80),
        }

    if mode_name == "mode4_start_goal_corr":
        return {
            # Orthogonal corridor width around the start-goal line
            "corridorSigma":      lambda rng: _rand_loguniform(rng, 0.2, 6.0),
            # Spread along the start-goal direction
            "corridorAlongSigma": lambda rng: _rand_loguniform(rng, 0.05, 2.0),
            # Attempts to get a valid corridor sample
            "corridorTries":      lambda rng: _rand_int(rng, 2, 30),
        }

    raise ValueError(f"Unknown mode: {mode_name}")


def sample_config(base_config, mode_name, rng, tune_globals=True):
    
    #Creates one candidate configuration:
    #- copies base_config
    #- sets enhanceMode
    #- optionally tunes global roadmap parameters
    #- tunes the mode-specific parameters
    #- keeps robotRadius/safetyMargin untouched unless you explicitly include them in base_config changes
    
    cfg = dict(base_config)
    cfg["enhanceMode"] = mode_name

    if tune_globals:
        for k, sampler in global_space().items():
            cfg[k] = sampler(rng)

    for k, sampler in param_space_for_mode(mode_name).items():
        cfg[k] = sampler(rng)

    # Plausibility rules / clipping
    if mode_name == "mode2_seed_dist":
        cfg["seedBeta"] = float(np.clip(cfg["seedBeta"], 0.0, 1.0))
        cfg["seedTries"] = int(max(1, cfg["seedTries"]))
        cfg["seedMaxStep"] = float(max(1e-6, cfg["seedMaxStep"]))

    if mode_name == "mode1_seed_gauss":
        cfg["seedSigma"] = float(max(1e-6, cfg["seedSigma"]))
        cfg["seedTries"] = int(max(1, cfg["seedTries"]))

    if mode_name == "mode3_max_min":
        cfg["dispersionCandidates"] = int(max(1, cfg["dispersionCandidates"]))

    if mode_name == "mode4_start_goal_corr":
        cfg["corridorSigma"] = float(max(1e-6, cfg["corridorSigma"]))
        cfg["corridorAlongSigma"] = float(max(1e-6, cfg["corridorAlongSigma"]))
        cfg["corridorTries"] = int(max(1, cfg["corridorTries"]))

    # ensure ints for these globals if tuned
    for k in ("initialRoadmapSize", "updateRoadmapSize", "kNearest", "maxIterations"):
        if k in cfg:
            cfg[k] = int(cfg[k])

    return cfg


# -----------------------------
# 4) Aggregation + ranking
# -----------------------------
def summarize_df(df):
    
    #Expects df columns like run_suite:
    #benchmark, mode, run, success, time_s, collision_checks, path_length, roadmap_size
    
    df2 = df.copy()
    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df2.loc[df2["success"] == False, c] = np.nan

    g = df2.groupby(["candidate_id"]).agg(
        runs=("run", "count"),
        success_rate=("success", "mean"),
        time_mean=("time_s", "mean"),
        time_std=("time_s", "std"),
        collision_checks_mean=("collision_checks", "mean"),
        path_length_mean=("path_length", "mean"),
        roadmap_size_mean=("roadmap_size", "mean"),
    ).reset_index()

    return g


def pick_best(summary_df):
    
    #Lexicographic ranking:
      #1) success_rate max
      #2) time_mean min
      #3) collision_checks_mean min
      #4) path_length_mean min
    
    s = summary_df.copy()
    s["time_mean"] = s["time_mean"].fillna(np.inf)
    s["collision_checks_mean"] = s["collision_checks_mean"].fillna(np.inf)
    s["path_length_mean"] = s["path_length_mean"].fillna(np.inf)

    s = s.sort_values(
        by=["success_rate", "time_mean", "collision_checks_mean", "path_length_mean"],
        ascending=[False, True, True, True],
        kind="mergesort"
    )
    return s.iloc[0]


# -----------------------------
# 5) Stage A / Stage B tuning
# -----------------------------
def tune_mode(
    planner_class,
    benchmarks,
    base_config,
    mode_name,
    n_candidates_stageA=40,
    runs_stageA=6,
    top_k_stageB=6,
    runs_stageB=30,
    base_seed=2025,
    tune_globals=True,
    progress_every=50,
    verbose=True,
):
    rng = np.random.default_rng(base_seed + abs(hash(mode_name)) % 10_000)

    candidate_cfgs = {}
    all_rows_A = []

    if verbose:
        print(f"\n[Stage A] mode={mode_name} | candidates={n_candidates_stageA} | runs/cand={runs_stageA}")

    for cid in range(n_candidates_stageA):
        cfg = sample_config(base_config, mode_name, rng, tune_globals=tune_globals)
        candidate_cfgs[cid] = cfg

        bench_mode_map = {b.name: [mode_name] for b in benchmarks}
        configs = {mode_name: cfg}

        df = run_suite(
            benchmarks=benchmarks,
            configs=configs,
            runs=runs_stageA,
            base_seed=int(base_seed) + 10_000 * int(cid),
            progress_every=progress_every,
            bench_mode_map=bench_mode_map,
        )
        df["candidate_id"] = cid
        all_rows_A.append(df)

    dfA = pd.concat(all_rows_A, ignore_index=True)

    rankedA = summarize_df(dfA).sort_values(
        by=["success_rate", "time_mean", "collision_checks_mean", "path_length_mean"],
        ascending=[False, True, True, True],
        kind="mergesort"
    ).reset_index(drop=True)

    top_ids = list(rankedA["candidate_id"].head(top_k_stageB).values)

    if verbose:
        print(f"[Stage A] Top-{top_k_stageB}: {top_ids}")
        print(rankedA.head(min(10, len(rankedA))))

    # ---- Stage B
    all_rows_B = []
    if verbose:
        print(f"\n[Stage B] mode={mode_name} | confirm={len(top_ids)} | runs/cand={runs_stageB}")

    for cid in top_ids:
        cfg = candidate_cfgs[int(cid)]

        bench_mode_map = {b.name: [mode_name] for b in benchmarks}
        configs = {mode_name: cfg}

        df = run_suite(
            benchmarks=benchmarks,
            configs=configs,
            runs=runs_stageB,
            base_seed=int(base_seed) + 99_000 * int(cid),
            progress_every=progress_every,
            bench_mode_map=bench_mode_map,
        )
        df["candidate_id"] = int(cid)
        all_rows_B.append(df)

    dfB = pd.concat(all_rows_B, ignore_index=True)

    rankedB = summarize_df(dfB).sort_values(
        by=["success_rate", "time_mean", "collision_checks_mean", "path_length_mean"],
        ascending=[False, True, True, True],
        kind="mergesort"
    ).reset_index(drop=True)

    best_row = pick_best(rankedB)
    best_id = int(best_row["candidate_id"])
    best_cfg = candidate_cfgs[best_id]

    if verbose:
        print(f"\n[BEST] mode={mode_name} | best_candidate_id={best_id}")
        print("Best summary:", best_row.to_dict())
        print("Best config:", best_cfg)

    return {
        "mode": mode_name,
        "best_candidate_id": best_id,
        "best_config": best_cfg,
        "summary_stageA": rankedA,
        "summary_stageB": rankedB,
        "raw_stageA": dfA,
        "raw_stageB": dfB,
    }


def tune_all_modes(
    planner_class,
    base_config,
    base_seed=2025,
    tune_globals=True,
    n_candidates_stageA=50,
    runs_stageA=6,
    top_k_stageB=8,
    runs_stageB=30,
    progress_every=50,
    verbose=True,
):
    benchmarks = build_all_benchmarks()

    modes = [
        "baseline_uniform",
        "mode1_seed_gauss",
        "mode2_seed_dist",
        "mode3_max_min",
        "mode4_start_goal_corr",
    ]

    tuned = {}
    for m in modes:
        tuned[m] = tune_mode(
            planner_class=planner_class,
            benchmarks=benchmarks,
            base_config=base_config,
            mode_name=m,
            n_candidates_stageA=n_candidates_stageA,
            runs_stageA=runs_stageA,
            top_k_stageB=top_k_stageB,
            runs_stageB=runs_stageB,
            base_seed=base_seed,
            tune_globals=tune_globals,
            progress_every=progress_every,
            verbose=verbose,
        )

    best_configs = {m: tuned[m]["best_config"] for m in modes}
    return tuned, best_configs


# ============================================================
# RUN: Tune all modes (example)
# ============================================================
# Keep your robot parameters here too, so tuning uses the same collision model.
base_config = {
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      90,

    "robotRadius": 0.25,
    "safetyMargin": 0.05,
}

tuned, best_configs = tune_all_modes(
    planner_class=EnhancedLazyPRM,
    base_config=base_config,
    base_seed=2025,

    # Budget (adjust to runtime)
    n_candidates_stageA=50,
    runs_stageA=6,
    top_k_stageB=8,
    runs_stageB=30,

    # As in your notebook rationale: keep globals fixed if you want baseline weak + enhancements visible
    tune_globals=False,

    progress_every=50,
    verbose=True,
)

print("\nBest configs per mode:")
for mode, cfg in best_configs.items():
    print(f"\n--- {mode} ---")
    for k, v in cfg.items():
        print(f"{k}: {v}")
"""

# Darstellung und Evaluation der Node-Enhancementstrategien

In diesem Kapitel werden die implementierten Node-Enhancementstrategien vorgestellt und systematisch miteinander verglichen. Ziel ist es, die Auswirkungen der unterschiedlichen Sampling- und Erweiterungsmechanismen auf die Struktur der Roadmap sowie auf die Qualität und Effizienz der resultierenden Pfade zu analysieren. Als Referenz dient eine Baseline mit uniformem Sampling, gegenüber der gezielte, lokal- oder zielgerichtete Erweiterungsstrategien evaluiert werden. Die Darstellung erfolgt sowohl qualitativ anhand der erzeugten Roadmaps und Pfade als auch quantitativ mittels aggregierter Metriken aus den Benchmark-Experimenten.

Allen Varianten gemeinsam ist der Lazy-PRM-Grundgedanke: Kanten werden nicht sofort vollständig kollisionsgeprüft, sondern erst dann validiert, wenn ein konkreter Pfad zwischen Start und Ziel gesucht wird. Kanten, die dabei als kollidierend erkannt werden, sind aus der Roadmap zu entfernen. Unabhängig vom Modus wird zu Beginn eine initiale Roadmap erzeugt. Hier wird die initiale Roadmap mit nur zwei Knoten bewusst minimal gewählt, um den Effekt der Node-Enhancementstrategien beim schrittweisen Ausbau der Roadmap deutlich hervorzuheben. Diese initialen Knoten werden zufällig im Konfigurationsraum gesampelt; ungültige Samples welche sich außerhalb der Roadmap befinden werden verworfen, bis genügend gültige Startknoten vorliegen. Die Verknüpfung dieser Knoten erfolgt über kNearest welche gemäß Lazy-PRM erst bei Bedarf validiert werden.

**Mode 0 Baseline**

Der Mode Baseline erweitert die Roadmap während des gesamten Planungsprozesses durch uniformes, ungerichtetes Sampling. Neue Knoten werden zufällig im Konfigurationsraum erzeugt und in die Roadmap integriert; eine gezielte Nachbesserung an Stellen, an denen zuvor Kanten aufgrund von Kollisionen entfernt wurden, findet nicht statt. Dadurch dient die Baseline als Referenz, um den Nutzen gezielter Enhancementstrategien isoliert beurteilen zu können.

**Mode 1 Seed Gauss**

Mode 1 setzt auf lokale Verdichtung an den tatsächlich kritischen Stellen, die durch den Lazy-Mechanismus sichtbar werden. Sobald beim Pfadversuch eine Kante als kollidierend erkannt und gelöscht wird, wird aus ihrem Mittelpunkt ein Seedpoint gebildet. Um diese Seedpoints werden anschließend neue Knoten mittels Gauß-Sampling erzeugt, wodurch die Roadmap genau dort dichter wird, wo zuvor eine Verbindung fehlgeschlagen ist. Die Parameter steuern dabei im Kern, wie breit um den Seed gestreut wird und wie viele Versuche pro Seed unternommen werden, um gültige Samples zu erhalten. Mit dieser Strategie soll durch lokale zusätzliche Knoten, alternative kollisionsfreie Verbindungen in der Umgebung einer fehlgeschlagenen Kante ermöglicht werden. Parameter:

- seedSigma: Standardabweichung der Gauß-Verteilung um einen Seed-Knoten; größer = breiteres, explorativeres Sampling, kleiner = lokaler.
- seedTries: Anzahl Versuche, einen gültigen (in-bounds) Sample-Punkt zu erzeugen, bevor abgebrochen/neu gestartet wird.

**Mode 2 Seed Distance**

Mode 2 verwendet ebenfalls Seedpoints aus den Mittelpunkten gelöschter Kanten, unterscheidet sich jedoch in der Art, wie neue Knoten um diese Seeds erzeugt werden. Anstatt rein lokal um den Seed zu verdichten, werden neue Knoten in kontrollierter Distanz zum Seed erzeugt, sodass die Roadmap schrittweise aus dem Kollisionsbereich heraus in benachbarte Regionen erweitert werden kann. Für jeden neuen Knoten wird vom Seed aus eine zufällige Richtung im Konfigurationsraum gewählt. Durch begrenzte Schrittweiten und eine steuerbare Sampling-Tendenz sollen neue Kandidaten entstehen, die Umgehungen ermöglichen, ohne die Roadmap ausschließlich in unmittelbarer Seed-Nähe zu überkonzentrieren. Parameter:

- seedMaxStep: Maximale Schrittweite bzw. maximale Verschiebung von dem Seedpoint.
- seedBeta: Skaliert/limitiert die tatsächliche Schrittweite
- seedTries: Anzahl Versuche, einen gültigen (in-bounds) Sample-Punkt zu erzeugen, bevor abgebrochen/neu gestartet wird.

**Mode 3 Max-Min Sampling**

Mode 3 verfolgt ein explizit abdeckungsorientiertes Prinzip und nutzt ein Max-Min-Kriterium zur Auswahl neuer Knoten. Pro Erweiterungsschritt werden zunächst mehrere Kandidaten zufällig (typischerweise uniform) im Konfigurationsraum gesampelt. Für jeden Kandidaten wird der Abstand zum nächstgelegenen existierenden Roadmap-Knoten bestimmt. Aus allen Kandidaten wird dann der ausgewählt, dessen minimaler Abstand am größten ist. Dieses Vorgehen erhöht systematisch die Dispersion der Roadmap und verbessert die globale Abdeckung, wodurch alternative, bislang nicht erschlossene Umgehungsrouten wahrscheinlicher werden – insbesondere dann, wenn die bisherige Roadmap aufgrund gelöschter Kanten in bestimmten Bereichen keine tragfähige Verbindung herstellen konnte. Parameter:

- dispersionCandidates: Anzahl zufällig erzeugter Kandidaten pro Iteration.

**Mode 4 Start-Goal Corridor**

Mode 4 ist eine zielgerichtete Strategie, die das Sampling auf pfadrelevante Regionen fokussiert. Anstatt neue Knoten im gesamten Konfigurationsraum zu erzeugen, werden Kandidaten bevorzugt in einem Korridor entlang der Verbindung zwischen Start- und Zielkonfiguration gesampelt. Dazu wird typischerweise eine Position entlang der Start–Ziel-Richtung gewählt und anschließend eine orthogonale Abweichung hinzugefügt, sodass sich ein bandförmiger Sampling-Bereich um die Verbindungslinie ergibt. Die Streuung quer zur Achse bestimmt die Breite des Korridors, während die Streuung entlang der Achse die Verteilung der Samples in Richtung Start–Ziel beeinflusst; ungültige Kandidaten werden verworfen und bis zu einer festgelegten Anzahl von Versuchen neu erzeugt. In Kombination mit Lazy-PRM führt dieser Ansatz dazu, dass Kollisionsprüfungen weiterhin nur auf tatsächlich relevanten Kanten erfolgen, während die Roadmap-Erweiterung räumlich dort konzentriert wird, wo eine Start–Ziel-Verbindung am wahrscheinlichsten entsteht.

- corridorSigma: Querstreuung des Korridors um die Start–Ziel-Achse; größer = breiterer Korridor, kleiner = stärker fokussiert.
- corridorAlongSigma: Streuung entlang der Start–Ziel-Richtung; steuert, wie konzentriert entlang der Achse gesampelt wird.
- corridorTries: Anzahl Versuche, einen gültigen Sample innerhalb des Korridors zu finden.

## Evaluation mit Benchmark 1: Baseline_CircleField

Für den Benchmark CircleField wurden alle fünf Sampling-Modi (Baseline sowie Mode 1–4) jeweils mit 50 unabhängigen Planungsdurchläufen evaluiert. Die Balkendiagramme zeigen jeweils Mittelwert und Varianz der betrachteten Metriken über alle erfolgreichen Läufe. Die darunter dargestellten Roadmap- und Pfadvisualisierungen repräsentieren hingegen einen Best-Case pro Modus und dienen der qualitativen Veranschaulichung typischer Lösungsstrukturen.
Die Auswahl dieses Best-Cases erfolgt nicht anhand einer einzelnen Metrik, sondern über eine gewichtete Mehrkriterienbewertung. Dazu werden zunächst nur erfolgreiche Läufe berücksichtigt und extreme Ausreißer verworfen, indem die betrachteten Metriken auf ein zentrales Quantilintervall (z. B. 5 % bis 95 %) beschränkt werden. Anschließend wird für jeden verbleibenden Lauf ein kombinierter Score berechnet, der sich aus einer gewichteten Summe von Planungszeit, Pfadlänge und Roadmapgröße zusammensetzt. Die Gewichtung priorisiert dabei die Planungszeit am stärksten, gefolgt von der Pfadlänge und der Größe der Roadmap. Der Lauf mit dem minimalen Gesamtscore wird als Best-Case ausgewählt und grafisch dargestellt.
Wichtig ist, dass dieser Best-Case nicht repräsentativ für den Mittelwert der jeweiligen Metriken ist, sondern einen besonders günstigen Einzelverlauf zeigt. 

In [ ]:
BENCH_MODE_MAP = None

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


# ============================================================
# 1) Evaluation Helpers
# ============================================================
def path_length_from_graph(graph, path):
    if not path:
        return np.nan
    pts = [graph.nodes[p]["pos"] for p in path]
    return float(sum(
        np.linalg.norm(np.array(pts[i]) - np.array(pts[i + 1]))
        for i in range(len(pts) - 1)
    ))


# ------------------------------------------------------------
# Out-of-bounds => Collision hart machen (falls CC das nicht selbst tut)
# ------------------------------------------------------------
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        lims = _cc.getEnvironmentLimits()
        (xmin, xmax), (ymin, ymax) = lims
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# 2) Run helpers
# ============================================================
def run_one(planner_class, benchmark, config, seed=None):
    """
    Führt genau einen Run aus. Robust gegen Exceptions.
    Zusätzlich: zählt Seeds (Mode 1/2) und "Nodes aus Seeds" (Mode 1/2), damit wir
    später einen Plot-Run wählen können, in dem Seeds wirklich sichtbar sind.
    """
    if seed is not None:
        seed = int(seed)
        random.seed(seed)
        np.random.seed(seed)

    _ensure_oob_is_collision(benchmark.collisionChecker)

    planner = planner_class(benchmark.collisionChecker)

    # optionaler Counter reset
    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    t0 = time.time()
    err = None
    path = []
    try:
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
    except Exception as e:
        err = str(e)
        path = []
    t1 = time.time()

    collisions = getattr(benchmark.collisionChecker, "counter", np.nan)

    # Graph-Metriken
    try:
        size = len(planner.graph.nodes())
    except Exception:
        size = np.nan

    try:
        length = path_length_from_graph(planner.graph, path)
    except Exception:
        length = np.nan

    success = (path is not None) and (path != [])

    # --- NEW: Seeds und Seed-Nodes zählen ---
    seed_used_count = len(getattr(planner, "_seed_points_used", []) or [])
    sample_log = getattr(planner, "_sample_log", []) or []
    seed_sampled_nodes_count = sum(
        1 for r in sample_log
        if (r.get("seed_key") is not None) and (r.get("mode") in ("mode1_seed_gauss", "mode2_seed_dist"))
    )

    return {
        "time_s": (t1 - t0),
        "collision_checks": collisions,
        "path_length": length,
        "roadmap_size": size,
        "success": success,
        "error": err,
        # New fields (für Plot-Auswahl; in Summary ignorieren)
        "seed_used_count": int(seed_used_count),
        "seed_sampled_nodes_count": int(seed_sampled_nodes_count),
    }


def _modes_for_benchmark(bench_name, configs, bench_mode_map=None):
    if bench_mode_map is None:
        return list(configs.keys())
    return bench_mode_map.get(bench_name, list(configs.keys()))


def run_suite(benchmarks, configs, runs=30, base_seed=1234, progress_every=10, bench_mode_map=None):
    rows = []

    total = 0
    for b in benchmarks:
        total += len(_modes_for_benchmark(b.name, configs, bench_mode_map)) * runs

    done = 0
    t0 = time.time()

    for b in benchmarks:
        mode_list = _modes_for_benchmark(b.name, configs, bench_mode_map)

        for mode_name in mode_list:
            cfg = configs[mode_name]

            for r in range(runs):
                seed = base_seed + 1000 * r
                metrics = run_one(EnhancedLazyPRM, b, cfg, seed=seed)

                rows.append({
                    "benchmark": b.name,
                    "mode": mode_name,
                    "run": r,
                    "seed": seed,
                    **metrics
                })

                done += 1
                if progress_every and (done % progress_every == 0):
                    elapsed = time.time() - t0
                    print(f"{done}/{total} done, elapsed {elapsed:.1f}s")

    return pd.DataFrame(rows)


# ============================================================
# 3) Aggregation (Summary)
# ============================================================
def summarize(df):
    df_metrics = df.copy()

    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df_metrics.loc[df_metrics["success"] == False, c] = np.nan

    summary = (df_metrics
        .groupby(["benchmark", "mode"])
        .agg(
            runs=("run", "count"),
            success_rate=("success", "mean"),

            time_mean=("time_s", "mean"),
            time_std=("time_s", "std"),

            coll_mean=("collision_checks", "mean"),
            coll_std=("collision_checks", "std"),

            len_mean=("path_length", "mean"),
            len_std=("path_length", "std"),

            size_mean=("roadmap_size", "mean"),
            size_std=("roadmap_size", "std"),

            n_errors=("error", lambda s: int(s.notna().sum()))
        )
        .reset_index()
    )
    return summary


# ============================================================
# 4) Plot Layout (1×4)
# ============================================================
METRIC_SPECS = [
    ("time_mean", "time_std", "Planungszeit (s)", "gold", "Planungszeit (s)"),
    ("coll_mean", "coll_std", "Kollisionschecks", "tomato", "Kollisionschecks"),
    ("len_mean",  "len_std",  "Pfadlänge",        "cornflowerblue", "Pfadlänge"),
    ("size_mean", "size_std", "Roadmap Größe",    "mediumpurple", "Roadmap Größe (#Nodes)"),
]

def compute_global_ylims(summary_df, metric_specs, pad=0.10):
    ylims = {}
    for mean_col, std_col, *_ in metric_specs:
        mean_vals = summary_df[mean_col].to_numpy(dtype=float)
        std_vals  = summary_df[std_col].to_numpy(dtype=float)
        comb = mean_vals + np.nan_to_num(std_vals, nan=0.0)
        y_max = np.nanmax(comb) if np.any(np.isfinite(comb)) else 1.0
        if not np.isfinite(y_max) or y_max <= 0:
            y_max = 1.0
        ylims[mean_col] = (0.0, y_max * (1.0 + pad))
    return ylims

def plot_benchmark_like_task1(summary_df, benchmark_name, mode_order):
    sub = summary_df[summary_df["benchmark"] == benchmark_name].copy()

    present = [m for m in mode_order if m in set(sub["mode"].values)]
    if present:
        sub["mode"] = pd.Categorical(sub["mode"], categories=present, ordered=True)
        sub = sub.sort_values("mode")
    else:
        sub = sub.sort_values("mode")

    labels = sub["mode"].astype(str).tolist()
    x = np.arange(len(labels), dtype=int)

    fig, axs = plt.subplots(1, 4, figsize=(18, 4))
    fig.suptitle(f"Aufgabe 2: Vergleich Sampling-Modi – {benchmark_name}", fontsize=14)

    GLOBAL_YLIMS = compute_global_ylims(summary_df, METRIC_SPECS, pad=0.10)

    for ax, (mean_col, std_col, title, color, ylabel) in zip(axs, METRIC_SPECS):
        y = sub[mean_col].to_numpy(dtype=float)
        yerr = sub[std_col].to_numpy(dtype=float)
        yerr_plot = np.nan_to_num(yerr, nan=0.0)

        ax.bar(x, y, yerr=yerr_plot, capsize=4, color=color, alpha=0.85)
        ax.set_title(title)
        ax.set_xlabel("Sampling-Modus")
        ax.set_ylabel(ylabel)
        ax.set_ylim(*GLOBAL_YLIMS[mean_col])
        ax.grid(axis="y", alpha=0.3)
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=25, ha="right")

    plt.tight_layout()
    plt.show()


# ============================================================
# 5) Best-Plot pro Mode & Benchmark (mit Sampling-Struktur)
# ============================================================
def _robust_minmax_norm(series, q_low=0.05, q_high=0.95):
    x = series.to_numpy(dtype=float)
    if np.all(~np.isfinite(x)):
        return pd.Series(np.zeros(len(series)), index=series.index)

    lo = np.nanquantile(x, q_low)
    hi = np.nanquantile(x, q_high)
    if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
        return pd.Series(np.zeros(len(series)), index=series.index)

    xn = (series - lo) / (hi - lo)
    xn = xn.clip(0.0, 1.0).fillna(1.0)
    return xn





def _pick_best_run_multicriteria(df_succ, w_time=0.6, w_len=0.25, w_size=0.15,
                                 q_low=0.05, q_high=0.95):
    d = df_succ.copy()

    for col in ["time_s", "path_length", "roadmap_size"]:
        if col not in d.columns:
            d[col] = np.nan

    d["time_n"] = _robust_minmax_norm(d["time_s"], q_low=q_low, q_high=q_high)
    d["len_n"]  = _robust_minmax_norm(d["path_length"], q_low=q_low, q_high=q_high)
    d["size_n"] = _robust_minmax_norm(d["roadmap_size"], q_low=q_low, q_high=q_high)
    d["score"] = w_time * d["time_n"] + w_len * d["len_n"] + w_size * d["size_n"]

    d = d.sort_values(["score", "time_s"], ascending=[True, True])
    return d.iloc[0]


def extract_best_seed_map(
    df,
    prefer_seed_runs_for_mode12=True,
    w_time=0.6, w_len=0.25, w_size=0.15
):
    """
    Gibt dict zurück: seed_map[(benchmark_name, mode_name)] = seed
    verwendet die exakt gleiche Auswahlregel wie visualize_best_per_mode_and_benchmark().
    """
    seed_map = {}

    for (bench_name, mode), df_bm in df.groupby(["benchmark", "mode"]):
        df_bm = df_bm.copy()
        df_succ = df_bm[df_bm["success"] == True].copy()
        if df_succ.empty:
            continue

        df_pick = df_succ
        if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
            df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
            if not df_seed.empty:
                df_pick = df_seed

        row = _pick_best_run_multicriteria(
            df_pick, w_time=w_time, w_len=w_len, w_size=w_size
        )
        seed_map[(bench_name, mode)] = int(row["seed"])

    return seed_map


def visualize_best_per_mode_and_benchmark(
    df, benchmarks, configs, nodeSize=80,
    mode_order=None, bench_order=None, figsize=(7, 7),
    w_time=0.6, w_len=0.25, w_size=0.15,
    prefer_seed_runs_for_mode12=True,
):
    """
    Plot-Regel:
    - Mode 1/2: zeige Seed (X) + Nodes-aus-Seed + Seed->Node-Linien,
      aber NUR wenn Seeds tatsächlich benutzt wurden.
      Falls prefer_seed_runs_for_mode12=True, wird ein erfolgreicher Run bevorzugt,
      der seed_used_count > 0 hat (sonst sieht man sonst oft "keine Seeds", weil
      der Planner zufällig direkt eine kollisionsfreie Lösung findet).
    - Mode 3: zeige Kandidatenwolke + selected (Stern) aus dem Planner-Log.
    - Mode 4: zeige Start–Goal-Achse + Korridor-Nodes.
    - Baseline: keine Seeds, keine Hotspots.
    """
    bench_by_name = {b.name: b for b in benchmarks}

    if bench_order is None:
        bench_order = [b.name for b in benchmarks]
    if mode_order is None:
        mode_order = list(configs.keys())

    for bench_name in bench_order:
        bench = bench_by_name[bench_name]

        modes_present = [m for m in mode_order if ((df["benchmark"] == bench_name) & (df["mode"] == m)).any()]
        for mode in modes_present:
            df_bm = df[(df["benchmark"] == bench_name) & (df["mode"] == mode)].copy()
            if df_bm.empty:
                continue

            df_succ = df_bm[df_bm["success"] == True].copy()

            # --- Auswahlregel: Mode1/2 bevorzugt Run mit Seeds ---
            df_pick = df_succ
            if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
                if not df_seed.empty:
                    df_pick = df_seed  # nur wenn es wirklich solche Runs gibt

            if not df_pick.empty:
                row = _pick_best_run_multicriteria(df_pick, w_time=w_time, w_len=w_len, w_size=w_size)
                tag = f"BEST (score={row['score']:.3f}; w_t={w_time}, w_l={w_len}, w_s={w_size})"
            else:
                row = df_bm.sort_values("run").iloc[-1]
                tag = "FALLBACK (no success)"

            run_idx = int(row["run"])
            seed = int(row["seed"])

            random.seed(seed)
            np.random.seed(seed)
            _ensure_oob_is_collision(bench.collisionChecker)

            planner = EnhancedLazyPRM(bench.collisionChecker)
            try:
                sol = planner.planPath(bench.startList, bench.goalList, configs[mode])
            except Exception as e:
                sol = []
                tag = f"{tag} – ERROR: {e}"

            fig, ax = plt.subplots(figsize=figsize)
            lazyPRMVisualize(planner, sol, ax=ax, nodeSize=nodeSize)

            # Achsen fixieren
            (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect("equal", adjustable="box")

            pos = nx.get_node_attributes(planner.graph, "pos")

            # ------------------------------------------------------------
            # MODE 1/2: Seed + Nodes-aus-Seed + Seed->Node Links
            # ------------------------------------------------------------
            if mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                seeds = getattr(planner, "_seed_points_used", []) or []
                sample_log = getattr(planner, "_sample_log", []) or []

                # Seeds plotten (nur wenn wirklich genutzt)
                if seeds:
                    sx = [p[0] for p in seeds]
                    sy = [p[1] for p in seeds]
                    ax.scatter(
                        sx, sy,
                        marker="x",
                        s=max(70, int(1.3 * nodeSize)),
                        linewidths=2.5,
                        zorder=30,
                        label="Seed used (Mode 1/2)"
                    )

                    # Nodes, die aus Seeds erzeugt wurden (seed_key != None)
                    seed_nodes = []
                    seed_links = []  # (seed_point, node_point)

                    # Mapping seed_key -> seed_point
                    seed_map = {}
                    for sp in seeds:
                        k = (round(float(sp[0]), 3), round(float(sp[1]), 3))
                        seed_map[k] = [float(sp[0]), float(sp[1])]

                    for r in sample_log:
                        if r.get("mode") != mode:
                            continue
                        k = r.get("seed_key", None)
                        if k is None:
                            continue
                        nid = int(r["node_id"])
                        if nid not in pos:
                            continue
                        seed_nodes.append(nid)
                        sp = seed_map.get(k, r.get("seed", None))
                        if sp is not None:
                            seed_links.append((sp, pos[nid]))

                    if seed_nodes:
                        nxs = [pos[n][0] for n in seed_nodes]
                        nys = [pos[n][1] for n in seed_nodes]
                        ax.scatter(
                            nxs, nys,
                            marker="o",
                            s=max(35, int(0.8 * nodeSize)),
                            edgecolors="black",
                            linewidths=1.2,
                            zorder=29,
                            label="Nodes sampled from seed"
                        )
                        # Linien Seed->Node
                        for sp, npnt in seed_links:
                            ax.plot(
                                [sp[0], npnt[0]],
                                [sp[1], npnt[1]],
                                linewidth=1.0,
                                alpha=0.6,
                                zorder=28
                            )

            # ------------------------------------------------------------
            # MODE 3: Max–Min – Kandidatenwolke + selected (letzter Schritt)
            # ------------------------------------------------------------
            if mode == "mode3_max_min":
                cand_log = getattr(planner, "_mode3_candidates_log", []) or []
                if cand_log:
                    last = cand_log[-1]
                    cands = last.get("candidates", []) or []
                    sel = last.get("selected", None)

                    if cands:
                        cx = [c[0] for c in cands]
                        cy = [c[1] for c in cands]
                        ax.scatter(
                            cx, cy,
                            marker="o",
                            s=18,
                            facecolors="none",
                            linewidths=1.0,
                            zorder=20,
                            label="Max–Min candidates"
                        )

                    if sel is not None:
                        ax.scatter(
                            [sel[0]], [sel[1]],
                            marker="*",
                            s=200,
                            zorder=21,
                            label="Max–Min selected"
                        )

            # ------------------------------------------------------------
            # MODE 4: Start–Goal Corridor – Achse + Korridor-Nodes
            # ------------------------------------------------------------
            if mode == "mode4_start_goal_corr":
                s = np.array(bench.startList[0], dtype=float)
                g = np.array(bench.goalList[0], dtype=float)

                ax.plot(
                    [s[0], g[0]], [s[1], g[1]],
                    linestyle="--",
                    linewidth=2,
                    alpha=0.8,
                    zorder=18,
                    label="Start–Goal corridor axis"
                )

                sample_log = getattr(planner, "_sample_log", []) or []
                corr_nodes = []
                for r in sample_log:
                    if r.get("mode") != mode:
                        continue
                    if r.get("source") != "start_goal_corridor":
                        continue
                    nid = int(r["node_id"])
                    if nid in pos:
                        corr_nodes.append(nid)

                if corr_nodes:
                    xs = [pos[n][0] for n in corr_nodes]
                    ys = [pos[n][1] for n in corr_nodes]
                    ax.scatter(
                        xs, ys,
                        marker="^",
                        s=max(25, int(0.7 * nodeSize)),
                        zorder=19,
                        label="Corridor-sampled nodes"
                    )

            # Legend nur wenn extra Elemente vorhanden
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc="upper right")

            # Titel
            t = float(row["time_s"]) if pd.notna(row.get("time_s", np.nan)) else np.nan
            L = float(row["path_length"]) if pd.notna(row.get("path_length", np.nan)) else np.nan
            S = float(row["roadmap_size"]) if pd.notna(row.get("roadmap_size", np.nan)) else np.nan
            ax.set_title(
                f"{bench_name} – {mode}\n"
                f"{tag}: run={run_idx}, time={t:.3f}s, len={L:.2f}, size={S:.0f}, seed={seed}"
            )

            plt.tight_layout()
            plt.show()


# ============================================================
# RUN + PLOTS
# ============================================================
benchmarks = [bench_baseline]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)
summary = summarize(df)

# Seeds aus denselben Evaluation-Ergebnissen ziehen
SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)

# 1) Summary-Plots (1×4)
for b in benchmarks:
     plot_benchmark_like_task1(summary, b.name, mode_order=MODE_ORDER)

# 2) Best-Run Visualisierung (Sampling-Struktur sichtbar)
BENCH_ORDER = [b.name for b in benchmarks]
visualize_best_per_mode_and_benchmark(
     df, benchmarks, configs,
     nodeSize=80,
     mode_order=MODE_ORDER,
     bench_order=BENCH_ORDER,
     figsize=(7, 7),
     prefer_seed_runs_for_mode12=True,
 )


### Diskussion der Ergebnisse

**Planungszeit**

Die Planungszeit wird maßgeblich durch den internen Aufwand der jeweiligen Erweiterungsstrategie bestimmt. Mode 3 (Max-Min) weist die höchsten mittleren Planungszeiten und die größte Varianz auf. Dies ist eine direkte Folge der Strategie, pro Iteration mehrere Kandidaten zu erzeugen und deren Abstände zur bestehenden Roadmap zu bewerten. Der zusätzliche Rechenaufwand wirkt sich unmittelbar auf die Laufzeit aus. Die Baseline, Mode 1, Mode 2 und Mode 4 liegen zeitlich eng beieinander. Bei Mode 2 führt das distanzbasierte Seed-Sampling zu einer effizienten Erweiterung der Roadmap mit geringem Zusatzaufwand. Mode 4 verursacht trotz zusätzlicher Strukturierung entlang des Start–Ziel-Korridors keine signifikant höheren Planungszeiten, da die Anzahl der erzeugten Knoten begrenzt bleibt.

**Kollisionschecks**

Die Anzahl der Kollisionschecks spiegelt wider, wie stark ein Verfahren in potenziell problematische Bereiche des Konfigurationsraums expandiert. Mode 3 verursacht die meisten Kollisionsprüfungen, da gezielt Regionen mit großem Abstand zur bestehenden Roadmap erschlossen werden, die häufig nahe an Hindernissen liegen. Mode 2 erreicht die geringste Anzahl an Kollisionschecks, da neue Knoten bevorzugt in kontrollierter Distanz um bestehende Seeds erzeugt werden. Dadurch werden ungünstige Kandidaten frühzeitig vermieden. Mode 4 liegt leicht unterhalb der Baseline, da das Sampling entlang des Start–Ziel-Korridors verstärkt hindernisnahe Bereiche berücksichtigt, ohne jedoch die starke Exploration von Mode 3 zu erreichen.

**Pfadlänge**

Die Pfadlängen unterscheiden sich nur moderat zwischen den Modi, was auf die Struktur des CircleField-Benchmarks mit mehreren gleichwertigen Durchgängen hinweist. Die Baseline erzeugt im Mittel die längsten Pfade, da keine gezielte Verdichtung entlang potenziell günstiger Routen erfolgt. Mode 1 und Mode 2 profitieren von der lokalen Nachbesserung in bereits vielversprechenden Bereichen, wodurch kürzere und direktere Verbindungen entstehen. Mode 4 erzielt ebenfalls kurze Pfade, da der Start–Ziel-Korridor bevorzugt Knoten entlang einer groben Verbindungslinie zwischen Start und Ziel erzeugt. Mode 3 erreicht trotz sehr kleiner Roadmap konkurrenzfähige Pfadlängen, was auf eine effektive globale Abdeckung des Raums hindeutet.

**Roadmapgröße**

Die Roadmapgröße zeigt deutliche Unterschiede in der Effizienz der Erweiterungsstrategien. Mode 3 erzeugt mit Abstand die kleinste Roadmap, da pro Iteration nur ein einzelner Knoten mit maximalem Mindestabstand zur bestehenden Struktur hinzugefügt wird. Dies führt zu einer sparsamen, aber gut verteilten Abdeckung des Konfigurationsraums. Mode 2 und Mode 1 erzeugen größere Roadmaps, da um jeden Seed mehrere lokale Knoten ergänzt werden, um die Konnektivität gezielt zu verbessern. Mode 4 liegt in der Roadmapgröße auf einem ähnlichen Niveau wie die Baseline. Der Start–Ziel-Korridor führt in diesem Benchmark nicht zu einer starken Reduktion der benötigten Knoten, da der freie Raum viele alternative Routen zulässt.

**Gesamtbewertung und Eignung für den Benchmark**

Der CircleField-Benchmark stellt geringe Anforderungen an eine explizite Start–Ziel-Führung und begünstigt Verfahren mit effizienter Raumabdeckung. Mode 3 ist besonders speichereffizient und erreicht mit sehr wenigen Knoten kurze Pfade, erkauft sich dies jedoch durch höhere Planungszeiten und viele Kollisionschecks. Mode 2 stellt den ausgewogensten Kompromiss dar, da es kurze Pfade, geringe Kollisionschecks und moderate Planungszeiten kombiniert, allerdings auf Kosten einer größeren Roadmap. Mode 4 zeigt in diesem Benchmark kein klares Alleinstellungsmerkmal gegenüber der Baseline. Die Ergebnisse verdeutlichen, dass im CircleField Strategien mit globaler Abdeckung oder moderater Seed-Nutzung gegenüber rein uniformem Sampling Vorteile bieten, während eine starke Start–Ziel-Fokussierung keinen entscheidenden Mehrwert liefert.

## Evaluation mit Benchmark 2: WallTinyDoor

Der Benchmark WallTinyDoor stellt eine klassische Engstellen-Situation dar: Start und Ziel sind durch eine nahezu geschlossene Wand getrennt, die nur eine sehr schmale Passage erlaubt. Alle fünf Sampling-Modi wurden jeweils mit 50 unabhängigen Planungsdurchläufen evaluiert. Die Balkendiagramme zeigen erneut Mittelwert und Varianz der betrachteten Metriken über alle erfolgreichen Läufe, während die darunter dargestellten Roadmap- und Pfadvisualisierungen jeweils einen Best-Case repräsentieren. Dieser Best-Case wird über eine gewichtete Mehrkriterienbewertung aus Planungszeit, Pfadlänge und Roadmapgröße bestimmt und dient der qualitativen Veranschaulichung typischer Lösungsstrukturen, nicht der Darstellung eines Mittelwerts.

In [ ]:
BENCH_MODE_MAP = None

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


# ============================================================
# 1) Evaluation Helpers
# ============================================================
def path_length_from_graph(graph, path):
    if not path:
        return np.nan
    pts = [graph.nodes[p]["pos"] for p in path]
    return float(sum(
        np.linalg.norm(np.array(pts[i]) - np.array(pts[i + 1]))
        for i in range(len(pts) - 1)
    ))


# ------------------------------------------------------------
# Out-of-bounds => Collision hart machen (falls CC das nicht selbst tut)
# ------------------------------------------------------------
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        lims = _cc.getEnvironmentLimits()
        (xmin, xmax), (ymin, ymax) = lims
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# 2) Run helpers
# ============================================================
def run_one(planner_class, benchmark, config, seed=None):
    """
    Führt genau einen Run aus. Robust gegen Exceptions.
    Zusätzlich: zählt Seeds (Mode 1/2) und "Nodes aus Seeds" (Mode 1/2), damit wir
    später einen Plot-Run wählen können, in dem Seeds wirklich sichtbar sind.
    """
    if seed is not None:
        seed = int(seed)
        random.seed(seed)
        np.random.seed(seed)

    _ensure_oob_is_collision(benchmark.collisionChecker)

    planner = planner_class(benchmark.collisionChecker)

    # optionaler Counter reset
    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    t0 = time.time()
    err = None
    path = []
    try:
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
    except Exception as e:
        err = str(e)
        path = []
    t1 = time.time()

    collisions = getattr(benchmark.collisionChecker, "counter", np.nan)

    # Graph-Metriken
    try:
        size = len(planner.graph.nodes())
    except Exception:
        size = np.nan

    try:
        length = path_length_from_graph(planner.graph, path)
    except Exception:
        length = np.nan

    success = (path is not None) and (path != [])

    # --- NEW: Seeds und Seed-Nodes zählen ---
    seed_used_count = len(getattr(planner, "_seed_points_used", []) or [])
    sample_log = getattr(planner, "_sample_log", []) or []
    seed_sampled_nodes_count = sum(
        1 for r in sample_log
        if (r.get("seed_key") is not None) and (r.get("mode") in ("mode1_seed_gauss", "mode2_seed_dist"))
    )

    return {
        "time_s": (t1 - t0),
        "collision_checks": collisions,
        "path_length": length,
        "roadmap_size": size,
        "success": success,
        "error": err,
        # New fields (für Plot-Auswahl; in Summary ignorieren)
        "seed_used_count": int(seed_used_count),
        "seed_sampled_nodes_count": int(seed_sampled_nodes_count),
    }


def _modes_for_benchmark(bench_name, configs, bench_mode_map=None):
    if bench_mode_map is None:
        return list(configs.keys())
    return bench_mode_map.get(bench_name, list(configs.keys()))


def run_suite(benchmarks, configs, runs=30, base_seed=1234, progress_every=10, bench_mode_map=None):
    rows = []

    total = 0
    for b in benchmarks:
        total += len(_modes_for_benchmark(b.name, configs, bench_mode_map)) * runs

    done = 0
    t0 = time.time()

    for b in benchmarks:
        mode_list = _modes_for_benchmark(b.name, configs, bench_mode_map)

        for mode_name in mode_list:
            cfg = configs[mode_name]

            for r in range(runs):
                seed = base_seed + 1000 * r
                metrics = run_one(EnhancedLazyPRM, b, cfg, seed=seed)

                rows.append({
                    "benchmark": b.name,
                    "mode": mode_name,
                    "run": r,
                    "seed": seed,
                    **metrics
                })

                done += 1
                if progress_every and (done % progress_every == 0):
                    elapsed = time.time() - t0
                    print(f"{done}/{total} done, elapsed {elapsed:.1f}s")

    return pd.DataFrame(rows)


# ============================================================
# 3) Aggregation (Summary)
# ============================================================
def summarize(df):
    df_metrics = df.copy()

    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df_metrics.loc[df_metrics["success"] == False, c] = np.nan

    summary = (df_metrics
        .groupby(["benchmark", "mode"])
        .agg(
            runs=("run", "count"),
            success_rate=("success", "mean"),

            time_mean=("time_s", "mean"),
            time_std=("time_s", "std"),

            coll_mean=("collision_checks", "mean"),
            coll_std=("collision_checks", "std"),

            len_mean=("path_length", "mean"),
            len_std=("path_length", "std"),

            size_mean=("roadmap_size", "mean"),
            size_std=("roadmap_size", "std"),

            n_errors=("error", lambda s: int(s.notna().sum()))
        )
        .reset_index()
    )
    return summary


# ============================================================
# 4) Plot Layout (1×4)
# ============================================================
METRIC_SPECS = [
    ("time_mean", "time_std", "Planungszeit (s)", "gold", "Planungszeit (s)"),
    ("coll_mean", "coll_std", "Kollisionschecks", "tomato", "Kollisionschecks"),
    ("len_mean",  "len_std",  "Pfadlänge",        "cornflowerblue", "Pfadlänge"),
    ("size_mean", "size_std", "Roadmap Größe",    "mediumpurple", "Roadmap Größe (#Nodes)"),
]

def compute_global_ylims(summary_df, metric_specs, pad=0.10):
    ylims = {}
    for mean_col, std_col, *_ in metric_specs:
        mean_vals = summary_df[mean_col].to_numpy(dtype=float)
        std_vals  = summary_df[std_col].to_numpy(dtype=float)
        comb = mean_vals + np.nan_to_num(std_vals, nan=0.0)
        y_max = np.nanmax(comb) if np.any(np.isfinite(comb)) else 1.0
        if not np.isfinite(y_max) or y_max <= 0:
            y_max = 1.0
        ylims[mean_col] = (0.0, y_max * (1.0 + pad))
    return ylims

def plot_benchmark_like_task1(summary_df, benchmark_name, mode_order):
    sub = summary_df[summary_df["benchmark"] == benchmark_name].copy()

    present = [m for m in mode_order if m in set(sub["mode"].values)]
    if present:
        sub["mode"] = pd.Categorical(sub["mode"], categories=present, ordered=True)
        sub = sub.sort_values("mode")
    else:
        sub = sub.sort_values("mode")

    labels = sub["mode"].astype(str).tolist()
    x = np.arange(len(labels), dtype=int)

    fig, axs = plt.subplots(1, 4, figsize=(18, 4))
    fig.suptitle(f"Aufgabe 2: Vergleich Sampling-Modi – {benchmark_name}", fontsize=14)

    GLOBAL_YLIMS = compute_global_ylims(summary_df, METRIC_SPECS, pad=0.10)

    for ax, (mean_col, std_col, title, color, ylabel) in zip(axs, METRIC_SPECS):
        y = sub[mean_col].to_numpy(dtype=float)
        yerr = sub[std_col].to_numpy(dtype=float)
        yerr_plot = np.nan_to_num(yerr, nan=0.0)

        ax.bar(x, y, yerr=yerr_plot, capsize=4, color=color, alpha=0.85)
        ax.set_title(title)
        ax.set_xlabel("Sampling-Modus")
        ax.set_ylabel(ylabel)
        ax.set_ylim(*GLOBAL_YLIMS[mean_col])
        ax.grid(axis="y", alpha=0.3)
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=25, ha="right")

    plt.tight_layout()
    plt.show()


# ============================================================
# 5) Best-Plot pro Mode & Benchmark (mit Sampling-Struktur)
# ============================================================
def _robust_minmax_norm(series, q_low=0.05, q_high=0.95):
    x = series.to_numpy(dtype=float)
    if np.all(~np.isfinite(x)):
        return pd.Series(np.zeros(len(series)), index=series.index)

    lo = np.nanquantile(x, q_low)
    hi = np.nanquantile(x, q_high)
    if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
        return pd.Series(np.zeros(len(series)), index=series.index)

    xn = (series - lo) / (hi - lo)
    xn = xn.clip(0.0, 1.0).fillna(1.0)
    return xn


def _pick_best_run_multicriteria(df_succ, w_time=0.6, w_len=0.25, w_size=0.15,
                                 q_low=0.05, q_high=0.95):
    d = df_succ.copy()

    for col in ["time_s", "path_length", "roadmap_size"]:
        if col not in d.columns:
            d[col] = np.nan

    d["time_n"] = _robust_minmax_norm(d["time_s"], q_low=q_low, q_high=q_high)
    d["len_n"]  = _robust_minmax_norm(d["path_length"], q_low=q_low, q_high=q_high)
    d["size_n"] = _robust_minmax_norm(d["roadmap_size"], q_low=q_low, q_high=q_high)
    d["score"] = w_time * d["time_n"] + w_len * d["len_n"] + w_size * d["size_n"]

    d = d.sort_values(["score", "time_s"], ascending=[True, True])
    return d.iloc[0]


def extract_best_seed_map(
    df,
    prefer_seed_runs_for_mode12=True,
    w_time=0.6, w_len=0.25, w_size=0.15
):
    """
    Gibt dict zurück: seed_map[(benchmark_name, mode_name)] = seed
    verwendet die exakt gleiche Auswahlregel wie visualize_best_per_mode_and_benchmark().
    """
    seed_map = {}

    for (bench_name, mode), df_bm in df.groupby(["benchmark", "mode"]):
        df_bm = df_bm.copy()
        df_succ = df_bm[df_bm["success"] == True].copy()
        if df_succ.empty:
            continue

        df_pick = df_succ
        if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
            df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
            if not df_seed.empty:
                df_pick = df_seed

        row = _pick_best_run_multicriteria(
            df_pick, w_time=w_time, w_len=w_len, w_size=w_size
        )
        seed_map[(bench_name, mode)] = int(row["seed"])

    return seed_map

def visualize_best_per_mode_and_benchmark(
    df, benchmarks, configs, nodeSize=80,
    mode_order=None, bench_order=None, figsize=(7, 7),
    w_time=0.6, w_len=0.25, w_size=0.15,
    prefer_seed_runs_for_mode12=True,
):
    """
    Plot-Regel:
    - Mode 1/2: zeige Seed (X) + Nodes-aus-Seed + Seed->Node-Linien,
      aber NUR wenn Seeds tatsächlich benutzt wurden.
      Falls prefer_seed_runs_for_mode12=True, wird ein erfolgreicher Run bevorzugt,
      der seed_used_count > 0 hat (sonst sieht man sonst oft "keine Seeds", weil
      der Planner zufällig direkt eine kollisionsfreie Lösung findet).
    - Mode 3: zeige Kandidatenwolke + selected (Stern) aus dem Planner-Log.
    - Mode 4: zeige Start–Goal-Achse + Korridor-Nodes.
    - Baseline: keine Seeds, keine Hotspots.
    """
    bench_by_name = {b.name: b for b in benchmarks}

    if bench_order is None:
        bench_order = [b.name for b in benchmarks]
    if mode_order is None:
        mode_order = list(configs.keys())

    for bench_name in bench_order:
        bench = bench_by_name[bench_name]

        modes_present = [m for m in mode_order if ((df["benchmark"] == bench_name) & (df["mode"] == m)).any()]
        for mode in modes_present:
            df_bm = df[(df["benchmark"] == bench_name) & (df["mode"] == mode)].copy()
            if df_bm.empty:
                continue

            df_succ = df_bm[df_bm["success"] == True].copy()

            # --- Auswahlregel: Mode1/2 bevorzugt Run mit Seeds ---
            df_pick = df_succ
            if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
                if not df_seed.empty:
                    df_pick = df_seed  # nur wenn es wirklich solche Runs gibt

            if not df_pick.empty:
                row = _pick_best_run_multicriteria(df_pick, w_time=w_time, w_len=w_len, w_size=w_size)
                tag = f"BEST (score={row['score']:.3f}; w_t={w_time}, w_l={w_len}, w_s={w_size})"
            else:
                row = df_bm.sort_values("run").iloc[-1]
                tag = "FALLBACK (no success)"

            run_idx = int(row["run"])
            seed = int(row["seed"])

            random.seed(seed)
            np.random.seed(seed)
            _ensure_oob_is_collision(bench.collisionChecker)

            planner = EnhancedLazyPRM(bench.collisionChecker)
            try:
                sol = planner.planPath(bench.startList, bench.goalList, configs[mode])
            except Exception as e:
                sol = []
                tag = f"{tag} – ERROR: {e}"

            fig, ax = plt.subplots(figsize=figsize)
            lazyPRMVisualize(planner, sol, ax=ax, nodeSize=nodeSize)

            # Achsen fixieren
            (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect("equal", adjustable="box")

            pos = nx.get_node_attributes(planner.graph, "pos")

            # ------------------------------------------------------------
            # MODE 1/2: Seed + Nodes-aus-Seed + Seed->Node Links
            # ------------------------------------------------------------
            if mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                seeds = getattr(planner, "_seed_points_used", []) or []
                sample_log = getattr(planner, "_sample_log", []) or []

                # Seeds plotten (nur wenn wirklich genutzt)
                if seeds:
                    sx = [p[0] for p in seeds]
                    sy = [p[1] for p in seeds]
                    ax.scatter(
                        sx, sy,
                        marker="x",
                        s=max(70, int(1.3 * nodeSize)),
                        linewidths=2.5,
                        zorder=30,
                        label="Seed used (Mode 1/2)"
                    )

                    # Nodes, die aus Seeds erzeugt wurden (seed_key != None)
                    seed_nodes = []
                    seed_links = []  # (seed_point, node_point)

                    # Mapping seed_key -> seed_point
                    seed_map = {}
                    for sp in seeds:
                        k = (round(float(sp[0]), 3), round(float(sp[1]), 3))
                        seed_map[k] = [float(sp[0]), float(sp[1])]

                    for r in sample_log:
                        if r.get("mode") != mode:
                            continue
                        k = r.get("seed_key", None)
                        if k is None:
                            continue
                        nid = int(r["node_id"])
                        if nid not in pos:
                            continue
                        seed_nodes.append(nid)
                        sp = seed_map.get(k, r.get("seed", None))
                        if sp is not None:
                            seed_links.append((sp, pos[nid]))

                    if seed_nodes:
                        nxs = [pos[n][0] for n in seed_nodes]
                        nys = [pos[n][1] for n in seed_nodes]
                        ax.scatter(
                            nxs, nys,
                            marker="o",
                            s=max(35, int(0.8 * nodeSize)),
                            edgecolors="black",
                            linewidths=1.2,
                            zorder=29,
                            label="Nodes sampled from seed"
                        )
                        # Linien Seed->Node
                        for sp, npnt in seed_links:
                            ax.plot(
                                [sp[0], npnt[0]],
                                [sp[1], npnt[1]],
                                linewidth=1.0,
                                alpha=0.6,
                                zorder=28
                            )

            # ------------------------------------------------------------
            # MODE 3: Max–Min – Kandidatenwolke + selected (letzter Schritt)
            # ------------------------------------------------------------
            if mode == "mode3_max_min":
                cand_log = getattr(planner, "_mode3_candidates_log", []) or []
                if cand_log:
                    last = cand_log[-1]
                    cands = last.get("candidates", []) or []
                    sel = last.get("selected", None)

                    if cands:
                        cx = [c[0] for c in cands]
                        cy = [c[1] for c in cands]
                        ax.scatter(
                            cx, cy,
                            marker="o",
                            s=18,
                            facecolors="none",
                            linewidths=1.0,
                            zorder=20,
                            label="Max–Min candidates"
                        )

                    if sel is not None:
                        ax.scatter(
                            [sel[0]], [sel[1]],
                            marker="*",
                            s=200,
                            zorder=21,
                            label="Max–Min selected"
                        )

            # ------------------------------------------------------------
            # MODE 4: Start–Goal Corridor – Achse + Korridor-Nodes
            # ------------------------------------------------------------
            if mode == "mode4_start_goal_corr":
                s = np.array(bench.startList[0], dtype=float)
                g = np.array(bench.goalList[0], dtype=float)

                ax.plot(
                    [s[0], g[0]], [s[1], g[1]],
                    linestyle="--",
                    linewidth=2,
                    alpha=0.8,
                    zorder=18,
                    label="Start–Goal corridor axis"
                )

                sample_log = getattr(planner, "_sample_log", []) or []
                corr_nodes = []
                for r in sample_log:
                    if r.get("mode") != mode:
                        continue
                    if r.get("source") != "start_goal_corridor":
                        continue
                    nid = int(r["node_id"])
                    if nid in pos:
                        corr_nodes.append(nid)

                if corr_nodes:
                    xs = [pos[n][0] for n in corr_nodes]
                    ys = [pos[n][1] for n in corr_nodes]
                    ax.scatter(
                        xs, ys,
                        marker="^",
                        s=max(25, int(0.7 * nodeSize)),
                        zorder=19,
                        label="Corridor-sampled nodes"
                    )

            # Legend nur wenn extra Elemente vorhanden
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc="upper right")

            # Titel
            t = float(row["time_s"]) if pd.notna(row.get("time_s", np.nan)) else np.nan
            L = float(row["path_length"]) if pd.notna(row.get("path_length", np.nan)) else np.nan
            S = float(row["roadmap_size"]) if pd.notna(row.get("roadmap_size", np.nan)) else np.nan
            ax.set_title(
                f"{bench_name} – {mode}\n"
                f"{tag}: run={run_idx}, time={t:.3f}s, len={L:.2f}, size={S:.0f}, seed={seed}"
            )

            plt.tight_layout()
            plt.show()


# ============================================================
# RUN + PLOTS
# ============================================================
benchmarks = [bench_m1]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)
summary = summarize(df)

# Seeds aus denselben Evaluation-Ergebnissen ziehen
SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)

# 1) Summary-Plots (1×4)
for b in benchmarks:
     plot_benchmark_like_task1(summary, b.name, mode_order=MODE_ORDER)

# 2) Best-Run Visualisierung (Sampling-Struktur sichtbar)
BENCH_ORDER = [b.name for b in benchmarks]
visualize_best_per_mode_and_benchmark(
     df, benchmarks, configs,
     nodeSize=80,
     mode_order=MODE_ORDER,
     bench_order=BENCH_ORDER,
     figsize=(7, 7),
     prefer_seed_runs_for_mode12=True,
 )


### Diskussion der Ergebnisse

**Planungszeit**

Die absoluten Planungszeiten liegen bei allen Modi im sehr niedrigen Millisekundenbereich. Trotz sichtbarer Unterschiede zwischen den Balken handelt es sich insgesamt um kleine absolute Laufzeiten. Mode 3 (Max-Min) weist den höchsten Mittelwert und die größte Streuung auf, was auf den zusätzlichen Rechenaufwand durch die Kandidatenbewertung zurückzuführen ist. Die Baseline, Mode 1 und Mode 4 zeigen sehr geringe und vergleichbare Planungszeiten. Mode 2 liegt leicht darüber, was mit der stärkeren lokalen Verdichtung und der damit verbundenen höheren Knotenzahl erklärbar ist.

**Kollisionschecks**

Mode 2 und Mode 3 verursachen die meisten Kollisionsprüfungen. Bei Mode 3 entsteht dies durch die gezielte Exploration bislang schlecht abgedeckter Bereiche, die im WallTinyDoor häufig nahe an Hindernissen liegen. Mode 2 erzeugt durch das intensive lokale Sampling um Seedpoints eine hohe Anzahl an Kandidaten, insbesondere im Bereich der schmalen Türöffnung. Die Baseline, Mode 1 und Mode 4 liegen deutlich darunter. Mode 4 profitiert davon, dass der Start–Ziel-Korridor die Suche auf den relevanten Durchgang fokussiert und großflächige Exploration vermeidet.

**Pfadlänge**

Die Pfadlängen zeigen Unterschiede im Bereich mehrerer Längeneinheiten und sind damit aussagekräftig. Die Baseline und Mode 3 erzeugen die längsten Pfade. Bei Mode 3 deutet dies darauf hin, dass die sehr sparsame Roadmap die geometrische Feinauflösung der engen Passage nicht ausreichend unterstützt. Mode 1, Mode 2 und Mode 4 liefern kürzere Pfade. Mode 1 profitiert von der lokalen Nachbesserung kritischer Kanten im Bereich der Engstelle. Mode 4 erreicht ebenfalls kurze Pfade, da der Start–Ziel-Korridor eine direkte Führung durch die Türöffnung begünstigt.

**Roadmapgröße**

Die Roadmapgröße unterscheidet sich stark zwischen den Modi. Mode 2 erzeugt mit Abstand die größte Roadmap, was aus der intensiven lokalen Erweiterung um Seedpoints resultiert. Diese hohe Knotendichte verbessert die Konnektivität, ist für diesen Benchmark jedoch nur begrenzt erforderlich. Mode 3 erzeugt die kleinste Roadmap. In einem Szenario mit schmaler Passage ist diese geringe Auflösung nachteilig, da wichtige geometrische Details nicht ausreichend erfasst werden. Mode 4 erreicht eine moderate Roadmapgröße, da der Korridor gezielt entlang des relevanten Durchgangs erweitert wird. Die Baseline und Mode 1 liegen zwischen diesen Extremen.

**Gesamtbewertung und Eignung für den Benchmark**

Der WallTinyDoor-Benchmark ist durch eine enge, stark restriktive Passage charakterisiert. Entscheidend ist weniger die globale Abdeckung des Raums als vielmehr die gezielte Auflösung des kritischen Durchgangs. Mode 4 ist hierfür gut geeignet, da der Start–Ziel-Korridor die Exploration fokussiert, eine moderate Roadmapgröße erzeugt und kurze Pfade ermöglicht, ohne eine hohe Anzahl an Kollisionschecks zu verursachen. Mode 1 zeigt ebenfalls gute Ergebnisse durch lokale Nachbesserung entlang kritischer Kanten. Mode 2 liefert robuste Lösungen, ist jedoch ineffizient aufgrund der sehr großen Roadmap und der hohen Anzahl an Kollisionschecks. Mode 3 ist für diesen Benchmark weniger geeignet, da die minimalistische Roadmap die geometrischen Anforderungen der schmalen Passage nicht ausreichend erfüllt.

## Evaluation mit Benchmark 3: WallTinyWideDoor

Der Benchmark WallTinyWideDoor stellt im Vergleich zu WallTinyDoor eine entschärfte Engstellen-Situation dar. Zwar trennt weiterhin eine Wand den Start- und Zielbereich, die Passage ist jedoch deutlich breiter, sodass mehrere alternative Durchgänge möglich sind. Dadurch verschiebt sich der Schwerpunkt von einer rein lokalen Engstellenauflösung hin zu einer ausgewogenen Abdeckung des Konfigurationsraums und einer effizienten Verbindung der beiden Raumhälften. Alle fünf Sampling-Modi (Baseline sowie Mode 1–4) wurden auf diesem Benchmark jeweils mit 50 unabhängigen Planungsdurchläufen getestet und evaluiert. Die Balkendiagramme zeigen dabei Mittelwert und Varianz der betrachteten Metriken über alle erfolgreichen Läufe und erlauben eine quantitative Bewertung der Stabilität und Effizienz der einzelnen Modi. Die darunter dargestellten Roadmap- und Pfadvisualisierungen repräsentieren jeweils einen Best-Case pro Modus. Dieser Best-Case wird über eine gewichtete Mehrkriterienbewertung bestimmt, die Planungszeit, Pfadlänge und Roadmapgröße berücksichtigt, wobei extreme Ausreißer vorab über Quantilfilter ausgeschlossen werden. Die Visualisierungen dienen somit der qualitativen Einordnung typischer Lösungsstrukturen und sind nicht direkt mit den Mittelwerten der Balkendiagramme gleichzusetzen.

In [ ]:
BENCH_MODE_MAP = None

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


# ============================================================
# 1) Evaluation Helpers
# ============================================================
def path_length_from_graph(graph, path):
    if not path:
        return np.nan
    pts = [graph.nodes[p]["pos"] for p in path]
    return float(sum(
        np.linalg.norm(np.array(pts[i]) - np.array(pts[i + 1]))
        for i in range(len(pts) - 1)
    ))


# ------------------------------------------------------------
# Out-of-bounds => Collision hart machen (falls CC das nicht selbst tut)
# ------------------------------------------------------------
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        lims = _cc.getEnvironmentLimits()
        (xmin, xmax), (ymin, ymax) = lims
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# 2) Run helpers
# ============================================================
def run_one(planner_class, benchmark, config, seed=None):
    """
    Führt genau einen Run aus. Robust gegen Exceptions.
    Zusätzlich: zählt Seeds (Mode 1/2) und "Nodes aus Seeds" (Mode 1/2), damit wir
    später einen Plot-Run wählen können, in dem Seeds wirklich sichtbar sind.
    """
    if seed is not None:
        seed = int(seed)
        random.seed(seed)
        np.random.seed(seed)

    _ensure_oob_is_collision(benchmark.collisionChecker)

    planner = planner_class(benchmark.collisionChecker)

    # optionaler Counter reset
    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    t0 = time.time()
    err = None
    path = []
    try:
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
    except Exception as e:
        err = str(e)
        path = []
    t1 = time.time()

    collisions = getattr(benchmark.collisionChecker, "counter", np.nan)

    # Graph-Metriken
    try:
        size = len(planner.graph.nodes())
    except Exception:
        size = np.nan

    try:
        length = path_length_from_graph(planner.graph, path)
    except Exception:
        length = np.nan

    success = (path is not None) and (path != [])

    # --- NEW: Seeds und Seed-Nodes zählen ---
    seed_used_count = len(getattr(planner, "_seed_points_used", []) or [])
    sample_log = getattr(planner, "_sample_log", []) or []
    seed_sampled_nodes_count = sum(
        1 for r in sample_log
        if (r.get("seed_key") is not None) and (r.get("mode") in ("mode1_seed_gauss", "mode2_seed_dist"))
    )

    return {
        "time_s": (t1 - t0),
        "collision_checks": collisions,
        "path_length": length,
        "roadmap_size": size,
        "success": success,
        "error": err,
        # New fields (für Plot-Auswahl; in Summary ignorieren)
        "seed_used_count": int(seed_used_count),
        "seed_sampled_nodes_count": int(seed_sampled_nodes_count),
    }


def _modes_for_benchmark(bench_name, configs, bench_mode_map=None):
    if bench_mode_map is None:
        return list(configs.keys())
    return bench_mode_map.get(bench_name, list(configs.keys()))


def run_suite(benchmarks, configs, runs=30, base_seed=1234, progress_every=10, bench_mode_map=None):
    rows = []

    total = 0
    for b in benchmarks:
        total += len(_modes_for_benchmark(b.name, configs, bench_mode_map)) * runs

    done = 0
    t0 = time.time()

    for b in benchmarks:
        mode_list = _modes_for_benchmark(b.name, configs, bench_mode_map)

        for mode_name in mode_list:
            cfg = configs[mode_name]

            for r in range(runs):
                seed = base_seed + 1000 * r
                metrics = run_one(EnhancedLazyPRM, b, cfg, seed=seed)

                rows.append({
                    "benchmark": b.name,
                    "mode": mode_name,
                    "run": r,
                    "seed": seed,
                    **metrics
                })

                done += 1
                if progress_every and (done % progress_every == 0):
                    elapsed = time.time() - t0
                    print(f"{done}/{total} done, elapsed {elapsed:.1f}s")

    return pd.DataFrame(rows)


# ============================================================
# 3) Aggregation (Summary)
# ============================================================
def summarize(df):
    df_metrics = df.copy()

    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df_metrics.loc[df_metrics["success"] == False, c] = np.nan

    summary = (df_metrics
        .groupby(["benchmark", "mode"])
        .agg(
            runs=("run", "count"),
            success_rate=("success", "mean"),

            time_mean=("time_s", "mean"),
            time_std=("time_s", "std"),

            coll_mean=("collision_checks", "mean"),
            coll_std=("collision_checks", "std"),

            len_mean=("path_length", "mean"),
            len_std=("path_length", "std"),

            size_mean=("roadmap_size", "mean"),
            size_std=("roadmap_size", "std"),

            n_errors=("error", lambda s: int(s.notna().sum()))
        )
        .reset_index()
    )
    return summary


# ============================================================
# 4) Plot Layout (1×4)
# ============================================================
METRIC_SPECS = [
    ("time_mean", "time_std", "Planungszeit (s)", "gold", "Planungszeit (s)"),
    ("coll_mean", "coll_std", "Kollisionschecks", "tomato", "Kollisionschecks"),
    ("len_mean",  "len_std",  "Pfadlänge",        "cornflowerblue", "Pfadlänge"),
    ("size_mean", "size_std", "Roadmap Größe",    "mediumpurple", "Roadmap Größe (#Nodes)"),
]

def compute_global_ylims(summary_df, metric_specs, pad=0.10):
    ylims = {}
    for mean_col, std_col, *_ in metric_specs:
        mean_vals = summary_df[mean_col].to_numpy(dtype=float)
        std_vals  = summary_df[std_col].to_numpy(dtype=float)
        comb = mean_vals + np.nan_to_num(std_vals, nan=0.0)
        y_max = np.nanmax(comb) if np.any(np.isfinite(comb)) else 1.0
        if not np.isfinite(y_max) or y_max <= 0:
            y_max = 1.0
        ylims[mean_col] = (0.0, y_max * (1.0 + pad))
    return ylims

def plot_benchmark_like_task1(summary_df, benchmark_name, mode_order):
    sub = summary_df[summary_df["benchmark"] == benchmark_name].copy()

    present = [m for m in mode_order if m in set(sub["mode"].values)]
    if present:
        sub["mode"] = pd.Categorical(sub["mode"], categories=present, ordered=True)
        sub = sub.sort_values("mode")
    else:
        sub = sub.sort_values("mode")

    labels = sub["mode"].astype(str).tolist()
    x = np.arange(len(labels), dtype=int)

    fig, axs = plt.subplots(1, 4, figsize=(18, 4))
    fig.suptitle(f"Aufgabe 2: Vergleich Sampling-Modi – {benchmark_name}", fontsize=14)

    GLOBAL_YLIMS = compute_global_ylims(summary_df, METRIC_SPECS, pad=0.10)

    for ax, (mean_col, std_col, title, color, ylabel) in zip(axs, METRIC_SPECS):
        y = sub[mean_col].to_numpy(dtype=float)
        yerr = sub[std_col].to_numpy(dtype=float)
        yerr_plot = np.nan_to_num(yerr, nan=0.0)

        ax.bar(x, y, yerr=yerr_plot, capsize=4, color=color, alpha=0.85)
        ax.set_title(title)
        ax.set_xlabel("Sampling-Modus")
        ax.set_ylabel(ylabel)
        ax.set_ylim(*GLOBAL_YLIMS[mean_col])
        ax.grid(axis="y", alpha=0.3)
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=25, ha="right")

    plt.tight_layout()
    plt.show()


# ============================================================
# 5) Best-Plot pro Mode & Benchmark (mit Sampling-Struktur)
# ============================================================
def _robust_minmax_norm(series, q_low=0.05, q_high=0.95):
    x = series.to_numpy(dtype=float)
    if np.all(~np.isfinite(x)):
        return pd.Series(np.zeros(len(series)), index=series.index)

    lo = np.nanquantile(x, q_low)
    hi = np.nanquantile(x, q_high)
    if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
        return pd.Series(np.zeros(len(series)), index=series.index)

    xn = (series - lo) / (hi - lo)
    xn = xn.clip(0.0, 1.0).fillna(1.0)
    return xn


def _pick_best_run_multicriteria(df_succ, w_time=0.6, w_len=0.25, w_size=0.15,
                                 q_low=0.05, q_high=0.95):
    d = df_succ.copy()

    for col in ["time_s", "path_length", "roadmap_size"]:
        if col not in d.columns:
            d[col] = np.nan

    d["time_n"] = _robust_minmax_norm(d["time_s"], q_low=q_low, q_high=q_high)
    d["len_n"]  = _robust_minmax_norm(d["path_length"], q_low=q_low, q_high=q_high)
    d["size_n"] = _robust_minmax_norm(d["roadmap_size"], q_low=q_low, q_high=q_high)
    d["score"] = w_time * d["time_n"] + w_len * d["len_n"] + w_size * d["size_n"]

    d = d.sort_values(["score", "time_s"], ascending=[True, True])
    return d.iloc[0]


def extract_best_seed_map(
    df,
    prefer_seed_runs_for_mode12=True,
    w_time=0.6, w_len=0.25, w_size=0.15
):
    """
    Gibt dict zurück: seed_map[(benchmark_name, mode_name)] = seed
    verwendet die exakt gleiche Auswahlregel wie visualize_best_per_mode_and_benchmark().
    """
    seed_map = {}

    for (bench_name, mode), df_bm in df.groupby(["benchmark", "mode"]):
        df_bm = df_bm.copy()
        df_succ = df_bm[df_bm["success"] == True].copy()
        if df_succ.empty:
            continue

        df_pick = df_succ
        if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
            df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
            if not df_seed.empty:
                df_pick = df_seed

        row = _pick_best_run_multicriteria(
            df_pick, w_time=w_time, w_len=w_len, w_size=w_size
        )
        seed_map[(bench_name, mode)] = int(row["seed"])

    return seed_map

def visualize_best_per_mode_and_benchmark(
    df, benchmarks, configs, nodeSize=80,
    mode_order=None, bench_order=None, figsize=(7, 7),
    w_time=0.6, w_len=0.25, w_size=0.15,
    prefer_seed_runs_for_mode12=True,
):
    """
    Plot-Regel:
    - Mode 1/2: zeige Seed (X) + Nodes-aus-Seed + Seed->Node-Linien,
      aber NUR wenn Seeds tatsächlich benutzt wurden.
      Falls prefer_seed_runs_for_mode12=True, wird ein erfolgreicher Run bevorzugt,
      der seed_used_count > 0 hat (sonst sieht man sonst oft "keine Seeds", weil
      der Planner zufällig direkt eine kollisionsfreie Lösung findet).
    - Mode 3: zeige Kandidatenwolke + selected (Stern) aus dem Planner-Log.
    - Mode 4: zeige Start–Goal-Achse + Korridor-Nodes.
    - Baseline: keine Seeds, keine Hotspots.
    """
    bench_by_name = {b.name: b for b in benchmarks}

    if bench_order is None:
        bench_order = [b.name for b in benchmarks]
    if mode_order is None:
        mode_order = list(configs.keys())

    for bench_name in bench_order:
        bench = bench_by_name[bench_name]

        modes_present = [m for m in mode_order if ((df["benchmark"] == bench_name) & (df["mode"] == m)).any()]
        for mode in modes_present:
            df_bm = df[(df["benchmark"] == bench_name) & (df["mode"] == mode)].copy()
            if df_bm.empty:
                continue

            df_succ = df_bm[df_bm["success"] == True].copy()

            # --- Auswahlregel: Mode1/2 bevorzugt Run mit Seeds ---
            df_pick = df_succ
            if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
                if not df_seed.empty:
                    df_pick = df_seed  # nur wenn es wirklich solche Runs gibt

            if not df_pick.empty:
                row = _pick_best_run_multicriteria(df_pick, w_time=w_time, w_len=w_len, w_size=w_size)
                tag = f"BEST (score={row['score']:.3f}; w_t={w_time}, w_l={w_len}, w_s={w_size})"
            else:
                row = df_bm.sort_values("run").iloc[-1]
                tag = "FALLBACK (no success)"

            run_idx = int(row["run"])
            seed = int(row["seed"])

            random.seed(seed)
            np.random.seed(seed)
            _ensure_oob_is_collision(bench.collisionChecker)

            planner = EnhancedLazyPRM(bench.collisionChecker)
            try:
                sol = planner.planPath(bench.startList, bench.goalList, configs[mode])
            except Exception as e:
                sol = []
                tag = f"{tag} – ERROR: {e}"

            fig, ax = plt.subplots(figsize=figsize)
            lazyPRMVisualize(planner, sol, ax=ax, nodeSize=nodeSize)

            # Achsen fixieren
            (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect("equal", adjustable="box")

            pos = nx.get_node_attributes(planner.graph, "pos")

            # ------------------------------------------------------------
            # MODE 1/2: Seed + Nodes-aus-Seed + Seed->Node Links
            # ------------------------------------------------------------
            if mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                seeds = getattr(planner, "_seed_points_used", []) or []
                sample_log = getattr(planner, "_sample_log", []) or []

                # Seeds plotten (nur wenn wirklich genutzt)
                if seeds:
                    sx = [p[0] for p in seeds]
                    sy = [p[1] for p in seeds]
                    ax.scatter(
                        sx, sy,
                        marker="x",
                        s=max(70, int(1.3 * nodeSize)),
                        linewidths=2.5,
                        zorder=30,
                        label="Seed used (Mode 1/2)"
                    )

                    # Nodes, die aus Seeds erzeugt wurden (seed_key != None)
                    seed_nodes = []
                    seed_links = []  # (seed_point, node_point)

                    # Mapping seed_key -> seed_point
                    seed_map = {}
                    for sp in seeds:
                        k = (round(float(sp[0]), 3), round(float(sp[1]), 3))
                        seed_map[k] = [float(sp[0]), float(sp[1])]

                    for r in sample_log:
                        if r.get("mode") != mode:
                            continue
                        k = r.get("seed_key", None)
                        if k is None:
                            continue
                        nid = int(r["node_id"])
                        if nid not in pos:
                            continue
                        seed_nodes.append(nid)
                        sp = seed_map.get(k, r.get("seed", None))
                        if sp is not None:
                            seed_links.append((sp, pos[nid]))

                    if seed_nodes:
                        nxs = [pos[n][0] for n in seed_nodes]
                        nys = [pos[n][1] for n in seed_nodes]
                        ax.scatter(
                            nxs, nys,
                            marker="o",
                            s=max(35, int(0.8 * nodeSize)),
                            edgecolors="black",
                            linewidths=1.2,
                            zorder=29,
                            label="Nodes sampled from seed"
                        )
                        # Linien Seed->Node
                        for sp, npnt in seed_links:
                            ax.plot(
                                [sp[0], npnt[0]],
                                [sp[1], npnt[1]],
                                linewidth=1.0,
                                alpha=0.6,
                                zorder=28
                            )

            # ------------------------------------------------------------
            # MODE 3: Max–Min – Kandidatenwolke + selected (letzter Schritt)
            # ------------------------------------------------------------
            if mode == "mode3_max_min":
                cand_log = getattr(planner, "_mode3_candidates_log", []) or []
                if cand_log:
                    last = cand_log[-1]
                    cands = last.get("candidates", []) or []
                    sel = last.get("selected", None)

                    if cands:
                        cx = [c[0] for c in cands]
                        cy = [c[1] for c in cands]
                        ax.scatter(
                            cx, cy,
                            marker="o",
                            s=18,
                            facecolors="none",
                            linewidths=1.0,
                            zorder=20,
                            label="Max–Min candidates"
                        )

                    if sel is not None:
                        ax.scatter(
                            [sel[0]], [sel[1]],
                            marker="*",
                            s=200,
                            zorder=21,
                            label="Max–Min selected"
                        )

            # ------------------------------------------------------------
            # MODE 4: Start–Goal Corridor – Achse + Korridor-Nodes
            # ------------------------------------------------------------
            if mode == "mode4_start_goal_corr":
                s = np.array(bench.startList[0], dtype=float)
                g = np.array(bench.goalList[0], dtype=float)

                ax.plot(
                    [s[0], g[0]], [s[1], g[1]],
                    linestyle="--",
                    linewidth=2,
                    alpha=0.8,
                    zorder=18,
                    label="Start–Goal corridor axis"
                )

                sample_log = getattr(planner, "_sample_log", []) or []
                corr_nodes = []
                for r in sample_log:
                    if r.get("mode") != mode:
                        continue
                    if r.get("source") != "start_goal_corridor":
                        continue
                    nid = int(r["node_id"])
                    if nid in pos:
                        corr_nodes.append(nid)

                if corr_nodes:
                    xs = [pos[n][0] for n in corr_nodes]
                    ys = [pos[n][1] for n in corr_nodes]
                    ax.scatter(
                        xs, ys,
                        marker="^",
                        s=max(25, int(0.7 * nodeSize)),
                        zorder=19,
                        label="Corridor-sampled nodes"
                    )

            # Legend nur wenn extra Elemente vorhanden
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc="upper right")

            # Titel
            t = float(row["time_s"]) if pd.notna(row.get("time_s", np.nan)) else np.nan
            L = float(row["path_length"]) if pd.notna(row.get("path_length", np.nan)) else np.nan
            S = float(row["roadmap_size"]) if pd.notna(row.get("roadmap_size", np.nan)) else np.nan
            ax.set_title(
                f"{bench_name} – {mode}\n"
                f"{tag}: run={run_idx}, time={t:.3f}s, len={L:.2f}, size={S:.0f}, seed={seed}"
            )

            plt.tight_layout()
            plt.show()


# ============================================================
# RUN + PLOTS
# ============================================================
benchmarks = [bench_m2]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)
summary = summarize(df)

# Seeds aus denselben Evaluation-Ergebnissen ziehen
SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)

# 1) Summary-Plots (1×4)
for b in benchmarks:
     plot_benchmark_like_task1(summary, b.name, mode_order=MODE_ORDER)

# 2) Best-Run Visualisierung (Sampling-Struktur sichtbar)
BENCH_ORDER = [b.name for b in benchmarks]
visualize_best_per_mode_and_benchmark(
     df, benchmarks, configs,
     nodeSize=80,
     mode_order=MODE_ORDER,
     bench_order=BENCH_ORDER,
     figsize=(7, 7),
     prefer_seed_runs_for_mode12=True,
 )


### Diskussion der Ergebnisse

**Planungszeit**

Die Planungszeiten unterscheiden sich deutlich zwischen den Modi, liegen jedoch insgesamt im niedrigen zweistelligen Millisekundenbereich. Die Baseline weist die geringsten mittleren Planungszeiten auf. Mode 1, Mode 2 und Mode 4 liegen darüber, während Mode 3 (Max-Min) die höchsten Planungszeiten und die größte Varianz zeigt. Der erhöhte Rechenaufwand von Mode 3 resultiert aus der Bewertung mehrerer Kandidaten pro Iteration. In diesem Benchmark ist der Engpass jedoch nicht punktuell, sondern über eine längere Strecke ausgedehnt. Die Max-Min-Strategie kann diese Struktur nicht gezielt ausnutzen, da sie Knoten nach globalem Abstand platziert und keine kontinuierliche Abdeckung entlang der Passage erzwingt. Der zusätzliche Rechenaufwand führt daher nicht zu einem entsprechenden planerischen Vorteil.

**Kollisionschecks**

Bei den Kollisionschecks zeigen sich deutliche Unterschiede in absoluten Werten. Die Baseline verursacht die geringste Anzahl an Kollisionsprüfungen. Mode 1 und Mode 2 liegen deutlich darüber, während Mode 3 die höchste Anzahl und zugleich eine große Streuung aufweist. Der langgezogene Engpass führt dazu, dass viele Kanten entlang der Passage geprüft werden müssen. Strategien mit starker lokaler Verdichtung (Mode 1 und Mode 2) erzeugen hierbei zahlreiche zusätzliche Kandidaten. Mode 3 prüft viele global verteilte Kandidaten, ohne die Exploration gezielt entlang der Engpassrichtung zu fokussieren. Mode 4 liegt unterhalb von Mode 1 und Mode 2, da der Start–Ziel-Korridor die Exploration räumlich auf den relevanten Bereich beschränkt..

**Pfadlänge**

Die Pfadlängen unterscheiden sich zwischen den Modi nur moderat. Baseline, Mode 1, Mode 2 und Mode 4 erreichen sehr ähnliche mittlere Pfadlängen. Mode 3 erzeugt im Mittel längere Pfade und weist eine erhöhte Varianz auf. Die Best-Case-Runs zeigen, dass in einem langgezogenen Engpass eine durchgehende Konnektivität entscheidend ist, nicht jedoch eine hochpräzise Platzierung einzelner Knoten. Zusätzliche globale Abdeckung oder starke lokale Verdichtung führen daher nicht zu signifikant kürzeren Pfaden.

**Roadmapgröße**

Die Roadmapgröße unterscheidet sich deutlich zwischen den Modi. Die Baseline erzeugt eine vergleichsweise kleine Roadmap, da ausschließlich uniform gesampelt wird und keine gezielte Nachverdichtung erfolgt. Mode 4 erzeugt hingegen eine deutlich größere Roadmap als die Baseline. Dies ist auf das Sampling entlang des Start–Ziel-Korridors zurückzuführen, das über die gesamte Länge des Engpasses hinweg zusätzliche Knoten platziert. Diese erhöhte Knotenzahl ist funktional begründet, da der langgezogene Engpass eine durchgehende Konnektivität entlang der Passage erfordert.
Mode 2 erzeugt die größte Roadmap, gefolgt von Mode 1, was auf umfangreiche lokale Erweiterungen um Seedpoints zurückzuführen ist. Mode 3 bleibt sehr kompakt, verfehlt jedoch eine gleichmäßige Abdeckung entlang der gesamten Engpasslänge.

**Gesamtbewertung und Eignung für den Benchmark**

Der WallTinyWideDoor-Benchmark ist durch einen langgezogenen Engpass geprägt, der eine kontinuierliche Konnektivität entlang der Passage erfordert, jedoch keine punktuell hochpräzise Platzierung einzelner Knoten. Die Baseline arbeitet sehr effizient und erreicht mit minimaler Roadmapgröße valide und ausreichend kurze Pfade, zeigt jedoch keine gezielte Strukturierung entlang des Engpasses. Mode 4 ist für diesen Benchmark besonders geeignet, da der Start–Ziel-Korridor eine gleichmäßige Abdeckung entlang der Engpassrichtung ermöglicht. Die dafür notwendige größere Roadmap führt zwar zu höheren Kosten als bei der Baseline, verbessert jedoch die strukturelle Robustheit der Lösung. Mode 1 und Mode 2 liefern ebenfalls gültige Lösungen, erzeugen jedoch deutlich größere Roadmaps und höhere Kosten, ohne einen entsprechenden Gewinn bei der Pfadqualität zu erzielen. Mode 3 ist am wenigsten geeignet, da seine abdeckungsorientierte Strategie den langgezogenen Engpass nicht gezielt unterstützt und der zusätzliche Rechenaufwand nicht zu besseren Lösungen führt.

## Evaluation mit Benchmark 4: U-Shape

Der U-Shape-Benchmark stellt eine Umgebung mit einer ausgeprägten Sackgassen- bzw. Umgehungsstruktur dar. Start und Ziel sind räumlich relativ nahe beieinander, jedoch durch eine U-förmige Hindernisstruktur getrennt, sodass ein direkter Weg blockiert ist und eine gezielte Umfahrung erforderlich wird. Der Benchmark erfordert damit weniger globale Abdeckung, sondern vor allem die Fähigkeit, strukturbedingt irreführende direkte Verbindungen zu vermeiden und alternative, nicht-triviale Pfade zu finden. Alle fünf Sampling-Modi (Baseline sowie Mode 1–4) wurden auf diesem Benchmark jeweils mit 50 unabhängigen Planungsdurchläufen getestet und evaluiert. Die Balkendiagramme zeigen Mittelwert und Varianz der betrachteten Metriken über alle erfolgreichen Läufe. Die darunter dargestellten Roadmap- und Pfadvisualisierungen repräsentieren jeweils einen Best-Case, der über eine gewichtete Mehrkriterienbewertung aus Planungszeit, Pfadlänge und Roadmapgröße bestimmt wird, wobei extreme Ausreißer vorab über Quantilfilter ausgeschlossen werden. Die Visualisierungen dienen somit der qualitativen Einordnung typischer Lösungsstrukturen und sind nicht direkt mit den Mittelwerten der Balkendiagramme gleichzusetzen.

In [ ]:
BENCH_MODE_MAP = None

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


# ============================================================
# 1) Evaluation Helpers
# ============================================================
def path_length_from_graph(graph, path):
    if not path:
        return np.nan
    pts = [graph.nodes[p]["pos"] for p in path]
    return float(sum(
        np.linalg.norm(np.array(pts[i]) - np.array(pts[i + 1]))
        for i in range(len(pts) - 1)
    ))


# ------------------------------------------------------------
# Out-of-bounds => Collision hart machen (falls CC das nicht selbst tut)
# ------------------------------------------------------------
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        lims = _cc.getEnvironmentLimits()
        (xmin, xmax), (ymin, ymax) = lims
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# 2) Run helpers
# ============================================================
def run_one(planner_class, benchmark, config, seed=None):
    """
    Führt genau einen Run aus. Robust gegen Exceptions.
    Zusätzlich: zählt Seeds (Mode 1/2) und "Nodes aus Seeds" (Mode 1/2), damit wir
    später einen Plot-Run wählen können, in dem Seeds wirklich sichtbar sind.
    """
    if seed is not None:
        seed = int(seed)
        random.seed(seed)
        np.random.seed(seed)

    _ensure_oob_is_collision(benchmark.collisionChecker)

    planner = planner_class(benchmark.collisionChecker)

    # optionaler Counter reset
    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    t0 = time.time()
    err = None
    path = []
    try:
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
    except Exception as e:
        err = str(e)
        path = []
    t1 = time.time()

    collisions = getattr(benchmark.collisionChecker, "counter", np.nan)

    # Graph-Metriken
    try:
        size = len(planner.graph.nodes())
    except Exception:
        size = np.nan

    try:
        length = path_length_from_graph(planner.graph, path)
    except Exception:
        length = np.nan

    success = (path is not None) and (path != [])

    # --- NEW: Seeds und Seed-Nodes zählen ---
    seed_used_count = len(getattr(planner, "_seed_points_used", []) or [])
    sample_log = getattr(planner, "_sample_log", []) or []
    seed_sampled_nodes_count = sum(
        1 for r in sample_log
        if (r.get("seed_key") is not None) and (r.get("mode") in ("mode1_seed_gauss", "mode2_seed_dist"))
    )

    return {
        "time_s": (t1 - t0),
        "collision_checks": collisions,
        "path_length": length,
        "roadmap_size": size,
        "success": success,
        "error": err,
        # New fields (für Plot-Auswahl; in Summary ignorieren)
        "seed_used_count": int(seed_used_count),
        "seed_sampled_nodes_count": int(seed_sampled_nodes_count),
    }


def _modes_for_benchmark(bench_name, configs, bench_mode_map=None):
    if bench_mode_map is None:
        return list(configs.keys())
    return bench_mode_map.get(bench_name, list(configs.keys()))


def run_suite(benchmarks, configs, runs=30, base_seed=1234, progress_every=10, bench_mode_map=None):
    rows = []

    total = 0
    for b in benchmarks:
        total += len(_modes_for_benchmark(b.name, configs, bench_mode_map)) * runs

    done = 0
    t0 = time.time()

    for b in benchmarks:
        mode_list = _modes_for_benchmark(b.name, configs, bench_mode_map)

        for mode_name in mode_list:
            cfg = configs[mode_name]

            for r in range(runs):
                seed = base_seed + 1000 * r
                metrics = run_one(EnhancedLazyPRM, b, cfg, seed=seed)

                rows.append({
                    "benchmark": b.name,
                    "mode": mode_name,
                    "run": r,
                    "seed": seed,
                    **metrics
                })

                done += 1
                if progress_every and (done % progress_every == 0):
                    elapsed = time.time() - t0
                    print(f"{done}/{total} done, elapsed {elapsed:.1f}s")

    return pd.DataFrame(rows)


# ============================================================
# 3) Aggregation (Summary)
# ============================================================
def summarize(df):
    df_metrics = df.copy()

    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df_metrics.loc[df_metrics["success"] == False, c] = np.nan

    summary = (df_metrics
        .groupby(["benchmark", "mode"])
        .agg(
            runs=("run", "count"),
            success_rate=("success", "mean"),

            time_mean=("time_s", "mean"),
            time_std=("time_s", "std"),

            coll_mean=("collision_checks", "mean"),
            coll_std=("collision_checks", "std"),

            len_mean=("path_length", "mean"),
            len_std=("path_length", "std"),

            size_mean=("roadmap_size", "mean"),
            size_std=("roadmap_size", "std"),

            n_errors=("error", lambda s: int(s.notna().sum()))
        )
        .reset_index()
    )
    return summary


# ============================================================
# 4) Plot Layout (1×4)
# ============================================================
METRIC_SPECS = [
    ("time_mean", "time_std", "Planungszeit (s)", "gold", "Planungszeit (s)"),
    ("coll_mean", "coll_std", "Kollisionschecks", "tomato", "Kollisionschecks"),
    ("len_mean",  "len_std",  "Pfadlänge",        "cornflowerblue", "Pfadlänge"),
    ("size_mean", "size_std", "Roadmap Größe",    "mediumpurple", "Roadmap Größe (#Nodes)"),
]

def compute_global_ylims(summary_df, metric_specs, pad=0.10):
    ylims = {}
    for mean_col, std_col, *_ in metric_specs:
        mean_vals = summary_df[mean_col].to_numpy(dtype=float)
        std_vals  = summary_df[std_col].to_numpy(dtype=float)
        comb = mean_vals + np.nan_to_num(std_vals, nan=0.0)
        y_max = np.nanmax(comb) if np.any(np.isfinite(comb)) else 1.0
        if not np.isfinite(y_max) or y_max <= 0:
            y_max = 1.0
        ylims[mean_col] = (0.0, y_max * (1.0 + pad))
    return ylims

def plot_benchmark_like_task1(summary_df, benchmark_name, mode_order):
    sub = summary_df[summary_df["benchmark"] == benchmark_name].copy()

    present = [m for m in mode_order if m in set(sub["mode"].values)]
    if present:
        sub["mode"] = pd.Categorical(sub["mode"], categories=present, ordered=True)
        sub = sub.sort_values("mode")
    else:
        sub = sub.sort_values("mode")

    labels = sub["mode"].astype(str).tolist()
    x = np.arange(len(labels), dtype=int)

    fig, axs = plt.subplots(1, 4, figsize=(18, 4))
    fig.suptitle(f"Aufgabe 2: Vergleich Sampling-Modi – {benchmark_name}", fontsize=14)

    GLOBAL_YLIMS = compute_global_ylims(summary_df, METRIC_SPECS, pad=0.10)

    for ax, (mean_col, std_col, title, color, ylabel) in zip(axs, METRIC_SPECS):
        y = sub[mean_col].to_numpy(dtype=float)
        yerr = sub[std_col].to_numpy(dtype=float)
        yerr_plot = np.nan_to_num(yerr, nan=0.0)

        ax.bar(x, y, yerr=yerr_plot, capsize=4, color=color, alpha=0.85)
        ax.set_title(title)
        ax.set_xlabel("Sampling-Modus")
        ax.set_ylabel(ylabel)
        ax.set_ylim(*GLOBAL_YLIMS[mean_col])
        ax.grid(axis="y", alpha=0.3)
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=25, ha="right")

    plt.tight_layout()
    plt.show()


# ============================================================
# 5) Best-Plot pro Mode & Benchmark (mit Sampling-Struktur)
# ============================================================
def _robust_minmax_norm(series, q_low=0.05, q_high=0.95):
    x = series.to_numpy(dtype=float)
    if np.all(~np.isfinite(x)):
        return pd.Series(np.zeros(len(series)), index=series.index)

    lo = np.nanquantile(x, q_low)
    hi = np.nanquantile(x, q_high)
    if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
        return pd.Series(np.zeros(len(series)), index=series.index)

    xn = (series - lo) / (hi - lo)
    xn = xn.clip(0.0, 1.0).fillna(1.0)
    return xn


def _pick_best_run_multicriteria(df_succ, w_time=0.6, w_len=0.25, w_size=0.15,
                                 q_low=0.05, q_high=0.95):
    d = df_succ.copy()

    for col in ["time_s", "path_length", "roadmap_size"]:
        if col not in d.columns:
            d[col] = np.nan

    d["time_n"] = _robust_minmax_norm(d["time_s"], q_low=q_low, q_high=q_high)
    d["len_n"]  = _robust_minmax_norm(d["path_length"], q_low=q_low, q_high=q_high)
    d["size_n"] = _robust_minmax_norm(d["roadmap_size"], q_low=q_low, q_high=q_high)
    d["score"] = w_time * d["time_n"] + w_len * d["len_n"] + w_size * d["size_n"]

    d = d.sort_values(["score", "time_s"], ascending=[True, True])
    return d.iloc[0]

def extract_best_seed_map(
    df,
    prefer_seed_runs_for_mode12=True,
    w_time=0.6, w_len=0.25, w_size=0.15
):
    """
    Gibt dict zurück: seed_map[(benchmark_name, mode_name)] = seed
    verwendet die exakt gleiche Auswahlregel wie visualize_best_per_mode_and_benchmark().
    """
    seed_map = {}

    for (bench_name, mode), df_bm in df.groupby(["benchmark", "mode"]):
        df_bm = df_bm.copy()
        df_succ = df_bm[df_bm["success"] == True].copy()
        if df_succ.empty:
            continue

        df_pick = df_succ
        if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
            df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
            if not df_seed.empty:
                df_pick = df_seed

        row = _pick_best_run_multicriteria(
            df_pick, w_time=w_time, w_len=w_len, w_size=w_size
        )
        seed_map[(bench_name, mode)] = int(row["seed"])

    return seed_map

def visualize_best_per_mode_and_benchmark(
    df, benchmarks, configs, nodeSize=80,
    mode_order=None, bench_order=None, figsize=(7, 7),
    w_time=0.6, w_len=0.25, w_size=0.15,
    prefer_seed_runs_for_mode12=True,
):
    """
    Plot-Regel:
    - Mode 1/2: zeige Seed (X) + Nodes-aus-Seed + Seed->Node-Linien,
      aber NUR wenn Seeds tatsächlich benutzt wurden.
      Falls prefer_seed_runs_for_mode12=True, wird ein erfolgreicher Run bevorzugt,
      der seed_used_count > 0 hat (sonst sieht man sonst oft "keine Seeds", weil
      der Planner zufällig direkt eine kollisionsfreie Lösung findet).
    - Mode 3: zeige Kandidatenwolke + selected (Stern) aus dem Planner-Log.
    - Mode 4: zeige Start–Goal-Achse + Korridor-Nodes.
    - Baseline: keine Seeds, keine Hotspots.
    """
    bench_by_name = {b.name: b for b in benchmarks}

    if bench_order is None:
        bench_order = [b.name for b in benchmarks]
    if mode_order is None:
        mode_order = list(configs.keys())

    for bench_name in bench_order:
        bench = bench_by_name[bench_name]

        modes_present = [m for m in mode_order if ((df["benchmark"] == bench_name) & (df["mode"] == m)).any()]
        for mode in modes_present:
            df_bm = df[(df["benchmark"] == bench_name) & (df["mode"] == mode)].copy()
            if df_bm.empty:
                continue

            df_succ = df_bm[df_bm["success"] == True].copy()

            # --- Auswahlregel: Mode1/2 bevorzugt Run mit Seeds ---
            df_pick = df_succ
            if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
                if not df_seed.empty:
                    df_pick = df_seed  # nur wenn es wirklich solche Runs gibt

            if not df_pick.empty:
                row = _pick_best_run_multicriteria(df_pick, w_time=w_time, w_len=w_len, w_size=w_size)
                tag = f"BEST (score={row['score']:.3f}; w_t={w_time}, w_l={w_len}, w_s={w_size})"
            else:
                row = df_bm.sort_values("run").iloc[-1]
                tag = "FALLBACK (no success)"

            run_idx = int(row["run"])
            seed = int(row["seed"])

            random.seed(seed)
            np.random.seed(seed)
            _ensure_oob_is_collision(bench.collisionChecker)

            planner = EnhancedLazyPRM(bench.collisionChecker)
            try:
                sol = planner.planPath(bench.startList, bench.goalList, configs[mode])
            except Exception as e:
                sol = []
                tag = f"{tag} – ERROR: {e}"

            fig, ax = plt.subplots(figsize=figsize)
            lazyPRMVisualize(planner, sol, ax=ax, nodeSize=nodeSize)

            # Achsen fixieren
            (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect("equal", adjustable="box")

            pos = nx.get_node_attributes(planner.graph, "pos")

            # ------------------------------------------------------------
            # MODE 1/2: Seed + Nodes-aus-Seed + Seed->Node Links
            # ------------------------------------------------------------
            if mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                seeds = getattr(planner, "_seed_points_used", []) or []
                sample_log = getattr(planner, "_sample_log", []) or []

                # Seeds plotten (nur wenn wirklich genutzt)
                if seeds:
                    sx = [p[0] for p in seeds]
                    sy = [p[1] for p in seeds]
                    ax.scatter(
                        sx, sy,
                        marker="x",
                        s=max(70, int(1.3 * nodeSize)),
                        linewidths=2.5,
                        zorder=30,
                        label="Seed used (Mode 1/2)"
                    )

                    # Nodes, die aus Seeds erzeugt wurden (seed_key != None)
                    seed_nodes = []
                    seed_links = []  # (seed_point, node_point)

                    # Mapping seed_key -> seed_point
                    seed_map = {}
                    for sp in seeds:
                        k = (round(float(sp[0]), 3), round(float(sp[1]), 3))
                        seed_map[k] = [float(sp[0]), float(sp[1])]

                    for r in sample_log:
                        if r.get("mode") != mode:
                            continue
                        k = r.get("seed_key", None)
                        if k is None:
                            continue
                        nid = int(r["node_id"])
                        if nid not in pos:
                            continue
                        seed_nodes.append(nid)
                        sp = seed_map.get(k, r.get("seed", None))
                        if sp is not None:
                            seed_links.append((sp, pos[nid]))

                    if seed_nodes:
                        nxs = [pos[n][0] for n in seed_nodes]
                        nys = [pos[n][1] for n in seed_nodes]
                        ax.scatter(
                            nxs, nys,
                            marker="o",
                            s=max(35, int(0.8 * nodeSize)),
                            edgecolors="black",
                            linewidths=1.2,
                            zorder=29,
                            label="Nodes sampled from seed"
                        )
                        # Linien Seed->Node
                        for sp, npnt in seed_links:
                            ax.plot(
                                [sp[0], npnt[0]],
                                [sp[1], npnt[1]],
                                linewidth=1.0,
                                alpha=0.6,
                                zorder=28
                            )

            # ------------------------------------------------------------
            # MODE 3: Max–Min – Kandidatenwolke + selected (letzter Schritt)
            # ------------------------------------------------------------
            if mode == "mode3_max_min":
                cand_log = getattr(planner, "_mode3_candidates_log", []) or []
                if cand_log:
                    last = cand_log[-1]
                    cands = last.get("candidates", []) or []
                    sel = last.get("selected", None)

                    if cands:
                        cx = [c[0] for c in cands]
                        cy = [c[1] for c in cands]
                        ax.scatter(
                            cx, cy,
                            marker="o",
                            s=18,
                            facecolors="none",
                            linewidths=1.0,
                            zorder=20,
                            label="Max–Min candidates"
                        )

                    if sel is not None:
                        ax.scatter(
                            [sel[0]], [sel[1]],
                            marker="*",
                            s=200,
                            zorder=21,
                            label="Max–Min selected"
                        )

            # ------------------------------------------------------------
            # MODE 4: Start–Goal Corridor – Achse + Korridor-Nodes
            # ------------------------------------------------------------
            if mode == "mode4_start_goal_corr":
                s = np.array(bench.startList[0], dtype=float)
                g = np.array(bench.goalList[0], dtype=float)

                ax.plot(
                    [s[0], g[0]], [s[1], g[1]],
                    linestyle="--",
                    linewidth=2,
                    alpha=0.8,
                    zorder=18,
                    label="Start–Goal corridor axis"
                )

                sample_log = getattr(planner, "_sample_log", []) or []
                corr_nodes = []
                for r in sample_log:
                    if r.get("mode") != mode:
                        continue
                    if r.get("source") != "start_goal_corridor":
                        continue
                    nid = int(r["node_id"])
                    if nid in pos:
                        corr_nodes.append(nid)

                if corr_nodes:
                    xs = [pos[n][0] for n in corr_nodes]
                    ys = [pos[n][1] for n in corr_nodes]
                    ax.scatter(
                        xs, ys,
                        marker="^",
                        s=max(25, int(0.7 * nodeSize)),
                        zorder=19,
                        label="Corridor-sampled nodes"
                    )

            # Legend nur wenn extra Elemente vorhanden
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc="upper right")

            # Titel
            t = float(row["time_s"]) if pd.notna(row.get("time_s", np.nan)) else np.nan
            L = float(row["path_length"]) if pd.notna(row.get("path_length", np.nan)) else np.nan
            S = float(row["roadmap_size"]) if pd.notna(row.get("roadmap_size", np.nan)) else np.nan
            ax.set_title(
                f"{bench_name} – {mode}\n"
                f"{tag}: run={run_idx}, time={t:.3f}s, len={L:.2f}, size={S:.0f}, seed={seed}"
            )

            plt.tight_layout()
            plt.show()


# ============================================================
# RUN + PLOTS
# ============================================================
benchmarks = [bench_m3]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)
summary = summarize(df)

# 1) Summary-Plots (1×4)
for b in benchmarks:
     plot_benchmark_like_task1(summary, b.name, mode_order=MODE_ORDER)

# Seeds aus denselben Evaluation-Ergebnissen ziehen
SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)

# 2) Best-Run Visualisierung (Sampling-Struktur sichtbar)
BENCH_ORDER = [b.name for b in benchmarks]
visualize_best_per_mode_and_benchmark(
     df, benchmarks, configs,
     nodeSize=80,
     mode_order=MODE_ORDER,
     bench_order=BENCH_ORDER,
     figsize=(7, 7),
     prefer_seed_runs_for_mode12=True,
 )


### Diskussion der Ergebnisse

**Planungszeit**

Die Planungszeiten liegen bei allen Modi im niedrigen Millisekundenbereich, unterscheiden sich jedoch klar im Mittelwert und in der Varianz. Baseline, Mode 3 (Max-Min) und Mode 4 weisen die geringsten mittleren Planungszeiten auf. Bei Mode 4 ist zu beachten, dass der geringe Zeitwert teilweise auf frühe Abbrüche ohne erfolgreiche Lösung (Fallback) zurückzuführen ist. Mode 2 liegt zeitlich leicht über diesen Verfahren, während Mode 1 die höchsten mittleren Planungszeiten und die größte Varianz zeigt. Dies deutet darauf hin, dass die lokale seedbasierte Verdichtung in dieser Umgebung häufig zusätzliche Erweiterungen auslöst, ohne die Sackgassenstruktur der U-Form gezielt zu überwinden. Mode 3 profitiert hingegen davon, dass bereits sehr wenige, global platzierte Knoten ausreichen, um eine valide Umgehung zu finden.

**Kollisionschecks**

Bei den Kollisionschecks zeigt Mode 3 die mit Abstand niedrigsten Werte. Mode 2 liegt ebenfalls deutlich unter Baseline und Mode 1. Baseline befindet sich im mittleren Bereich. Mode 1 weist die höchsten Kollisionscheck-Zahlen sowie eine große Streuung auf. Die lokale Nachverdichtung um Seeds führt im U-Shape-Benchmark häufig zu Kantenprüfungen entlang blockierter Innenbereiche der U-Struktur, ohne die notwendige Umgehung effektiv zu unterstützen.

**Pfadlänge**

Die Pfadlängen unterscheiden sich moderat, zeigen jedoch erkennbare Trends. Mode 2 erzielt im Mittel die kürzesten Pfade, gefolgt von Mode 1. Baseline und Mode 3 liegen darüber, während Mode 4 aufgrund fehlgeschlagener Läufe nicht konsistent bewertet werden kann. Die Best-Case-Visualisierungen zeigen, dass Mode 3 häufig sehr direkte, aber geometrisch grobe Umgehungen erzeugt, während Mode 2 durch zusätzliche lokale Struktur glattere Pfadverläufe entlang der U-Form ermöglicht. Der Pfadgewinn geht dabei jedoch mit einer höheren Roadmapgröße einher.

**Roadmapgröße**

Die Roadmapgrößen unterscheiden sich sehr deutlich. Mode 3 erzeugt mit Abstand die kleinsten Roadmaps, häufig mit nur wenigen Knoten. Dies unterstreicht die Effektivität der Max-Min-Strategie in dieser Umgebung, da eine globale Umgehung der U-Form mit minimaler Struktur möglich ist. Baseline liegt ebenfalls im unteren Bereich. Mode 1 und insbesondere Mode 2 erzeugen deutlich größere Roadmaps, da entlang der Hinderniskanten zahlreiche zusätzliche Knoten ergänzt werden. Die Korridorstrategie führt hier zu einer starken Überverdichtung im inneren Bereich der U-Form.

**Gesamtbewertung und Eignung für den Benchmark**

Der U-Shape-Benchmark erfordert vor allem die Fähigkeit, strukturell irreführende direkte Verbindungen zu vermeiden und eine globale Umgehung effizient zu erkennen. Mode 3 ist hierfür besonders geeignet, da er mit sehr kleinen Roadmaps, wenigen Kollisionschecks und geringen Planungszeiten zuverlässig gültige Lösungen findet. Mode 2 stellt eine sinnvolle Alternative dar, wenn kürzere und glattere Pfade priorisiert werden, erkauft dies jedoch durch eine deutlich größere Roadmap. Baseline liefert stabile Ergebnisse, adressiert die U-Struktur jedoch nicht gezielt. Mode 4 ist für diesen Benchmark ungeeignet, da die Start–Ziel-Korridorstrategie in einer Sackgassen-ähnlichen Struktur zu starker Überverdichtung führt und keine zuverlässige Führung um das Hindernis ermöglicht. Mode 1 zeigt ebenfalls Nachteile, da lokale Verdichtung ohne globale Orientierung zu hohem Mehraufwand führt. Insgesamt zeigt der Benchmark, dass sparsame, global orientierte Abdeckungsstrategien in Sackgassen-ähnlichen Umgebungen deutlich effektiver sind als lokal oder zielgerichtet fokussierte Samplingverfahren.

## Evaluation mit Benchmark 5: Snail

Der Snail-Benchmark stellt eine besonders anspruchsvolle Umgebung dar, die durch eine lange, schmale, spiralförmige Passage geprägt ist. Start und Ziel liegen zwar räumlich nicht extrem weit auseinander, sind jedoch durch die verschachtelte Hindernisstruktur nur über einen sehr spezifischen, winding Pfad miteinander verbunden. Der Benchmark erfordert daher weniger lokale Engstellenauflösung im klassischen Sinn, sondern vielmehr eine globale, strukturtreue Exploration entlang einer stark eingeschränkten Geometrie. Direkte oder lokal verdichtende Strategien laufen hier Gefahr, sich frühzeitig in Sackgassen oder irrelevanten Bereichen festzufahren. Alle fünf Sampling-Modi (Baseline sowie Mode 1–4) wurden auf diesem Benchmark jeweils mit 50 unabhängigen Planungsdurchläufen getestet und evaluiert. Die Balkendiagramme zeigen Mittelwert und Varianz der betrachteten Metriken über erfolgreiche Läufe. Da mehrere Modi in diesem Benchmark keine oder nur extrem wenige gültige Lösungen finden, sind einige Balken leer bzw. nicht aussagekräftig. Die darunter dargestellten Roadmap- und Pfadvisualisierungen zeigen jeweils entweder einen Best-Case (falls vorhanden) oder einen Fallback-Zustand, wenn kein gültiger Pfad gefunden wurde. Der Best-Case wird – analog zu den vorherigen Benchmarks – über eine gewichtete Mehrkriterienbewertung aus Planungszeit, Pfadlänge und Roadmapgröße bestimmt, wobei extreme Ausreißer über Quantilfilter ausgeschlossen werden.

In [ ]:
BENCH_MODE_MAP = None

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


# ============================================================
# 1) Evaluation Helpers
# ============================================================
def path_length_from_graph(graph, path):
    if not path:
        return np.nan
    pts = [graph.nodes[p]["pos"] for p in path]
    return float(sum(
        np.linalg.norm(np.array(pts[i]) - np.array(pts[i + 1]))
        for i in range(len(pts) - 1)
    ))


# ------------------------------------------------------------
# Out-of-bounds => Collision hart machen (falls CC das nicht selbst tut)
# ------------------------------------------------------------
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        lims = _cc.getEnvironmentLimits()
        (xmin, xmax), (ymin, ymax) = lims
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# 2) Run helpers
# ============================================================
def run_one(planner_class, benchmark, config, seed=None):
    """
    Führt genau einen Run aus. Robust gegen Exceptions.
    Zusätzlich: zählt Seeds (Mode 1/2) und "Nodes aus Seeds" (Mode 1/2), damit wir
    später einen Plot-Run wählen können, in dem Seeds wirklich sichtbar sind.
    """
    if seed is not None:
        seed = int(seed)
        random.seed(seed)
        np.random.seed(seed)

    _ensure_oob_is_collision(benchmark.collisionChecker)

    planner = planner_class(benchmark.collisionChecker)

    # optionaler Counter reset
    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    t0 = time.time()
    err = None
    path = []
    try:
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
    except Exception as e:
        err = str(e)
        path = []
    t1 = time.time()

    collisions = getattr(benchmark.collisionChecker, "counter", np.nan)

    # Graph-Metriken
    try:
        size = len(planner.graph.nodes())
    except Exception:
        size = np.nan

    try:
        length = path_length_from_graph(planner.graph, path)
    except Exception:
        length = np.nan

    success = (path is not None) and (path != [])

    # --- NEW: Seeds und Seed-Nodes zählen ---
    seed_used_count = len(getattr(planner, "_seed_points_used", []) or [])
    sample_log = getattr(planner, "_sample_log", []) or []
    seed_sampled_nodes_count = sum(
        1 for r in sample_log
        if (r.get("seed_key") is not None) and (r.get("mode") in ("mode1_seed_gauss", "mode2_seed_dist"))
    )

    return {
        "time_s": (t1 - t0),
        "collision_checks": collisions,
        "path_length": length,
        "roadmap_size": size,
        "success": success,
        "error": err,
        # New fields (für Plot-Auswahl; in Summary ignorieren)
        "seed_used_count": int(seed_used_count),
        "seed_sampled_nodes_count": int(seed_sampled_nodes_count),
    }


def _modes_for_benchmark(bench_name, configs, bench_mode_map=None):
    if bench_mode_map is None:
        return list(configs.keys())
    return bench_mode_map.get(bench_name, list(configs.keys()))


def run_suite(benchmarks, configs, runs=30, base_seed=1234, progress_every=10, bench_mode_map=None):
    rows = []

    total = 0
    for b in benchmarks:
        total += len(_modes_for_benchmark(b.name, configs, bench_mode_map)) * runs

    done = 0
    t0 = time.time()

    for b in benchmarks:
        mode_list = _modes_for_benchmark(b.name, configs, bench_mode_map)

        for mode_name in mode_list:
            cfg = configs[mode_name]

            for r in range(runs):
                seed = base_seed + 1000 * r
                metrics = run_one(EnhancedLazyPRM, b, cfg, seed=seed)

                rows.append({
                    "benchmark": b.name,
                    "mode": mode_name,
                    "run": r,
                    "seed": seed,
                    **metrics
                })

                done += 1
                if progress_every and (done % progress_every == 0):
                    elapsed = time.time() - t0
                    print(f"{done}/{total} done, elapsed {elapsed:.1f}s")

    return pd.DataFrame(rows)


# ============================================================
# 3) Aggregation (Summary)
# ============================================================
def summarize(df):
    df_metrics = df.copy()

    metric_cols = ["time_s", "collision_checks", "path_length", "roadmap_size"]
    for c in metric_cols:
        df_metrics.loc[df_metrics["success"] == False, c] = np.nan

    summary = (df_metrics
        .groupby(["benchmark", "mode"])
        .agg(
            runs=("run", "count"),
            success_rate=("success", "mean"),

            time_mean=("time_s", "mean"),
            time_std=("time_s", "std"),

            coll_mean=("collision_checks", "mean"),
            coll_std=("collision_checks", "std"),

            len_mean=("path_length", "mean"),
            len_std=("path_length", "std"),

            size_mean=("roadmap_size", "mean"),
            size_std=("roadmap_size", "std"),

            n_errors=("error", lambda s: int(s.notna().sum()))
        )
        .reset_index()
    )
    return summary


# ============================================================
# 4) Plot Layout (1×4)
# ============================================================
METRIC_SPECS = [
    ("time_mean", "time_std", "Planungszeit (s)", "gold", "Planungszeit (s)"),
    ("coll_mean", "coll_std", "Kollisionschecks", "tomato", "Kollisionschecks"),
    ("len_mean",  "len_std",  "Pfadlänge",        "cornflowerblue", "Pfadlänge"),
    ("size_mean", "size_std", "Roadmap Größe",    "mediumpurple", "Roadmap Größe (#Nodes)"),
]

def compute_global_ylims(summary_df, metric_specs, pad=0.10):
    ylims = {}
    for mean_col, std_col, *_ in metric_specs:
        mean_vals = summary_df[mean_col].to_numpy(dtype=float)
        std_vals  = summary_df[std_col].to_numpy(dtype=float)
        comb = mean_vals + np.nan_to_num(std_vals, nan=0.0)
        y_max = np.nanmax(comb) if np.any(np.isfinite(comb)) else 1.0
        if not np.isfinite(y_max) or y_max <= 0:
            y_max = 1.0
        ylims[mean_col] = (0.0, y_max * (1.0 + pad))
    return ylims

def plot_benchmark_like_task1(summary_df, benchmark_name, mode_order):
    sub = summary_df[summary_df["benchmark"] == benchmark_name].copy()

    present = [m for m in mode_order if m in set(sub["mode"].values)]
    if present:
        sub["mode"] = pd.Categorical(sub["mode"], categories=present, ordered=True)
        sub = sub.sort_values("mode")
    else:
        sub = sub.sort_values("mode")

    labels = sub["mode"].astype(str).tolist()
    x = np.arange(len(labels), dtype=int)

    fig, axs = plt.subplots(1, 4, figsize=(18, 4))
    fig.suptitle(f"Aufgabe 2: Vergleich Sampling-Modi – {benchmark_name}", fontsize=14)

    GLOBAL_YLIMS = compute_global_ylims(summary_df, METRIC_SPECS, pad=0.10)

    for ax, (mean_col, std_col, title, color, ylabel) in zip(axs, METRIC_SPECS):
        y = sub[mean_col].to_numpy(dtype=float)
        yerr = sub[std_col].to_numpy(dtype=float)
        yerr_plot = np.nan_to_num(yerr, nan=0.0)

        ax.bar(x, y, yerr=yerr_plot, capsize=4, color=color, alpha=0.85)
        ax.set_title(title)
        ax.set_xlabel("Sampling-Modus")
        ax.set_ylabel(ylabel)
        ax.set_ylim(*GLOBAL_YLIMS[mean_col])
        ax.grid(axis="y", alpha=0.3)
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=25, ha="right")

    plt.tight_layout()
    plt.show()


# ============================================================
# 5) Best-Plot pro Mode & Benchmark (mit Sampling-Struktur)
# ============================================================
def _robust_minmax_norm(series, q_low=0.05, q_high=0.95):
    x = series.to_numpy(dtype=float)
    if np.all(~np.isfinite(x)):
        return pd.Series(np.zeros(len(series)), index=series.index)

    lo = np.nanquantile(x, q_low)
    hi = np.nanquantile(x, q_high)
    if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
        return pd.Series(np.zeros(len(series)), index=series.index)

    xn = (series - lo) / (hi - lo)
    xn = xn.clip(0.0, 1.0).fillna(1.0)
    return xn


def _pick_best_run_multicriteria(df_succ, w_time=0.6, w_len=0.25, w_size=0.15,
                                 q_low=0.05, q_high=0.95):
    d = df_succ.copy()

    for col in ["time_s", "path_length", "roadmap_size"]:
        if col not in d.columns:
            d[col] = np.nan

    d["time_n"] = _robust_minmax_norm(d["time_s"], q_low=q_low, q_high=q_high)
    d["len_n"]  = _robust_minmax_norm(d["path_length"], q_low=q_low, q_high=q_high)
    d["size_n"] = _robust_minmax_norm(d["roadmap_size"], q_low=q_low, q_high=q_high)
    d["score"] = w_time * d["time_n"] + w_len * d["len_n"] + w_size * d["size_n"]

    d = d.sort_values(["score", "time_s"], ascending=[True, True])
    return d.iloc[0]


def extract_best_seed_map(
    df,
    prefer_seed_runs_for_mode12=True,
    w_time=0.6, w_len=0.25, w_size=0.15
):
    """
    Gibt dict zurück: seed_map[(benchmark_name, mode_name)] = seed
    verwendet die exakt gleiche Auswahlregel wie visualize_best_per_mode_and_benchmark().
    """
    seed_map = {}

    for (bench_name, mode), df_bm in df.groupby(["benchmark", "mode"]):
        df_bm = df_bm.copy()
        df_succ = df_bm[df_bm["success"] == True].copy()
        if df_succ.empty:
            continue

        df_pick = df_succ
        if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
            df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
            if not df_seed.empty:
                df_pick = df_seed

        row = _pick_best_run_multicriteria(
            df_pick, w_time=w_time, w_len=w_len, w_size=w_size
        )
        seed_map[(bench_name, mode)] = int(row["seed"])

    return seed_map

def visualize_best_per_mode_and_benchmark(
    df, benchmarks, configs, nodeSize=80,
    mode_order=None, bench_order=None, figsize=(7, 7),
    w_time=0.6, w_len=0.25, w_size=0.15,
    prefer_seed_runs_for_mode12=True,
):
    """
    Plot-Regel:
    - Mode 1/2: zeige Seed (X) + Nodes-aus-Seed + Seed->Node-Linien,
      aber NUR wenn Seeds tatsächlich benutzt wurden.
      Falls prefer_seed_runs_for_mode12=True, wird ein erfolgreicher Run bevorzugt,
      der seed_used_count > 0 hat (sonst sieht man sonst oft "keine Seeds", weil
      der Planner zufällig direkt eine kollisionsfreie Lösung findet).
    - Mode 3: zeige Kandidatenwolke + selected (Stern) aus dem Planner-Log.
    - Mode 4: zeige Start–Goal-Achse + Korridor-Nodes.
    - Baseline: keine Seeds, keine Hotspots.
    """
    bench_by_name = {b.name: b for b in benchmarks}

    if bench_order is None:
        bench_order = [b.name for b in benchmarks]
    if mode_order is None:
        mode_order = list(configs.keys())

    for bench_name in bench_order:
        bench = bench_by_name[bench_name]

        modes_present = [m for m in mode_order if ((df["benchmark"] == bench_name) & (df["mode"] == m)).any()]
        for mode in modes_present:
            df_bm = df[(df["benchmark"] == bench_name) & (df["mode"] == mode)].copy()
            if df_bm.empty:
                continue

            df_succ = df_bm[df_bm["success"] == True].copy()

            # --- Auswahlregel: Mode1/2 bevorzugt Run mit Seeds ---
            df_pick = df_succ
            if prefer_seed_runs_for_mode12 and mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                df_seed = df_succ[df_succ["seed_used_count"] > 0].copy()
                if not df_seed.empty:
                    df_pick = df_seed  # nur wenn es wirklich solche Runs gibt

            if not df_pick.empty:
                row = _pick_best_run_multicriteria(df_pick, w_time=w_time, w_len=w_len, w_size=w_size)
                tag = f"BEST (score={row['score']:.3f}; w_t={w_time}, w_l={w_len}, w_s={w_size})"
            else:
                row = df_bm.sort_values("run").iloc[-1]
                tag = "FALLBACK (no success)"

            run_idx = int(row["run"])
            seed = int(row["seed"])

            random.seed(seed)
            np.random.seed(seed)
            _ensure_oob_is_collision(bench.collisionChecker)

            planner = EnhancedLazyPRM(bench.collisionChecker)
            try:
                sol = planner.planPath(bench.startList, bench.goalList, configs[mode])
            except Exception as e:
                sol = []
                tag = f"{tag} – ERROR: {e}"

            fig, ax = plt.subplots(figsize=figsize)
            lazyPRMVisualize(planner, sol, ax=ax, nodeSize=nodeSize)

            # Achsen fixieren
            (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect("equal", adjustable="box")

            pos = nx.get_node_attributes(planner.graph, "pos")

            # ------------------------------------------------------------
            # MODE 1/2: Seed + Nodes-aus-Seed + Seed->Node Links
            # ------------------------------------------------------------
            if mode in ("mode1_seed_gauss", "mode2_seed_dist"):
                seeds = getattr(planner, "_seed_points_used", []) or []
                sample_log = getattr(planner, "_sample_log", []) or []

                # Seeds plotten (nur wenn wirklich genutzt)
                if seeds:
                    sx = [p[0] for p in seeds]
                    sy = [p[1] for p in seeds]
                    ax.scatter(
                        sx, sy,
                        marker="x",
                        s=max(70, int(1.3 * nodeSize)),
                        linewidths=2.5,
                        zorder=30,
                        label="Seed used (Mode 1/2)"
                    )

                    # Nodes, die aus Seeds erzeugt wurden (seed_key != None)
                    seed_nodes = []
                    seed_links = []  # (seed_point, node_point)

                    # Mapping seed_key -> seed_point
                    seed_map = {}
                    for sp in seeds:
                        k = (round(float(sp[0]), 3), round(float(sp[1]), 3))
                        seed_map[k] = [float(sp[0]), float(sp[1])]

                    for r in sample_log:
                        if r.get("mode") != mode:
                            continue
                        k = r.get("seed_key", None)
                        if k is None:
                            continue
                        nid = int(r["node_id"])
                        if nid not in pos:
                            continue
                        seed_nodes.append(nid)
                        sp = seed_map.get(k, r.get("seed", None))
                        if sp is not None:
                            seed_links.append((sp, pos[nid]))

                    if seed_nodes:
                        nxs = [pos[n][0] for n in seed_nodes]
                        nys = [pos[n][1] for n in seed_nodes]
                        ax.scatter(
                            nxs, nys,
                            marker="o",
                            s=max(35, int(0.8 * nodeSize)),
                            edgecolors="black",
                            linewidths=1.2,
                            zorder=29,
                            label="Nodes sampled from seed"
                        )
                        # Linien Seed->Node
                        for sp, npnt in seed_links:
                            ax.plot(
                                [sp[0], npnt[0]],
                                [sp[1], npnt[1]],
                                linewidth=1.0,
                                alpha=0.6,
                                zorder=28
                            )

            # ------------------------------------------------------------
            # MODE 3: Max–Min – Kandidatenwolke + selected (letzter Schritt)
            # ------------------------------------------------------------
            if mode == "mode3_max_min":
                cand_log = getattr(planner, "_mode3_candidates_log", []) or []
                if cand_log:
                    last = cand_log[-1]
                    cands = last.get("candidates", []) or []
                    sel = last.get("selected", None)

                    if cands:
                        cx = [c[0] for c in cands]
                        cy = [c[1] for c in cands]
                        ax.scatter(
                            cx, cy,
                            marker="o",
                            s=18,
                            facecolors="none",
                            linewidths=1.0,
                            zorder=20,
                            label="Max–Min candidates"
                        )

                    if sel is not None:
                        ax.scatter(
                            [sel[0]], [sel[1]],
                            marker="*",
                            s=200,
                            zorder=21,
                            label="Max–Min selected"
                        )

            # ------------------------------------------------------------
            # MODE 4: Start–Goal Corridor – Achse + Korridor-Nodes
            # ------------------------------------------------------------
            if mode == "mode4_start_goal_corr":
                s = np.array(bench.startList[0], dtype=float)
                g = np.array(bench.goalList[0], dtype=float)

                ax.plot(
                    [s[0], g[0]], [s[1], g[1]],
                    linestyle="--",
                    linewidth=2,
                    alpha=0.8,
                    zorder=18,
                    label="Start–Goal corridor axis"
                )

                sample_log = getattr(planner, "_sample_log", []) or []
                corr_nodes = []
                for r in sample_log:
                    if r.get("mode") != mode:
                        continue
                    if r.get("source") != "start_goal_corridor":
                        continue
                    nid = int(r["node_id"])
                    if nid in pos:
                        corr_nodes.append(nid)

                if corr_nodes:
                    xs = [pos[n][0] for n in corr_nodes]
                    ys = [pos[n][1] for n in corr_nodes]
                    ax.scatter(
                        xs, ys,
                        marker="^",
                        s=max(25, int(0.7 * nodeSize)),
                        zorder=19,
                        label="Corridor-sampled nodes"
                    )

            # Legend nur wenn extra Elemente vorhanden
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc="upper right")

            # Titel
            t = float(row["time_s"]) if pd.notna(row.get("time_s", np.nan)) else np.nan
            L = float(row["path_length"]) if pd.notna(row.get("path_length", np.nan)) else np.nan
            S = float(row["roadmap_size"]) if pd.notna(row.get("roadmap_size", np.nan)) else np.nan
            ax.set_title(
                f"{bench_name} – {mode}\n"
                f"{tag}: run={run_idx}, time={t:.3f}s, len={L:.2f}, size={S:.0f}, seed={seed}"
            )

            plt.tight_layout()
            plt.show()


# ============================================================
# RUN + PLOTS
# ============================================================
benchmarks = [bench_m4]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)
summary = summarize(df)

# Seeds aus denselben Evaluation-Ergebnissen ziehen
SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)

# 1) Summary-Plots (1×4)
for b in benchmarks:
     plot_benchmark_like_task1(summary, b.name, mode_order=MODE_ORDER)

# 2) Best-Run Visualisierung (Sampling-Struktur sichtbar)
BENCH_ORDER = [b.name for b in benchmarks]
visualize_best_per_mode_and_benchmark(
     df, benchmarks, configs,
     nodeSize=80,
     mode_order=MODE_ORDER,
     bench_order=BENCH_ORDER,
     figsize=(7, 7),
     prefer_seed_runs_for_mode12=True,
 )


### Diskussion der Ergebnisse

Das uniforme Sampling scheitert zuverlässig an der Struktur des Benchmarks. Obwohl eine sehr große Roadmap aufgebaut wird, gelingt keine konsistente Erschließung der Spiralstruktur. Die Roadmap wächst ungerichtet im äußeren Bereich, ohne systematisch in tiefere Bereiche der Spirale vorzudringen. Dies führt zu hohen Roadmapgrößen und moderaten Planungszeiten, jedoch ohne erfolgreiche Pfadfindung.

Die seedbasierte lokale Verdichtung verstärkt dieses Problem. Sampling konzentriert sich stark um bestehende Knoten, die sich überwiegend im äußeren Bereich der Spirale befinden. Dadurch entstehen sehr große Roadmaps und viele Kollisionschecks, ohne dass die innere Struktur der Spirale zuverlässig erreicht wird. Die lokale Strategie verhindert eine effektive globale Exploration.

Auch Mode 2 leidet unter einer Überkonzentration entlang bereits erschlossener Bereiche. Zwar ist das Sampling etwas gleichmäßiger verteilt als bei Mode 1, dennoch bleibt die Exploration entlang der Spiralrichtung unzureichend. Die Roadmap wächst stark an, ohne dass eine valide Lösung gefunden wird. Der zusätzliche Aufwand führt nicht zu einem strukturellen Fortschritt.

Mode 3 ist der einzige Ansatz, der den strukturellen Anforderungen des Snail-Benchmarks gerecht wird. Durch die gezielte Auswahl von Knoten mit maximalem Abstand zur bestehenden Roadmap erzwingt das Verfahren eine globale Abdeckung des freien Raums. Dadurch wird die Spiralstruktur schrittweise erschlossen, bis eine vollständige Verbindung von Start zu Ziel entsteht. Der erhöhte Rechenaufwand ist dabei kein Nebenprodukt, sondern eine notwendige Voraussetzung für den Planungserfolg.

Die Start–Ziel-Korridorstrategie ist für den Snail-Benchmark ungeeignet. Der direkte Korridor zwischen Start und Ziel verläuft durch vollständig blockierte Bereiche der Spirale. Das Sampling wird dadurch systematisch in nicht passierbare Regionen gelenkt. Dies führt zu sehr großen Roadmaps und erhöhten Kollisionschecks, ohne eine erfolgreiche Pfadfindung zu ermöglichen.

**Gesamtbewertung und Eignung für den Benchmark**

Der Snail-Benchmark erfordert eine konsequent globale, abdeckungsorientierte Exploration des Konfigurationsraums. Lokale Verdichtung, distanzbasierte Seed-Strategien oder Start–Ziel-Fokussierung sind in dieser Umgebung strukturell ungeeignet und führen zuverlässig zum Scheitern.
Mode 3 (Max-Min) ist der einzige untersuchte Ansatz, der diese Anforderungen erfüllt. Der erhöhte Rechenaufwand in Form von Planungszeit und Kollisionschecks ist notwendig, um die komplexe Spiralstruktur systematisch zu erschließen. Insgesamt zeigt der Benchmark klar, dass in stark verschachtelten Sackgassen-Umgebungen eine globale Abdeckungsstrategie unverzichtbar ist.

# 2DoF Punktroboter

Zur ergänzenden qualitativen Analyse der Node-Enhancementstrategien wird der 2-DOF-Punktroboter betrachtet. Dieses Robotermodell stellt die einfachste untersuchte Systemklasse dar und dient als Referenz, um die Wirkungsweise der unterschiedlichen Sampling-Modi anschaulich zu untersuchen. Der Punktroboter bewegt sich rein translational im zweidimensionalen Arbeitsraum, sodass Konfigurationsraum und Arbeitsraum identisch sind. Obwohl der Roboter kinematisch als Punkt modelliert ist, wird ein endlicher Roboter­radius in der Kollisionsprüfung berücksichtigt. Die Hindernisse werden somit effektiv um diesen Radius aufgeblasen, sodass auch für den Punktroboter realistische Abstands- und Kollisionsbedingungen gelten. Im Vergleich zum 2-DOF-Planarroboter entfallen jedoch rotationsabhängige Freiheitsgrade und kinematische Kopplungen, wodurch die geometrischen Effekte der Sampling-Strategien besonders klar sichtbar werden. Der Punktroboter eignet sich daher hervorragend als Analyse-Baseline, um Unterschiede in Samplingdichte, Roadmapstruktur und Pfadführung isoliert zu untersuchen, ohne dass diese durch komplexe Gelenkkinematik oder aufwendige Kollisionsmodelle überlagert werden.

### Benchmark 1: Baseline_CircleField

Die dargestellten Animationen zeigen für jeden Sampling-Modus einen exemplarischen, zuvor berechneten Pfad innerhalb der aufgebauten Roadmap im Benchmark CircleField. Es ist zu beachten, dass die Animation ausschließlich der Visualisierung des Pfades dient. Ein eigener Kollisionschecker ist in der Animation nicht enthalten; Änderungen an Robotergeometrie oder Benchmark erfordern daher eine erneute Planung im Evaluationsschritt. In den Grafiken werden die strukturellen Unterschiede der Sampling-Modi deutlich sichtbar. Die Baseline erzeugt eine relativ gleichmäßige Verteilung von Knoten im gesamten Arbeitsraum, was zu stabilen, aber wenig zielgerichteten Pfaden führt. Mode 1 und Mode 2 zeigen hingegen eine ausgeprägte lokale Verdichtung entlang der späteren Pfadregion. Die Sampling-Strategien konzentrieren sich sichtbar auf problematische Bereiche zwischen Hindernissen, wodurch kompaktere und geometrisch besser angepasste Pfade entstehen können. Mode 3 hebt sich durch eine sehr sparsame Roadmapstruktur ab. Die Knoten liegen weit auseinander und decken den Raum nur grob ab, reichen jedoch aus, um eine gültige Verbindung zwischen Start und Ziel herzustellen. Der resultierende Pfad ist entsprechend gröber, verdeutlicht aber die Effizienz der Max-Min-Strategie im Hinblick auf minimale Roadmapgröße. Mode 4 erzeugt schließlich eine deutlich start-ziel-gerichtete Struktur. Die Roadmap folgt sichtbar der Verbindung zwischen Start und Ziel, ohne den gesamten Raum gleichmäßig zu explorieren.

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import networkx as nx
import matplotlib.patches as patches

from IPython.display import HTML, display
from IPLazyPRM_Task2 import EnhancedLazyPRM

mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 256


# ============================================================
# OOB-Guard (wie Evaluation)
# ============================================================
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        (xmin, xmax), (ymin, ymax) = _cc.getEnvironmentLimits()
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# Helpers fürs Zeichnen/Animation (Prof-Stil)
# ============================================================
def interpolate_line(a, b, step=0.5):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    dist = float(np.linalg.norm(b - a))
    if dist < 1e-12:
        return [a.tolist()]
    n = max(1, int(np.ceil(dist / float(step))))
    return [(a + (i / n) * (b - a)).tolist() for i in range(n + 1)]

def get_ws_limits(bench):
    (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
    return (xmin, xmax, ymin, ymax)

def draw_obstacles_from_scene(ax, cc, alpha=0.25, edgecolor="white", linewidth=2):
    scene = getattr(cc, "scene", {})
    if not isinstance(scene, dict) or not scene:
        return
    for geom in scene.values():
        if geom is None or getattr(geom, "is_empty", False):
            continue
        gtype = getattr(geom, "geom_type", None)
        if gtype == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        elif gtype == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        else:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
            except Exception:
                pass

def visualize_prm_in_axis(planner, solution_nodes, ax, roadmap_alpha=0.15, node_alpha=0.15):
    G = planner.graph
    pos = nx.get_node_attributes(G, "pos")
    if not pos:
        return

    for u, v in G.edges():
        if u in pos and v in pos:
            ax.plot([pos[u][0], pos[v][0]],
                    [pos[u][1], pos[v][1]],
                    alpha=roadmap_alpha)

    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    ax.scatter(xs, ys, s=8, alpha=node_alpha)

    if solution_nodes:
        sol_xy = [pos[n] for n in solution_nodes if n in pos]
        if len(sol_xy) >= 2:
            ax.plot([p[0] for p in sol_xy], [p[1] for p in sol_xy], linewidth=3)


# ============================================================
# Robot-Parameter aus Config holen
# ============================================================
def _robot_radius_from_mode_config(mode_cfg, cc=None, robot_visual="robot"):
    """
    robot_visual:
      - "robot":   zeichnet robotRadius
      - "clearance": zeichnet robotRadius + safetyMargin (oder cfg["clearance"])
    """
    cfg = mode_cfg or {}

    # 1) explizite Clearance, falls vorhanden
    if robot_visual == "clearance" and ("clearance" in cfg):
        return float(cfg["clearance"])

    # 2) robotRadius + safetyMargin
    r = float(cfg.get("robotRadius", 0.0) or 0.0)
    if robot_visual == "clearance":
        s = float(cfg.get("safetyMargin", 0.0) or 0.0)
        return r + s

    # 3) nur Roboterradius
    if r > 0.0:
        return r

    # 4) Fallback: CollisionChecker (falls du dort min_clearance gesetzt hast)
    if cc is not None and hasattr(cc, "min_clearance"):
        return float(getattr(cc, "min_clearance", 0.0) or 0.0)

    return 0.0


# ============================================================
# Animation: PlanPath-Aufruf exakt wie Evaluation (WICHTIG!)
# ============================================================
def animate_point_robot_eval_exact(
    bench,
    mode_name,
    seed,
    configs,
    interp_step=0.5,
    fig_size=(7, 7),
    obstacle_alpha=0.25,
    robot_visual="robot",     # NEU: "robot" oder "clearance"
    draw_robot_fill=False,
    robot_alpha=0.35
):
    cc = bench.collisionChecker

    # exakt wie Evaluation:
    random.seed(int(seed))
    np.random.seed(int(seed))
    _ensure_oob_is_collision(cc)

    planner = EnhancedLazyPRM(cc)

    mode_cfg = configs.get(mode_name, {}) or {}
    robot_radius = _robot_radius_from_mode_config(mode_cfg, cc=cc, robot_visual=robot_visual)

    sol = planner.planPath(bench.startList, bench.goalList, mode_cfg)

    ##################################################################################################################################################Debug
    # --- Debug: Mode4 wirklich aktiv? ---
    pos = nx.get_node_attributes(planner.graph, "pos")
    sample_log = getattr(planner, "_sample_log", []) or []
    
    n_nodes = len(pos)
    n_edges = planner.graph.number_of_edges()
    
    src_corr = sum(1 for r in sample_log if r.get("source") == "start_goal_corridor")
    src_uni  = sum(1 for r in sample_log if r.get("source") in ("uniform", "uniform_fallback"))
    
    print(
        f"[{bench.name} | {mode_name}] nodes={n_nodes} edges={n_edges} | "
        f"stats={getattr(planner,'_stats',None)} | "
        f"log_total={len(sample_log)} corr_src={src_corr} uni_src={src_uni} | "
        f"start={getattr(planner,'_start_for_mode4',None)} goal={getattr(planner,'_goal_for_mode4',None)}"
    )
    ####################################################################################################################################################Debug


    if not sol or len(sol) < 2:
        print(f"[{bench.name}] ❌ Kein Pfad gefunden (seed={seed}, mode={mode_name})")
        return None

    sol_pos = [planner.graph.nodes[n]["pos"] for n in sol]

    i_sol = [sol_pos[0]]
    for i in range(1, len(sol_pos)):
        i_sol += interpolate_line(sol_pos[i-1], sol_pos[i], interp_step)[1:]

    ws_limits = get_ws_limits(bench)
    frames = len(i_sol)

    fig_local = plt.figure(figsize=fig_size)
    ax = fig_local.add_subplot(1, 1, 1)

    start = bench.startList[0]
    goal  = bench.goalList[0]

    def _anim(t):
        ax.clear()
        xmin, xmax, ymin, ymax = ws_limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal", adjustable="box")
        ax.grid(True, alpha=0.25)

        title = f"{bench.name} | {mode_name} | seed={seed}"
        if robot_radius > 0:
            title += f" | {robot_visual}_r={robot_radius:.2f}"
        ax.set_title(title, fontsize=12)

        draw_obstacles_from_scene(ax, cc, alpha=obstacle_alpha)

        ax.scatter([start[0]], [start[1]], s=120, c="green", zorder=5, label="Start")
        ax.scatter([goal[0]],  [goal[1]],  s=120, c="red",   zorder=5, label="Goal")

        visualize_prm_in_axis(planner, sol, ax=ax)

        p = i_sol[t]
        ax.scatter([p[0]], [p[1]], s=30, c="blue", zorder=7)

        if robot_radius > 0:
            circ = patches.Circle(
                (float(p[0]), float(p[1])),
                radius=float(robot_radius),
                fill=bool(draw_robot_fill),
                alpha=float(robot_alpha) if draw_robot_fill else 1.0,
                linewidth=2.0,
                edgecolor="blue",
                facecolor="blue" if draw_robot_fill else "none",
                zorder=6,
                label="Robot shape"
            )
            ax.add_patch(circ)

        ax.legend(loc="upper right")

    ani = animation.FuncAnimation(fig_local, _anim, frames=frames)
    display(HTML(ani.to_jshtml()))
    plt.close(fig_local)
    return ani


# ============================================================
# RUN: Evaluation -> SeedMap -> Animation
# ============================================================
MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

BENCH_LIST = [
    ("bench_baseline", bench_baseline),
    #("bench_m1",       bench_m1),
    #("bench_m2",       bench_m2),
    #("bench_m3",       bench_m3),
    #("bench_m4",       bench_m4),
]

benchmarks = [b for _, b in BENCH_LIST]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)

SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)
print("SEED_MAP entries:", len(SEED_MAP))

for bench_label, bench_obj in BENCH_LIST:
    bname = bench_obj.name
    print("\n" + "#" * 90)
    print("BENCHMARK:", bench_label, "|", bname)
    print("#" * 90)

    for mode_name in MODE_ORDER:
        seed = SEED_MAP.get((bname, mode_name), None)
        if seed is None:
            print(f"SKIP: {bname} | {mode_name} (kein erfolgreicher Run in df)")
            continue

        print("\n" + "=" * 70)
        print(f"ANIMATION: {bench_label} | {mode_name} | seed={seed}")

        # WICHTIG: kein robot_radius mehr übergeben
        ani = animate_point_robot_eval_exact(
            bench=bench_obj,
            mode_name=mode_name,
            seed=seed,
            configs=configs,
            interp_step=0.5,
            fig_size=(7, 7),
            obstacle_alpha=0.25,
            robot_visual="robot",      # oder "clearance"
            draw_robot_fill=False,
            robot_alpha=0.25
        )


### Benchmark 2: WallTinyDoor

Die dargestellten Animationen zeigen für jeden Sampling-Modus einen exemplarischen, zuvor berechneten Pfad innerhalb der aufgebauten Roadmap im Benchmark WallTinyDoor.Dieser Benchmark ist durch eine schmale Passage zwischen zwei Wandsegmenten geprägt, wodurch sich die Unterschiede der Sampling-Strategien besonders deutlich zeigen. In den Pfadgrafiken wird sichtbar, wie stark die Modi darauf reagieren, diese Engstelle gezielt aufzulösen oder zu umgehen. Die Baseline erzeugt eine relativ gleichmäßige, aber grobe Roadmap. Der Pfad durch die schmale Tür ist funktional, jedoch wenig angepasst an die Engstelle und nutzt nur wenige Knoten in deren unmittelbarer Umgebung. Dadurch entsteht ein vergleichsweise kantiger Pfadverlauf, der die Passage zwar meistert, aber ohne gezielte lokale Struktur. Mode 1 zeigt eine klar erkennbare lokale Verdichtung entlang der Engstelle. Um die kritischen Bereiche werden zusätzliche Knoten erzeugt, was in den Grafiken durch eine kompakte Struktur nahe der Tür sichtbar wird. Der resultierende Pfad verläuft deutlich kontrollierter durch die Passage und nutzt diese Verdichtung effektiv aus. Ein ähnliches, jedoch noch stärker ausgeprägtes Verhalten zeigt Mode 2. Die Roadmap weist eine hohe Knotendichte entlang des Durchgangs auf, was zu einem sehr stabilen, aber teilweise überstrukturierten Pfad führt. In den Animationen ist sichtbar, dass der Roboter entlang der Engstelle auf viele nahe beieinanderliegende Knoten zurückgreifen kann, was Robustheit schafft, jedoch auf Kosten der Roadmapgröße. Mode 3 unterscheidet sich deutlich von den seedbasierten Ansätzen. Die Roadmap ist sehr sparsam aufgebaut und enthält nur wenige, weit auseinanderliegende Knoten. Diese reichen jedoch aus, um eine Verbindung durch die Tür herzustellen. Der Pfad ist entsprechend grob und wenig geglättet, verdeutlicht aber, dass bereits eine minimale, gut platzierte Struktur zur Lösung ausreichen kann. Mode 4 zeigt eine stark start-ziel-gerichtete Struktur. Die Roadmap folgt primär der Verbindungslinie zwischen Start und Ziel, wodurch die Engstelle zwar passiert wird, jedoch ohne gezielte lokale Anpassung. Insgesamt verdeutlichen die Animationen für den Punktroboter im WallTinyDoor-Benchmark sehr anschaulich, wie unterschiedlich die Sampling-Modi mit einer schmalen Passage umgehen. Während seedbasierte Strategien die Engstelle gezielt lokal auflösen, setzen abdeckungs- oder richtungsbasierte Ansätze eher auf eine globale Verbindung. 

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import networkx as nx
import matplotlib.patches as patches

from IPython.display import HTML, display
from IPLazyPRM_Task2 import EnhancedLazyPRM

mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 256


# ============================================================
# OOB-Guard (wie Evaluation)
# ============================================================
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        (xmin, xmax), (ymin, ymax) = _cc.getEnvironmentLimits()
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# Helpers fürs Zeichnen/Animation (Prof-Stil)
# ============================================================
def interpolate_line(a, b, step=0.5):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    dist = float(np.linalg.norm(b - a))
    if dist < 1e-12:
        return [a.tolist()]
    n = max(1, int(np.ceil(dist / float(step))))
    return [(a + (i / n) * (b - a)).tolist() for i in range(n + 1)]

def get_ws_limits(bench):
    (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
    return (xmin, xmax, ymin, ymax)

def draw_obstacles_from_scene(ax, cc, alpha=0.25, edgecolor="white", linewidth=2):
    scene = getattr(cc, "scene", {})
    if not isinstance(scene, dict) or not scene:
        return
    for geom in scene.values():
        if geom is None or getattr(geom, "is_empty", False):
            continue
        gtype = getattr(geom, "geom_type", None)
        if gtype == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        elif gtype == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        else:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
            except Exception:
                pass

def visualize_prm_in_axis(planner, solution_nodes, ax, roadmap_alpha=0.15, node_alpha=0.15):
    G = planner.graph
    pos = nx.get_node_attributes(G, "pos")
    if not pos:
        return

    for u, v in G.edges():
        if u in pos and v in pos:
            ax.plot([pos[u][0], pos[v][0]],
                    [pos[u][1], pos[v][1]],
                    alpha=roadmap_alpha)

    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    ax.scatter(xs, ys, s=8, alpha=node_alpha)

    if solution_nodes:
        sol_xy = [pos[n] for n in solution_nodes if n in pos]
        if len(sol_xy) >= 2:
            ax.plot([p[0] for p in sol_xy], [p[1] for p in sol_xy], linewidth=3)


# ============================================================
# Robot-Parameter aus Config holen
# ============================================================
def _robot_radius_from_mode_config(mode_cfg, cc=None, robot_visual="robot"):
    """
    robot_visual:
      - "robot":   zeichnet robotRadius
      - "clearance": zeichnet robotRadius + safetyMargin (oder cfg["clearance"])
    """
    cfg = mode_cfg or {}

    # 1) explizite Clearance, falls vorhanden
    if robot_visual == "clearance" and ("clearance" in cfg):
        return float(cfg["clearance"])

    # 2) robotRadius + safetyMargin
    r = float(cfg.get("robotRadius", 0.0) or 0.0)
    if robot_visual == "clearance":
        s = float(cfg.get("safetyMargin", 0.0) or 0.0)
        return r + s

    # 3) nur Roboterradius
    if r > 0.0:
        return r

    # 4) Fallback: CollisionChecker (falls du dort min_clearance gesetzt hast)
    if cc is not None and hasattr(cc, "min_clearance"):
        return float(getattr(cc, "min_clearance", 0.0) or 0.0)

    return 0.0


# ============================================================
# Animation: PlanPath-Aufruf exakt wie Evaluation (WICHTIG!)
# ============================================================
def animate_point_robot_eval_exact(
    bench,
    mode_name,
    seed,
    configs,
    interp_step=0.5,
    fig_size=(7, 7),
    obstacle_alpha=0.25,
    robot_visual="robot",     # NEU: "robot" oder "clearance"
    draw_robot_fill=False,
    robot_alpha=0.35
):
    cc = bench.collisionChecker

    # exakt wie Evaluation:
    random.seed(int(seed))
    np.random.seed(int(seed))
    _ensure_oob_is_collision(cc)

    planner = EnhancedLazyPRM(cc)

    mode_cfg = configs.get(mode_name, {}) or {}
    robot_radius = _robot_radius_from_mode_config(mode_cfg, cc=cc, robot_visual=robot_visual)

    sol = planner.planPath(bench.startList, bench.goalList, mode_cfg)

    if not sol or len(sol) < 2:
        print(f"[{bench.name}] ❌ Kein Pfad gefunden (seed={seed}, mode={mode_name})")
        return None

    sol_pos = [planner.graph.nodes[n]["pos"] for n in sol]

    i_sol = [sol_pos[0]]
    for i in range(1, len(sol_pos)):
        i_sol += interpolate_line(sol_pos[i-1], sol_pos[i], interp_step)[1:]

    ws_limits = get_ws_limits(bench)
    frames = len(i_sol)

    fig_local = plt.figure(figsize=fig_size)
    ax = fig_local.add_subplot(1, 1, 1)

    start = bench.startList[0]
    goal  = bench.goalList[0]

    def _anim(t):
        ax.clear()
        xmin, xmax, ymin, ymax = ws_limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal", adjustable="box")
        ax.grid(True, alpha=0.25)

        title = f"{bench.name} | {mode_name} | seed={seed}"
        if robot_radius > 0:
            title += f" | {robot_visual}_r={robot_radius:.2f}"
        ax.set_title(title, fontsize=12)

        draw_obstacles_from_scene(ax, cc, alpha=obstacle_alpha)

        ax.scatter([start[0]], [start[1]], s=120, c="green", zorder=5, label="Start")
        ax.scatter([goal[0]],  [goal[1]],  s=120, c="red",   zorder=5, label="Goal")

        visualize_prm_in_axis(planner, sol, ax=ax)

        p = i_sol[t]
        ax.scatter([p[0]], [p[1]], s=30, c="blue", zorder=7)

        if robot_radius > 0:
            circ = patches.Circle(
                (float(p[0]), float(p[1])),
                radius=float(robot_radius),
                fill=bool(draw_robot_fill),
                alpha=float(robot_alpha) if draw_robot_fill else 1.0,
                linewidth=2.0,
                edgecolor="blue",
                facecolor="blue" if draw_robot_fill else "none",
                zorder=6,
                label="Robot shape"
            )
            ax.add_patch(circ)

        ax.legend(loc="upper right")

    ani = animation.FuncAnimation(fig_local, _anim, frames=frames)
    display(HTML(ani.to_jshtml()))
    plt.close(fig_local)
    return ani


# ============================================================
# RUN: Evaluation -> SeedMap -> Animation
# ============================================================
MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

BENCH_LIST = [
    #("bench_baseline", bench_baseline),
    ("bench_m1",       bench_m1),
    #("bench_m2",       bench_m2),
    #("bench_m3",       bench_m3),
    #("bench_m4",       bench_m4),
]

benchmarks = [b for _, b in BENCH_LIST]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)

SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)
print("SEED_MAP entries:", len(SEED_MAP))

for bench_label, bench_obj in BENCH_LIST:
    bname = bench_obj.name
    print("\n" + "#" * 90)
    print("BENCHMARK:", bench_label, "|", bname)
    print("#" * 90)

    for mode_name in MODE_ORDER:
        seed = SEED_MAP.get((bname, mode_name), None)
        if seed is None:
            print(f"SKIP: {bname} | {mode_name} (kein erfolgreicher Run in df)")
            continue

        print("\n" + "=" * 70)
        print(f"ANIMATION: {bench_label} | {mode_name} | seed={seed}")

        # WICHTIG: kein robot_radius mehr übergeben
        ani = animate_point_robot_eval_exact(
            bench=bench_obj,
            mode_name=mode_name,
            seed=seed,
            configs=configs,
            interp_step=0.5,
            fig_size=(7, 7),
            obstacle_alpha=0.25,
            robot_visual="robot",      # oder "clearance"
            draw_robot_fill=False,
            robot_alpha=0.25
        )


### Benchmark 3: WallTinyWideDoor

Die dargestellten Animationen zeigen für jeden Sampling-Modus einen exemplarischen Pfad innerhalb der jeweils aufgebauten Roadmap im Benchmark WallTinyWideDoor. Im Vergleich zum WallTinyDoor-Benchmark ist die Passage hier breiter und über eine größere Strecke nutzbar, sodass keine hochpräzise lokale Engstellenauflösung erforderlich ist. Stattdessen genügt eine durchgehende, grobe Konnektivität entlang der freien Passage, um Start- und Zielbereich zuverlässig zu verbinden. Diese strukturelle Eigenschaft spiegelt sich deutlich in den Animationen wider. Die Baseline erzeugt eine relativ gleichmäßig verteilte, sparsame Roadmap mit klaren Clustern in gut zugänglichen Bereichen. Der resultierende Pfad verläuft stabil durch den oberen freien Bereich und folgt anschließend dem rechten Rand in Richtung Ziel. Die Pfadführung ist direkt und nutzt die Offenheit der Umgebung effektiv aus, ohne auf zusätzliche lokale Verdichtung angewiesen zu sein. Mode 1 zeigt in den Animationen eine starke lokale Verdichtung um Seedpunkte, insbesondere im linken oberen Bereich nahe dem Start. Dadurch entsteht eine sehr dichte Roadmapstruktur, die viele alternative lokale Verbindungen bietet. Der resultierende Pfad bleibt jedoch geometrisch ähnlich zur Baseline und weist keine wesentliche Verbesserung auf. Die zusätzliche Struktur äußert sich vielmehr in einem komplexeren, lokal stärker segmentierten Pfadverlauf, ohne den globalen Durchgang klarer oder kürzer zu machen. Mode 2 verstärkt diesen Effekt weiter. Die Roadmap ist deutlich breiter und gleichmäßiger über den Raum verteilt, einschließlich Bereichen, die für den erfolgreichen Durchgang nicht zwingend erforderlich sind. In den Animationen zeigt sich ein weniger geradliniger Pfad mit zusätzlichen seitlichen Ausweichbewegungen, insbesondere im rechten Bereich vor dem Ziel. Dies entspricht einer Strategie, die robuste Konnektivität priorisiert, dabei jedoch Umwege und erhöhte strukturelle Komplexität in Kauf nimmt. Mode 3 ist durch eine sehr kompakte Roadmap gekennzeichnet, die nur wenige, weit auseinanderliegende Knoten enthält. Dennoch entsteht ein funktionaler und stabiler Pfad, der dem Verlauf der Baseline ähnelt: Querung durch die breite Passage und anschließender Abstieg zum Ziel. In den Animationen wirkt der Pfad kantiger und weniger fein aufgelöst, bleibt jedoch gültig. Die breite Passage erlaubt es der Max-Min-Strategie, mit minimaler Struktur eine ausreichende globale Abdeckung zu erreichen. Mode 4 erzeugt eine deutlich start–ziel-gerichtete Roadmapstruktur. In den Animationen ist erkennbar, dass das Sampling entlang einer groben Achse zwischen Start und Ziel erfolgt, wodurch die Passage zuverlässig abgedeckt wird. Gleichzeitig entstehen im Start- und Zielbereich seitliche Auslenkungen und lokale Schleifen, da mehrere nahezu gleichwertige Optionen entlang des Korridors erzeugt werden. Der resultierende Pfad ist stabil und zielgerichtet, jedoch strukturell komplexer als bei der Baseline oder Mode 3. Zusammenfassend verdeutlichen die Animationen, dass im WallTinyWideDoor-Benchmark keine feingranulare Engstellenbehandlung erforderlich ist. Baseline und Mode 3 zeigen, dass bereits eine sparsame, grob strukturierte Roadmap ausreicht, um valide und direkte Pfade zu erzeugen. Mode 4 liefert ebenfalls zuverlässige Lösungen, erzeugt jedoch sichtbar mehr Struktur entlang des Korridors. Mode 1 und Mode 2 sind funktionsfähig, führen aber zu einer deutlich erhöhten Roadmapkomplexität, ohne einen klaren geometrischen oder strukturellen Vorteil im Pfadverlauf zu erzielen.

In [ ]:
# NEU

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import networkx as nx
import matplotlib.patches as patches

from IPython.display import HTML, display
from IPLazyPRM_Task2 import EnhancedLazyPRM

mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 256


# ============================================================
# OOB-Guard (wie Evaluation)
# ============================================================
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        (xmin, xmax), (ymin, ymax) = _cc.getEnvironmentLimits()
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# Helpers fürs Zeichnen/Animation (Prof-Stil)
# ============================================================
def interpolate_line(a, b, step=0.5):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    dist = float(np.linalg.norm(b - a))
    if dist < 1e-12:
        return [a.tolist()]
    n = max(1, int(np.ceil(dist / float(step))))
    return [(a + (i / n) * (b - a)).tolist() for i in range(n + 1)]

def get_ws_limits(bench):
    (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
    return (xmin, xmax, ymin, ymax)

def draw_obstacles_from_scene(ax, cc, alpha=0.25, edgecolor="white", linewidth=2):
    scene = getattr(cc, "scene", {})
    if not isinstance(scene, dict) or not scene:
        return
    for geom in scene.values():
        if geom is None or getattr(geom, "is_empty", False):
            continue
        gtype = getattr(geom, "geom_type", None)
        if gtype == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        elif gtype == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        else:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
            except Exception:
                pass

def visualize_prm_in_axis(planner, solution_nodes, ax, roadmap_alpha=0.15, node_alpha=0.15):
    G = planner.graph
    pos = nx.get_node_attributes(G, "pos")
    if not pos:
        return

    for u, v in G.edges():
        if u in pos and v in pos:
            ax.plot([pos[u][0], pos[v][0]],
                    [pos[u][1], pos[v][1]],
                    alpha=roadmap_alpha)

    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    ax.scatter(xs, ys, s=8, alpha=node_alpha)

    if solution_nodes:
        sol_xy = [pos[n] for n in solution_nodes if n in pos]
        if len(sol_xy) >= 2:
            ax.plot([p[0] for p in sol_xy], [p[1] for p in sol_xy], linewidth=3)


# ============================================================
# Robot-Parameter aus Config holen
# ============================================================
def _robot_radius_from_mode_config(mode_cfg, cc=None, robot_visual="robot"):
    """
    robot_visual:
      - "robot":   zeichnet robotRadius
      - "clearance": zeichnet robotRadius + safetyMargin (oder cfg["clearance"])
    """
    cfg = mode_cfg or {}

    # 1) explizite Clearance, falls vorhanden
    if robot_visual == "clearance" and ("clearance" in cfg):
        return float(cfg["clearance"])

    # 2) robotRadius + safetyMargin
    r = float(cfg.get("robotRadius", 0.0) or 0.0)
    if robot_visual == "clearance":
        s = float(cfg.get("safetyMargin", 0.0) or 0.0)
        return r + s

    # 3) nur Roboterradius
    if r > 0.0:
        return r

    # 4) Fallback: CollisionChecker (falls du dort min_clearance gesetzt hast)
    if cc is not None and hasattr(cc, "min_clearance"):
        return float(getattr(cc, "min_clearance", 0.0) or 0.0)

    return 0.0


# ============================================================
# Animation: PlanPath-Aufruf exakt wie Evaluation (WICHTIG!)
# ============================================================
def animate_point_robot_eval_exact(
    bench,
    mode_name,
    seed,
    configs,
    interp_step=0.5,
    fig_size=(7, 7),
    obstacle_alpha=0.25,
    robot_visual="robot",     # NEU: "robot" oder "clearance"
    draw_robot_fill=False,
    robot_alpha=0.35
):
    cc = bench.collisionChecker

    # exakt wie Evaluation:
    random.seed(int(seed))
    np.random.seed(int(seed))
    _ensure_oob_is_collision(cc)

    planner = EnhancedLazyPRM(cc)

    mode_cfg = configs.get(mode_name, {}) or {}
    robot_radius = _robot_radius_from_mode_config(mode_cfg, cc=cc, robot_visual=robot_visual)

    sol = planner.planPath(bench.startList, bench.goalList, mode_cfg)

    if not sol or len(sol) < 2:
        print(f"[{bench.name}] ❌ Kein Pfad gefunden (seed={seed}, mode={mode_name})")
        return None

    sol_pos = [planner.graph.nodes[n]["pos"] for n in sol]

    i_sol = [sol_pos[0]]
    for i in range(1, len(sol_pos)):
        i_sol += interpolate_line(sol_pos[i-1], sol_pos[i], interp_step)[1:]

    ws_limits = get_ws_limits(bench)
    frames = len(i_sol)

    fig_local = plt.figure(figsize=fig_size)
    ax = fig_local.add_subplot(1, 1, 1)

    start = bench.startList[0]
    goal  = bench.goalList[0]

    def _anim(t):
        ax.clear()
        xmin, xmax, ymin, ymax = ws_limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal", adjustable="box")
        ax.grid(True, alpha=0.25)

        title = f"{bench.name} | {mode_name} | seed={seed}"
        if robot_radius > 0:
            title += f" | {robot_visual}_r={robot_radius:.2f}"
        ax.set_title(title, fontsize=12)

        draw_obstacles_from_scene(ax, cc, alpha=obstacle_alpha)

        ax.scatter([start[0]], [start[1]], s=120, c="green", zorder=5, label="Start")
        ax.scatter([goal[0]],  [goal[1]],  s=120, c="red",   zorder=5, label="Goal")

        visualize_prm_in_axis(planner, sol, ax=ax)

        p = i_sol[t]
        ax.scatter([p[0]], [p[1]], s=30, c="blue", zorder=7)

        if robot_radius > 0:
            circ = patches.Circle(
                (float(p[0]), float(p[1])),
                radius=float(robot_radius),
                fill=bool(draw_robot_fill),
                alpha=float(robot_alpha) if draw_robot_fill else 1.0,
                linewidth=2.0,
                edgecolor="blue",
                facecolor="blue" if draw_robot_fill else "none",
                zorder=6,
                label="Robot shape"
            )
            ax.add_patch(circ)

        ax.legend(loc="upper right")

    ani = animation.FuncAnimation(fig_local, _anim, frames=frames)
    display(HTML(ani.to_jshtml()))
    plt.close(fig_local)
    return ani


# ============================================================
# RUN: Evaluation -> SeedMap -> Animation
# ============================================================
MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

BENCH_LIST = [
    #("bench_baseline", bench_baseline),
    #("bench_m1",       bench_m1),
    ("bench_m2",       bench_m2),
    #("bench_m3",       bench_m3),
    #("bench_m4",       bench_m4),
]

benchmarks = [b for _, b in BENCH_LIST]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)

SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)
print("SEED_MAP entries:", len(SEED_MAP))

for bench_label, bench_obj in BENCH_LIST:
    bname = bench_obj.name
    print("\n" + "#" * 90)
    print("BENCHMARK:", bench_label, "|", bname)
    print("#" * 90)

    for mode_name in MODE_ORDER:
        seed = SEED_MAP.get((bname, mode_name), None)
        if seed is None:
            print(f"SKIP: {bname} | {mode_name} (kein erfolgreicher Run in df)")
            continue

        print("\n" + "=" * 70)
        print(f"ANIMATION: {bench_label} | {mode_name} | seed={seed}")

        # WICHTIG: kein robot_radius mehr übergeben
        ani = animate_point_robot_eval_exact(
            bench=bench_obj,
            mode_name=mode_name,
            seed=seed,
            configs=configs,
            interp_step=0.5,
            fig_size=(7, 7),
            obstacle_alpha=0.25,
            robot_visual="robot",      # oder "clearance"
            draw_robot_fill=False,
            robot_alpha=0.25
        )


## Benchmark 4: U-Shape

Die Animationen zeigen für die einzelnen Sampling-Modi exemplarische Pfade innerhalb der jeweils aufgebauten Roadmap im U-Shape-Benchmark. Die Baseline erzeugt eine sehr sparsame Roadmap mit wenigen, weit auseinanderliegenden Knoten. Der resultierende Pfad verläuft grob entlang der linken Randregion und schwenkt anschließend oberhalb der U-Struktur in Richtung Ziel. Die Pfadführung ist geometrisch einfach und wirkt teilweise kantig, bleibt jedoch gültig. 
Die Animation zeigt, dass bereits eine minimale, gleichmäßig verteilte Struktur ausreichen kann, um die Sackgasse global zu umgehen. Bei Mode 1 ist in der Animation eine deutliche Konzentration der Roadmap entlang der linken Außenkante sichtbar. Der Pfad folgt dieser Randstruktur nahezu vollständig und umgeht die U-Form frühzeitig, ohne den oberen freien Raum breit zu nutzen. Die lokale Verdichtung um Seedpunkte führt dazu, dass der Pfad aus mehreren kurzen Segmenten besteht und lokal stark gelenkt wird. Die Umgehung der Sackgasse gelingt, ist jedoch eher das Resultat einer einseitigen Randexploration als einer gezielten globalen Struktur­erfassung. Die zusätzliche lokale Dichte erzeugt keinen klareren oder kürzeren Pfad, sondern hauptsächlich eine höhere strukturelle Komplexität entlang derselben Route. Mode 2 zeigt ebenfalls eine dominante Führung entlang der linken Randregion, allerdings mit etwas gleichmäßigerer Verteilung der Knoten entlang dieser Route. Der Pfad ist visuell stabiler und weniger stark segmentiert als bei Mode 1, bleibt jedoch klar randorientiert. Die distanzbasierte Seed-Strategie verbessert die Konnektivität entlang der gewählten Umgehung, ohne alternative globale Routen zu erschließen. Die Sackgassenstruktur wird nicht explizit aufgelöst, sondern durch eine robuste, aber einseitige Umgehung vermieden. Der Pfad wirkt dadurch konsistenter, bleibt jedoch strukturell ähnlich zu Mode 1. Zusammenfassend zeigen die Animationen, dass der U-Shape-Benchmark primär eine globale Struktur­erkennung erfordert. Verfahren, die den Raum grob, aber systematisch abdecken, sind hier im Vorteil. Mode 3 und in abgeschwächter Form auch die Baseline erfassen die notwendige Umgehung besonders klar. Lokale oder seedbasierte Verdichtung führt zwar zu detaillierteren Roadmaps, verbessert den globalen Pfadverlauf jedoch nicht wesentlich und erhöht vor allem die strukturelle Komplexität. Mode 4 findet keinen Pfad.

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import networkx as nx
import matplotlib.patches as patches

from IPython.display import HTML, display
from IPLazyPRM_Task2 import EnhancedLazyPRM

mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 256


# ============================================================
# OOB-Guard (wie Evaluation)
# ============================================================
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        (xmin, xmax), (ymin, ymax) = _cc.getEnvironmentLimits()
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# Helpers fürs Zeichnen/Animation (Prof-Stil)
# ============================================================
def interpolate_line(a, b, step=0.5):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    dist = float(np.linalg.norm(b - a))
    if dist < 1e-12:
        return [a.tolist()]
    n = max(1, int(np.ceil(dist / float(step))))
    return [(a + (i / n) * (b - a)).tolist() for i in range(n + 1)]

def get_ws_limits(bench):
    (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
    return (xmin, xmax, ymin, ymax)

def draw_obstacles_from_scene(ax, cc, alpha=0.25, edgecolor="white", linewidth=2):
    scene = getattr(cc, "scene", {})
    if not isinstance(scene, dict) or not scene:
        return
    for geom in scene.values():
        if geom is None or getattr(geom, "is_empty", False):
            continue
        gtype = getattr(geom, "geom_type", None)
        if gtype == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        elif gtype == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        else:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
            except Exception:
                pass

def visualize_prm_in_axis(planner, solution_nodes, ax, roadmap_alpha=0.15, node_alpha=0.15):
    G = planner.graph
    pos = nx.get_node_attributes(G, "pos")
    if not pos:
        return

    for u, v in G.edges():
        if u in pos and v in pos:
            ax.plot([pos[u][0], pos[v][0]],
                    [pos[u][1], pos[v][1]],
                    alpha=roadmap_alpha)

    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    ax.scatter(xs, ys, s=8, alpha=node_alpha)

    if solution_nodes:
        sol_xy = [pos[n] for n in solution_nodes if n in pos]
        if len(sol_xy) >= 2:
            ax.plot([p[0] for p in sol_xy], [p[1] for p in sol_xy], linewidth=3)


# ============================================================
# Robot-Parameter aus Config holen
# ============================================================
def _robot_radius_from_mode_config(mode_cfg, cc=None, robot_visual="robot"):
    """
    robot_visual:
      - "robot":   zeichnet robotRadius
      - "clearance": zeichnet robotRadius + safetyMargin (oder cfg["clearance"])
    """
    cfg = mode_cfg or {}

    # 1) explizite Clearance, falls vorhanden
    if robot_visual == "clearance" and ("clearance" in cfg):
        return float(cfg["clearance"])

    # 2) robotRadius + safetyMargin
    r = float(cfg.get("robotRadius", 0.0) or 0.0)
    if robot_visual == "clearance":
        s = float(cfg.get("safetyMargin", 0.0) or 0.0)
        return r + s

    # 3) nur Roboterradius
    if r > 0.0:
        return r

    # 4) Fallback: CollisionChecker (falls du dort min_clearance gesetzt hast)
    if cc is not None and hasattr(cc, "min_clearance"):
        return float(getattr(cc, "min_clearance", 0.0) or 0.0)

    return 0.0


# ============================================================
# Animation: PlanPath-Aufruf exakt wie Evaluation (WICHTIG!)
# ============================================================
def animate_point_robot_eval_exact(
    bench,
    mode_name,
    seed,
    configs,
    interp_step=0.5,
    fig_size=(7, 7),
    obstacle_alpha=0.25,
    robot_visual="robot",     # NEU: "robot" oder "clearance"
    draw_robot_fill=False,
    robot_alpha=0.35
):
    cc = bench.collisionChecker

    # exakt wie Evaluation:
    random.seed(int(seed))
    np.random.seed(int(seed))
    _ensure_oob_is_collision(cc)

    planner = EnhancedLazyPRM(cc)

    mode_cfg = configs.get(mode_name, {}) or {}
    robot_radius = _robot_radius_from_mode_config(mode_cfg, cc=cc, robot_visual=robot_visual)

    sol = planner.planPath(bench.startList, bench.goalList, mode_cfg)

    if not sol or len(sol) < 2:
        print(f"[{bench.name}] ❌ Kein Pfad gefunden (seed={seed}, mode={mode_name})")
        return None

    sol_pos = [planner.graph.nodes[n]["pos"] for n in sol]

    i_sol = [sol_pos[0]]
    for i in range(1, len(sol_pos)):
        i_sol += interpolate_line(sol_pos[i-1], sol_pos[i], interp_step)[1:]

    ws_limits = get_ws_limits(bench)
    frames = len(i_sol)

    fig_local = plt.figure(figsize=fig_size)
    ax = fig_local.add_subplot(1, 1, 1)

    start = bench.startList[0]
    goal  = bench.goalList[0]

    def _anim(t):
        ax.clear()
        xmin, xmax, ymin, ymax = ws_limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal", adjustable="box")
        ax.grid(True, alpha=0.25)

        title = f"{bench.name} | {mode_name} | seed={seed}"
        if robot_radius > 0:
            title += f" | {robot_visual}_r={robot_radius:.2f}"
        ax.set_title(title, fontsize=12)

        draw_obstacles_from_scene(ax, cc, alpha=obstacle_alpha)

        ax.scatter([start[0]], [start[1]], s=120, c="green", zorder=5, label="Start")
        ax.scatter([goal[0]],  [goal[1]],  s=120, c="red",   zorder=5, label="Goal")

        visualize_prm_in_axis(planner, sol, ax=ax)

        p = i_sol[t]
        ax.scatter([p[0]], [p[1]], s=30, c="blue", zorder=7)

        if robot_radius > 0:
            circ = patches.Circle(
                (float(p[0]), float(p[1])),
                radius=float(robot_radius),
                fill=bool(draw_robot_fill),
                alpha=float(robot_alpha) if draw_robot_fill else 1.0,
                linewidth=2.0,
                edgecolor="blue",
                facecolor="blue" if draw_robot_fill else "none",
                zorder=6,
                label="Robot shape"
            )
            ax.add_patch(circ)

        ax.legend(loc="upper right")

    ani = animation.FuncAnimation(fig_local, _anim, frames=frames)
    display(HTML(ani.to_jshtml()))
    plt.close(fig_local)
    return ani


# ============================================================
# RUN: Evaluation -> SeedMap -> Animation
# ============================================================
MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

BENCH_LIST = [
    #("bench_baseline", bench_baseline),
    #("bench_m1",       bench_m1),
    #("bench_m2",       bench_m2),
    ("bench_m3",       bench_m3),
    #("bench_m4",       bench_m4),
]

benchmarks = [b for _, b in BENCH_LIST]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)

SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)
print("SEED_MAP entries:", len(SEED_MAP))

for bench_label, bench_obj in BENCH_LIST:
    bname = bench_obj.name
    print("\n" + "#" * 90)
    print("BENCHMARK:", bench_label, "|", bname)
    print("#" * 90)

    for mode_name in MODE_ORDER:
        seed = SEED_MAP.get((bname, mode_name), None)
        if seed is None:
            print(f"SKIP: {bname} | {mode_name} (kein erfolgreicher Run in df)")
            continue

        print("\n" + "=" * 70)
        print(f"ANIMATION: {bench_label} | {mode_name} | seed={seed}")

        # WICHTIG: kein robot_radius mehr übergeben
        ani = animate_point_robot_eval_exact(
            bench=bench_obj,
            mode_name=mode_name,
            seed=seed,
            configs=configs,
            interp_step=0.5,
            fig_size=(7, 7),
            obstacle_alpha=0.25,
            robot_visual="robot",      # oder "clearance"
            draw_robot_fill=False,
            robot_alpha=0.25
        )


## Benchmark 5: Snail

Die Animationen des Snail-Benchmarks verdeutlichen die besondere Schwierigkeit dieser Umgebung. Die spiral-förmige Hindernisstruktur erzwingt eine lange, verschachtelte Umgehung, bei der direkte oder lokal verbesserte Verbindungen systematisch in Sackgassen führen. Eine erfolgreiche Planung erfordert daher eine schrittweise globale Erschließung des freien Raums entlang der Spiralform. In den Animationen zeigt sich, dass nur Mode 3 (Max-Min) diese Anforderung erfüllt. Der dargestellte Pfad folgt der Spiralstruktur nahezu vollständig und bewegt sich konsequent entlang des freien Korridors von außen nach innen, bis das Ziel erreicht wird. Die Pfadsegmente sind vergleichsweise lang und kantig, was auf die sehr sparsame Roadmap zurückzuführen ist. Dennoch bildet der Pfad die globale Geometrie der Spirale korrekt ab und vermeidet irreführende Abkürzungen. Die Roadmap von Mode 3 ist in der Animation deutlich kompakt. Die Knoten sind weit voneinander entfernt und werden gezielt in bislang unerschlossenen Bereichen platziert. Dadurch entsteht eine schrittweise Fortschrittsbewegung entlang der Spiralrichtung, ohne lokale Überverdichtung oder unnötige Exploration außerhalb des relevanten Korridors. Die Animation verdeutlicht, dass die Max-Min-Strategie nicht versucht, den Raum fein aufzulösen, sondern zunächst dessen globale Struktur zu erfassen. Die fehlenden erfolgreichen Animationen der übrigen Modi lassen sich ebenfalls strukturell erklären. Baseline, Mode 1 und Mode 2 konzentrieren sich entweder zu stark auf lokale Regionen oder verteilen Knoten ungerichtet, sodass die Spiralform nicht systematisch erschlossen wird. Mode 4 scheitert, da eine direkte Start-Ziel-Fokussierung in diesem Benchmark irreführend ist und das Sampling in blockierte Bereiche lenkt. In den entsprechenden Animationen ist eine starke Verdichtung ohne gerichteten Fortschritt sichtbar. Zusammenfassend zeigt die Animation des Snail-Benchmarks, dass eine erfolgreiche Planung nur durch eine globale, abdeckungsorientierte Exploration möglich ist. Mode 3 ist der einzige untersuchte Ansatz, der diese globale Struktur erkennt und schrittweise nutzt. Die Animation macht deutlich, dass der Planungserfolg hier weniger von lokaler Präzision als vielmehr von der korrekten Erfassung der Gesamtgeometrie des freien Raums abhängt.

In [ ]:
# NEU

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import networkx as nx
import matplotlib.patches as patches

from IPython.display import HTML, display
from IPLazyPRM_Task2 import EnhancedLazyPRM

mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 256


# ============================================================
# OOB-Guard (wie Evaluation)
# ============================================================
def _ensure_oob_is_collision(collisionChecker):
    if getattr(collisionChecker, "_oob_guard_enabled", False):
        return

    orig_point_in_collision = collisionChecker.pointInCollision

    def wrapped_point_in_collision(pos, _cc=collisionChecker, _orig=orig_point_in_collision):
        (xmin, xmax), (ymin, ymax) = _cc.getEnvironmentLimits()
        if pos[0] < xmin or pos[0] > xmax or pos[1] < ymin or pos[1] > ymax:
            if hasattr(_cc, "counter"):
                _cc.counter += 1
            return True
        return _orig(pos)

    collisionChecker.pointInCollision = wrapped_point_in_collision
    collisionChecker._oob_guard_enabled = True


# ============================================================
# Helpers fürs Zeichnen/Animation (Prof-Stil)
# ============================================================
def interpolate_line(a, b, step=0.5):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    dist = float(np.linalg.norm(b - a))
    if dist < 1e-12:
        return [a.tolist()]
    n = max(1, int(np.ceil(dist / float(step))))
    return [(a + (i / n) * (b - a)).tolist() for i in range(n + 1)]

def get_ws_limits(bench):
    (xmin, xmax), (ymin, ymax) = bench.collisionChecker.getEnvironmentLimits()
    return (xmin, xmax, ymin, ymax)

def draw_obstacles_from_scene(ax, cc, alpha=0.25, edgecolor="white", linewidth=2):
    scene = getattr(cc, "scene", {})
    if not isinstance(scene, dict) or not scene:
        return
    for geom in scene.values():
        if geom is None or getattr(geom, "is_empty", False):
            continue
        gtype = getattr(geom, "geom_type", None)
        if gtype == "Polygon":
            xs, ys = geom.exterior.xy
            ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        elif gtype == "MultiPolygon":
            for poly in geom.geoms:
                xs, ys = poly.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
        else:
            try:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=alpha, edgecolor=edgecolor, linewidth=linewidth)
            except Exception:
                pass

def visualize_prm_in_axis(planner, solution_nodes, ax, roadmap_alpha=0.15, node_alpha=0.15):
    G = planner.graph
    pos = nx.get_node_attributes(G, "pos")
    if not pos:
        return

    for u, v in G.edges():
        if u in pos and v in pos:
            ax.plot([pos[u][0], pos[v][0]],
                    [pos[u][1], pos[v][1]],
                    alpha=roadmap_alpha)

    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    ax.scatter(xs, ys, s=8, alpha=node_alpha)

    if solution_nodes:
        sol_xy = [pos[n] for n in solution_nodes if n in pos]
        if len(sol_xy) >= 2:
            ax.plot([p[0] for p in sol_xy], [p[1] for p in sol_xy], linewidth=3)


# ============================================================
# Robot-Parameter aus Config holen
# ============================================================
def _robot_radius_from_mode_config(mode_cfg, cc=None, robot_visual="robot"):
    """
    robot_visual:
      - "robot":   zeichnet robotRadius
      - "clearance": zeichnet robotRadius + safetyMargin (oder cfg["clearance"])
    """
    cfg = mode_cfg or {}

    # 1) explizite Clearance, falls vorhanden
    if robot_visual == "clearance" and ("clearance" in cfg):
        return float(cfg["clearance"])

    # 2) robotRadius + safetyMargin
    r = float(cfg.get("robotRadius", 0.0) or 0.0)
    if robot_visual == "clearance":
        s = float(cfg.get("safetyMargin", 0.0) or 0.0)
        return r + s

    # 3) nur Roboterradius
    if r > 0.0:
        return r

    # 4) Fallback: CollisionChecker (falls du dort min_clearance gesetzt hast)
    if cc is not None and hasattr(cc, "min_clearance"):
        return float(getattr(cc, "min_clearance", 0.0) or 0.0)

    return 0.0


# ============================================================
# Animation: PlanPath-Aufruf exakt wie Evaluation (WICHTIG!)
# ============================================================
def animate_point_robot_eval_exact(
    bench,
    mode_name,
    seed,
    configs,
    interp_step=0.5,
    fig_size=(7, 7),
    obstacle_alpha=0.25,
    robot_visual="robot",     # NEU: "robot" oder "clearance"
    draw_robot_fill=False,
    robot_alpha=0.35
):
    cc = bench.collisionChecker

    # exakt wie Evaluation:
    random.seed(int(seed))
    np.random.seed(int(seed))
    _ensure_oob_is_collision(cc)

    planner = EnhancedLazyPRM(cc)

    mode_cfg = configs.get(mode_name, {}) or {}
    robot_radius = _robot_radius_from_mode_config(mode_cfg, cc=cc, robot_visual=robot_visual)

    sol = planner.planPath(bench.startList, bench.goalList, mode_cfg)

    if not sol or len(sol) < 2:
        print(f"[{bench.name}] ❌ Kein Pfad gefunden (seed={seed}, mode={mode_name})")
        return None

    sol_pos = [planner.graph.nodes[n]["pos"] for n in sol]

    i_sol = [sol_pos[0]]
    for i in range(1, len(sol_pos)):
        i_sol += interpolate_line(sol_pos[i-1], sol_pos[i], interp_step)[1:]

    ws_limits = get_ws_limits(bench)
    frames = len(i_sol)

    fig_local = plt.figure(figsize=fig_size)
    ax = fig_local.add_subplot(1, 1, 1)

    start = bench.startList[0]
    goal  = bench.goalList[0]

    def _anim(t):
        ax.clear()
        xmin, xmax, ymin, ymax = ws_limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal", adjustable="box")
        ax.grid(True, alpha=0.25)

        title = f"{bench.name} | {mode_name} | seed={seed}"
        if robot_radius > 0:
            title += f" | {robot_visual}_r={robot_radius:.2f}"
        ax.set_title(title, fontsize=12)

        draw_obstacles_from_scene(ax, cc, alpha=obstacle_alpha)

        ax.scatter([start[0]], [start[1]], s=120, c="green", zorder=5, label="Start")
        ax.scatter([goal[0]],  [goal[1]],  s=120, c="red",   zorder=5, label="Goal")

        visualize_prm_in_axis(planner, sol, ax=ax)

        p = i_sol[t]
        ax.scatter([p[0]], [p[1]], s=30, c="blue", zorder=7)

        if robot_radius > 0:
            circ = patches.Circle(
                (float(p[0]), float(p[1])),
                radius=float(robot_radius),
                fill=bool(draw_robot_fill),
                alpha=float(robot_alpha) if draw_robot_fill else 1.0,
                linewidth=2.0,
                edgecolor="blue",
                facecolor="blue" if draw_robot_fill else "none",
                zorder=6,
                label="Robot shape"
            )
            ax.add_patch(circ)

        ax.legend(loc="upper right")

    ani = animation.FuncAnimation(fig_local, _anim, frames=frames)
    display(HTML(ani.to_jshtml()))
    plt.close(fig_local)
    return ani


# ============================================================
# RUN: Evaluation -> SeedMap -> Animation
# ============================================================
MODE_ORDER = ["baseline_uniform", "mode1_seed_gauss", "mode2_seed_dist", "mode3_max_min", "mode4_start_goal_corr"]

BENCH_LIST = [
    #("bench_baseline", bench_baseline),
    #("bench_m1",       bench_m1),
    #("bench_m2",       bench_m2),
    #("bench_m3",       bench_m3),
    ("bench_m4",       bench_m4),
]

benchmarks = [b for _, b in BENCH_LIST]
df = run_suite(benchmarks, configs, runs=50, base_seed=2025)

SEED_MAP = extract_best_seed_map(df, prefer_seed_runs_for_mode12=True)
print("SEED_MAP entries:", len(SEED_MAP))

for bench_label, bench_obj in BENCH_LIST:
    bname = bench_obj.name
    print("\n" + "#" * 90)
    print("BENCHMARK:", bench_label, "|", bname)
    print("#" * 90)

    for mode_name in MODE_ORDER:
        seed = SEED_MAP.get((bname, mode_name), None)
        if seed is None:
            print(f"SKIP: {bname} | {mode_name} (kein erfolgreicher Run in df)")
            continue

        print("\n" + "=" * 70)
        print(f"ANIMATION: {bench_label} | {mode_name} | seed={seed}")

        # WICHTIG: kein robot_radius mehr übergeben
        ani = animate_point_robot_eval_exact(
            bench=bench_obj,
            mode_name=mode_name,
            seed=seed,
            configs=configs,
            interp_step=0.5,
            fig_size=(7, 7),
            obstacle_alpha=0.25,
            robot_visual="robot",      # oder "clearance"
            draw_robot_fill=False,
            robot_alpha=0.25
        )


# 2DoF Planarroboter

Im Gegensatz zum zuvor betrachteten 2-DoF-Punktroboter bewegt sich der 2-DoF-Planarroboter nicht in einem kartesischen Arbeitsraum, sondern in einem zweidimensionalen Konfigurationsraum aus Gelenkwinkeln. Jeder Konfigurationspunkt beschreibt dabei eine vollständige Roboterpose, während geometrische Eigenschaften wie Nähe zu Hindernissen erst indirekt über Vorwärtskinematik und Kollisionsprüfung bewertet werden können. Die Struktur des Planungsproblems unterscheidet sich damit grundlegend von der Punktrobotervariante. Durch die explizite Modellierung der Robotergeometrie werden Kollisionstests deutlich aufwendiger. Während beim Punktroboter einfache Punkt-Hindernis-Tests ausreichen, müssen beim Planarroboter mehrere Robotersegmente gegen alle Hindernisse geprüft werden. Dieser Mehraufwand wirkt sich insbesondere bei der Kantenprüfung aus, da jede Verbindung im Konfigurationsraum über Interpolationen mehrerer Zwischenkonfigurationen validiert werden muss. Die Auswirkungen der unterschiedlichen Sampling-Modi sind beim Planarroboter daher weniger unmittelbar geometrisch sichtbar. Während sich beim Punktroboter Unterschiede in der Roadmapstruktur und Pfadführung direkt im Arbeitsraum erkennen lassen, zeigen sich beim Planarroboter die Effekte vor allem in Rechenzeit, Anzahl der Kollisionschecks und der erforderlichen Roadmapdichte. Um valide und zusammenhängende Pfade zu erzeugen, sind in der Regel dichtere Roadmaps erforderlich als im kartesischen Fall. In diesem Kontext gewinnt das Lazy-PRM-Prinzip besondere Bedeutung. Da teure Kollisions- und Kantenprüfungen zunächst vermieden und erst bei Bedarf durchgeführt werden, kann der erhöhte rechnerische Aufwand des Planarroboters effektiv abgefedert werden. Die folgenden Experimente untersuchen daher, wie sich die unterschiedlichen Sampling-Modi unter diesen veränderten Randbedingungen verhalten und inwieweit sich die beim Punktroboter beobachteten Tendenzen auf den 2-DoF-Planarroboter übertragen lassen.

In [ ]:
from IPython.display import HTML, display
import matplotlib as mpl
mpl.rcParams["animation.html"] = "jshtml"
mpl.rcParams["animation.embed_limit"] = 50  # MB, z.B. 50 oder 100

import matplotlib.pyplot as plt#
import networkx as nx
import numpy as np
import math
import random
import inspect

from matplotlib.animation import FuncAnimation
from shapely.geometry import Point as ShPoint, Polygon, LineString
from shapely.geometry import LineString as ShLineString
from shapely.affinity import translate
from dataclasses import dataclass

# ============================================================
# Benchmarks + CollisionChecker (mit Clearance für Planar-2R)
# ============================================================

def _make_benchmark(name, collisionChecker, start, goal, description=None, level=1):
    if description is None:
        description = name
    startList = [start]
    goalList  = [goal]

    sig = inspect.signature(Benchmark.__init__)
    params = list(sig.parameters.keys())[1:]  # ohne self

    mapping = {
        "name": name,
        "collisionChecker": collisionChecker,
        "startList": startList,
        "goalList": goalList,
        "description": description,
        "level": level,
    }

    args = []
    for p in params:
        if p in mapping:
            args.append(mapping[p])
        else:
            if sig.parameters[p].default is inspect._empty:
                raise TypeError(f"Benchmark.__init__ hat unbekannten Pflichtparameter: {p}")
            args.append(sig.parameters[p].default)

    b = Benchmark(*args)

    # Komfortfelder
    if hasattr(b, "name"):
        b.name = name
    else:
        setattr(b, "name", name)
    setattr(b, "start", start)
    setattr(b, "goal", goal)

    return b


# ------------------------------------------------------------
# 1) Gemeinsames CollisionChecker-Interface (Framework-kompatibel)
# ------------------------------------------------------------
class CCBase:
    """
    Adapter-Basis:
      - getDim()
      - getEnvironmentLimits()
      - pointInCollision(q)
      - lineInCollision(q1, q2)  (Alias: edgeInCollision)
    """
    def __init__(self, bounds, obstacles):
        # bounds: (xmin, ymin, xmax, ymax) fürs Workspace-Sampling (Point-Robot)
        self.bounds = bounds
        self.obstacles = list(obstacles) if obstacles is not None else []
        self.counter = 0

    def getDim(self):
        raise NotImplementedError

    def getEnvironmentLimits(self):
        xmin, ymin, xmax, ymax = self.bounds
        return [[float(xmin), float(xmax)], [float(ymin), float(ymax)]]

    def lineInCollision(self, q1, q2):
        return self.edgeInCollision(q1, q2)

    def edgeInCollision(self, q1, q2):
        # Default: diskretisiere Segment -> point checks
        q1 = np.asarray(q1, float)
        q2 = np.asarray(q2, float)
        steps = max(2, int(np.linalg.norm(q2 - q1) / 0.05))
        for t in np.linspace(0.0, 1.0, steps):
            q = (1 - t) * q1 + t * q2
            if self.pointInCollision(q.tolist()):
                return True
        return False


# ------------------------------------------------------------
# 2) 2DoF Point-Robot (q=[x,y]) in 2D Workspace
# ------------------------------------------------------------
class PointRobot2DoFCollisionChecker(CCBase):
    def __init__(self, bounds, obstacles, min_clearance=0.0):
        super().__init__(bounds=bounds, obstacles=obstacles)
        self.min_clearance = float(min_clearance)

        # Für Abstand: Hindernisse "aufblasen" (Minkowski Summe)
        if self.min_clearance > 0.0:
            self._inflated_obstacles = [obs.buffer(self.min_clearance) for obs in self.obstacles]
        else:
            self._inflated_obstacles = list(self.obstacles)

    def getDim(self):
        return 2

    def pointInCollision(self, q):
        self.counter += 1
        p = ShPoint(float(q[0]), float(q[1]))

        xmin, ymin, xmax, ymax = self.bounds
        if p.x < xmin or p.x > xmax or p.y < ymin or p.y > ymax:
            return True

        for obs in self._inflated_obstacles:
            if obs.contains(p) or obs.touches(p):
                return True
        return False

    def edgeInCollision(self, q1, q2):
        self.counter += 1
        seg = LineString([(float(q1[0]), float(q1[1])), (float(q2[0]), float(q2[1]))])

        xmin, ymin, xmax, ymax = self.bounds
        if (seg.bounds[0] < xmin) or (seg.bounds[2] > xmax) or (seg.bounds[1] < ymin) or (seg.bounds[3] > ymax):
            return True

        for obs in self._inflated_obstacles:
            if seg.intersects(obs):
                return True
        return False


# ------------------------------------------------------------
# 3) 2DoF Planar Manipulator (q=[theta1,theta2]) – Kollision im Workspace
# ------------------------------------------------------------
class PlanarRobot2DoFCollisionChecker(CCBase):
    """
    Konfiguration q=[theta1,theta2] (Radiant)
    Abstand/Roboterdicke wird robust als "Capsule" der Links modelliert:
      link_capsule = LineString(...).buffer(min_clearance)
    Damit gilt: Link muss mindestens min_clearance Abstand zum Hindernis halten.
    """
    def __init__(self, theta_limits, obstacles, base=(0.0, 0.0), L1=1.6, L2=1.2, min_clearance=0.0):
        super().__init__(bounds=(-math.pi, -math.pi, math.pi, math.pi), obstacles=obstacles)
        self.theta_limits = theta_limits
        self.base = base
        self.L1 = float(L1)
        self.L2 = float(L2)
        self.min_clearance = float(min_clearance)

    def getDim(self):
        return 2

    def getEnvironmentLimits(self):
        (a1, b1), (a2, b2) = self.theta_limits
        return [[float(a1), float(b1)], [float(a2), float(b2)]]

    def _fk_points(self, q):
        t1, t2 = float(q[0]), float(q[1])
        x0, y0 = self.base
        x1 = x0 + self.L1 * math.cos(t1)
        y1 = y0 + self.L1 * math.sin(t1)
        x2 = x1 + self.L2 * math.cos(t1 + t2)
        y2 = y1 + self.L2 * math.sin(t1 + t2)
        return (x0, y0), (x1, y1), (x2, y2)

    def pointInCollision(self, q):
        self.counter += 1
        p0, p1, p2 = self._fk_points(q)

        link1 = LineString([p0, p1])
        link2 = LineString([p1, p2])

        # Clearance: Links als Kapseln (robust)
        if self.min_clearance > 0.0:
            link1 = link1.buffer(self.min_clearance, cap_style=1, join_style=1)
            link2 = link2.buffer(self.min_clearance, cap_style=1, join_style=1)

        for obs in self.obstacles:
            if link1.intersects(obs) or link2.intersects(obs):
                return True
        return False

    def edgeInCollision(self, q1, q2):
        q1 = np.asarray(q1, float)
        q2 = np.asarray(q2, float)
        steps = max(8, int(np.linalg.norm(q2 - q1) / 0.05))
        for t in np.linspace(0.0, 1.0, steps):
            q = (1 - t) * q1 + t * q2
            if self.pointInCollision(q.tolist()):
                return True
        return False


# ------------------------------------------------------------
# 4) Benchmarks
# ------------------------------------------------------------
def _rounded_box(xmin, ymin, xmax, ymax, r=0.15):
    rect = Polygon([(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)])
    return rect.buffer(r).buffer(-r)

def build_2b_benchmarks_from_task1style(
    robot_radius=0.0,
    safety_margin=0.0
):
    """
    robot_radius + safety_margin => CLEARANCE (in Workspace-Einheiten)
    Clearance gilt:
      - Point-Robot: Hindernisse werden inflated (Minkowski) geprüft
      - Planar-2R: Links werden als Capsules buffer(min_clearance) geprüft
    """
    CLEARANCE = float(robot_radius) + float(safety_margin)

    # A) Workspace obstacles
    bounds = (-3.0, -3.0, 3.0, 3.0)

    obs_easy = [
        LineString([(-2.0, 0.0), (-0.8, 0.0)]).buffer(0.5),
        LineString([( 2.0, 0.0), ( 2.0, 1.0)]).buffer(0.2),
        LineString([(-1.0, 2.0), ( 1.0, 2.0)]).buffer(0.1),
    ]

    obs_passage = [
        LineString([(1.8, -3.0), (1.8, -0.8)]).buffer(0.2),
        LineString([(1.8,  0.8), (1.8,  3.0)]).buffer(0.2),
    ]

    # B) Point 2DoF (q=[x,y]) – Clearance aktiv
    ccP_easy    = PointRobot2DoFCollisionChecker(bounds=bounds, obstacles=obs_easy,    min_clearance=CLEARANCE)
    ccP_passage = PointRobot2DoFCollisionChecker(bounds=bounds, obstacles=obs_passage, min_clearance=CLEARANCE)

    bP_easy = _make_benchmark(
        "Point 2-DoF (Easy)", ccP_easy,
        start=[-2.6,  2.2], goal=[ 2.6, -2.2],
        description=f"Point 2DoF easy (clearance={CLEARANCE:.2f})", level=1
    )
    bP_passage = _make_benchmark(
        "Point 2-DoF (Passage)", ccP_passage,
        start=[-2.6,  0.0], goal=[ 2.6,  0.0],
        description=f"Point 2DoF passage (clearance={CLEARANCE:.2f})", level=2
    )

    # C) Planar 2DoF (2R) – Clearance aktiv (WICHTIG: für BEIDE Benchmarks setzen)
    theta_limits = [(-math.pi, math.pi), (-math.pi, math.pi)]

    ccA_easy = PlanarRobot2DoFCollisionChecker(
        theta_limits=theta_limits,
        obstacles=obs_easy,
        base=(0.0, 0.0),
        L1=1.6,
        L2=1.2,
        min_clearance=CLEARANCE
    )
    ccA_passage = PlanarRobot2DoFCollisionChecker(
        theta_limits=theta_limits,
        obstacles=obs_passage,
        base=(0.0, 0.0),
        L1=1.6,
        L2=1.2,
        min_clearance=CLEARANCE
    )

    bA_easy = _make_benchmark(
        "Manipulator 2-DoF (Easy)", ccA_easy,
        start=[2.0, 0.5], goal=[-2.0, -0.5],
        description=f"Planar 2DoF easy (clearance={CLEARANCE:.2f})", level=1
    )
    bA_passage = _make_benchmark(
        "Manipulator 2-DoF (Passage)", ccA_passage,
        start=[math.pi, 0.0], goal=[0.0, 0.0],
        description=f"Planar 2DoF passage (clearance={CLEARANCE:.2f})", level=2
    )

    return [bP_easy, bP_passage, bA_easy, bA_passage]


def sample_free_angles(cc, max_tries=20000):
    lims = cc.getEnvironmentLimits()
    for _ in range(max_tries):
        q = [random.uniform(*lims[0]), random.uniform(*lims[1])]
        if not cc.pointInCollision(q):
            return q
    raise RuntimeError("No collision-free angles found (max_tries reached).")

def ensure_valid_start_goal(b, cc, max_tries=20000):
    """
    Stellt sicher, dass Start und Goal kollisionsfrei sind.
    Wenn nicht, wird jeweils ein neuer Start/Goal im Winkelraum gesampelt.
    Aktualisiert b.startList/b.goalList sowie b.start/b.goal.
    """
    start_list = getattr(b, "startList", [b.start])
    goal_list  = getattr(b, "goalList",  [b.goal])

    # START fixen
    if cc.pointInCollision(start_list[0]):
        print("Start ist in Kollision -> sample neuen kollisionsfreien Start ...")
        qS = sample_free_angles(cc, max_tries=max_tries)
        b.startList = [qS]
        b.start = qS
        start_list = [qS]
        print("Neuer Start:", qS, "collision?", cc.pointInCollision(qS))

    # GOAL fixen
    if cc.pointInCollision(goal_list[0]):
        print("Goal ist in Kollision -> sample neues kollisionsfreies Goal ...")
        qG = sample_free_angles(cc, max_tries=max_tries)
        b.goalList = [qG]
        b.goal = qG
        goal_list = [qG]
        print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

    return start_list, goal_list


def run_one_2b(planner_class, benchmark, config, seed=None):
    import time
    if seed is not None:
        random.seed(int(seed))
        np.random.seed(int(seed))

    if hasattr(benchmark.collisionChecker, "counter"):
        benchmark.collisionChecker.counter = 0

    planner = planner_class(benchmark.collisionChecker)

    t0 = time.time()
    err = None
    path = []
    try:
        start_list = getattr(benchmark, "startList", [benchmark.start])
        goal_list  = getattr(benchmark, "goalList",  [benchmark.goal])
        path = planner.planPath(start_list, goal_list, config)
        ok = bool(path) and len(path) >= 2
    except Exception as e:
        ok = False
        err = str(e)

    dt = time.time() - t0
    return {
        "success": ok,
        "time_s": float(dt),
        "collision_checks": int(getattr(benchmark.collisionChecker, "counter", -1)),
        "path": path,
        "err": err,
        "planner": planner,
    }

def path_positions_from_graph(graph, path_nodes):
    if not path_nodes:
        return []
    out = []
    for n in path_nodes:
        if n not in graph.nodes:
            continue
        pos = graph.nodes[n].get("pos", None)
        if pos is None:
            continue
        out.append(list(pos))
    return out


## Sampling-Mode: Baseline

Die Animationen zeigen exemplarische Pfade des 2-DoF-Planarroboters für zwei Benchmarks unter uniformem Sampling. Während die Bewegungen im Arbeitsraum teilweise einfach und intuitiv wirken, verlaufen die zugehörigen Pfade im zweidimensionalen Konfigurationsraum deutlich komplexer und weisen mehrere Richtungswechsel auf. Dies verdeutlicht, dass die eigentliche Planungsschwierigkeit beim Planarroboter primär im Konfigurationsraum liegt.

Im Planar Easy-Benchmark genügt bereits eine relativ grobe Roadmap, um eine gültige Verbindung zu erzeugen. Im Planar Passage-Benchmark hingegen entsteht trotz nahezu geradliniger Bewegung im Arbeitsraum ein stark eingeschränkter, strukturierter Pfad im Konfigurationsraum. Die Animationen zeigen damit, dass auch einfache geometrische Situationen im Arbeitsraum im Konfigurationsraum zu anspruchsvollen Planungsproblemen führen können.

Insgesamt verdeutlicht die Baseline-Animation, dass Unterschiede zwischen Sampling-Modi beim 2-DoF-Planarroboter weniger unmittelbar im Arbeitsraum sichtbar sind, sondern sich vor allem in der Struktur und Dichte der Konfigurationsraum-Pfade widerspiegeln.

### Benchmark: Planar Easy

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Easy
# ============================================================

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,
    start_q=None,
    goal_q=None
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Easy")
    ax_q.set_title("Configuration Space")

    # Obstacles im Workspace
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Pfad im C-space
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space (bewegter Punkt)
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    # Start/Goal im C-space anzeigen
    if start_q is not None:
        ax_q.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (C)")
    if goal_q is not None:
        ax_q.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (C)")

    if robot_kind == "point":
        # Point robot: Workspace == C-space
        if start_q is not None:
            ax_ws.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (W)")
        if goal_q is not None:
            ax_ws.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (W)")

        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # Arm im Workspace
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal im WORKSPACE anzeigen (Endeffektor + gestrichelte Pose)
        base_xy = getattr(cc, "base", (0.0, 0.0))

        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=base_xy)
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")


# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "baseline_uniform",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
}

# Clearance zentral setzen
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[2]
cc = b.collisionChecker

# Start & Ziel FEST als C-space Winkel (Radiant!)
start_q = [-2.0, -1.0]
goal_q  = [2.0, 1.0]

# Setzen + prüfen (Parameter bleiben fix)
set_benchmark_start_goal(b, start=start_q, goal=goal_q, check=True)

# Fix übernehmen (nichts mehr überschreiben)
start_list = b.startList
goal_list  = b.goalList

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())
print("EnvLimits:", cc.getEnvironmentLimits())
print("Start:", start_list[0], "collision?", cc.pointInCollision(start_list[0]))
print("Goal :", goal_list[0],  "collision?", cc.pointInCollision(goal_list[0]))

# Start/Goal override (hilft Mode4, schadet nicht)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

path_q = path_positions_from_graph(planner.graph, path_nodes)

ani = animate_workspace_and_cspace(
    b, planner, path_q,
    q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
    ws_limits=(-3, 3, -3, 3),
    robot_kind="arm",
    interval_ms=30,
    densify_step=0.03,
    arm_L1=getattr(cc, "L1", 1.6),
    arm_L2=getattr(cc, "L2", 1.2),
    show_clearance=False,
    start_q=start_q,
    goal_q=goal_q
)

display(HTML(ani.to_jshtml()))


### Benchmark: Planar Passage

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Passage
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,     # zeigt inflated obstacles
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten (für saubere Animation)
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Pasage")
    ax_q.set_title("Configuration Space")

    # Workspace obstacles (und optional Sicherheitszone)
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Path im C-space (Graph lebt in C-space!)
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    if robot_kind == "point":
        # Point robot im Workspace (C-space == Workspace)
        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)  # Pfad im Workspace

        # Start/Goal im Workspace (direkt x,y)
        start_w = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_w  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]
        if start_w is not None:
            ax_ws.scatter([start_w[0]], [start_w[1]], s=90, marker="o", label="Start")
        if goal_w is not None:
            ax_ws.scatter([goal_w[0]],  [goal_w[1]],  s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # ---------------------------
        # Arm im Workspace + Start/Goal anzeigen
        # ---------------------------

        # Roboter als 2 Liniensegmente
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal aus Benchmark (C-space Winkel)
        start_q = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_q  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]

        # Start/Goal im Workspace darstellen:
        # - sinnvoll: Endeffektorpunkt + optional gestrichelte Pose
        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")

# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "baseline_uniform",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
}

# Clearance zentral setzen (wie du es wolltest)
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[3]

# Start & Ziel festlegen (NICHT verändern)
set_benchmark_start_goal(b, start=[math.pi, 0.0], goal=[0.0, 0.0], check=True)

cc = b.collisionChecker

print("Start collision?", cc.pointInCollision(getattr(b, "startList", [b.start])[0]))
print("Goal  collision?", cc.pointInCollision(getattr(b, "goalList",  [b.goal])[0]))

start_list, goal_list = ensure_valid_start_goal(b, cc, max_tries=20000)

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())

# Falls Goal kollidiert: neues Goal sampeln (für Planar sinnvoll)
if cc.pointInCollision(goal_list[0]):
    print("\nGoal ist in Kollision -> suche kollisionsfreies Goal ...")
    qG = sample_free_angles(cc, max_tries=20000)
    b.goalList = [qG]
    b.goal = qG
    goal_list = [qG]
    print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

# Sinnvoll: Start/Goal override (hilft Mode4)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

G = planner.graph
start_key = "start" if "start" in G.nodes else None
goal_key  = "goal"  if "goal"  in G.nodes else None

print("\n--- GRAPH DIAG ---")
print("Graph nodes:", G.number_of_nodes())
print("Graph edges:", G.number_of_edges())
print("Has 'start':", start_key is not None, "| Has 'goal':", goal_key is not None)
if start_key is not None:
    print("deg(start):", G.degree[start_key])
if goal_key is not None:
    print("deg(goal) :", G.degree[goal_key])
if start_key is not None and goal_key is not None:
    print("nx.has_path(start,goal):", nx.has_path(G, start_key, goal_key))
    comps = list(nx.connected_components(G))
    print("Connected components:", len(comps))
    s_comp = next((i for i, c in enumerate(comps) if start_key in c), None)
    g_comp = next((i for i, c in enumerate(comps) if goal_key in c), None)
    print("start component:", s_comp, "| goal component:", g_comp)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

if not ok:
    print("Hinweis: Wenn success=False, erhöhe testweise maxIterations oder updateRoadmapSize,")
    print("oder prüfe, ob Start/Goal wirklich in derselben freien Komponente liegen.")

path_q = path_positions_from_graph(planner.graph, path_nodes)

if isinstance(cc, PlanarRobot2DoFCollisionChecker):
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="arm",
        interval_ms=30,
        densify_step=0.03,
        arm_L1=getattr(cc, "L1", 1.6),
        arm_L2=getattr(cc, "L2", 1.2),
        show_clearance=False,
    )
else:
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-3, 3), (-3, 3)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="point",
        interval_ms=30,
        densify_step=0.03,
        show_clearance=True,
    )

display(HTML(ani.to_jshtml()))


## Sampling-Mode: Mode1_Seed_Gaussian

Die Animationen zeigen beim 2-DoF-Planarroboter keine deutlich von der Baseline abweichende Roadmapstruktur im Konfigurationsraum. Sowohl im Planar Easy- als auch im Planar Passage-Benchmark ähnelt die Verteilung der Knoten stark dem uniformen Sampling. Der resultierende Pfad unterscheidet sich nur geringfügig von der Baseline und folgt denselben strukturell vorgegebenen Konfigurationsraumkorridoren.

### Benchmark: Planar Easy

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Easy
# ============================================================

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,
    start_q=None,
    goal_q=None
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Easy")
    ax_q.set_title("Configuration Space")

    # Obstacles im Workspace
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Pfad im C-space
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space (bewegter Punkt)
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    # Start/Goal im C-space anzeigen
    if start_q is not None:
        ax_q.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (C)")
    if goal_q is not None:
        ax_q.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (C)")

    if robot_kind == "point":
        # Point robot: Workspace == C-space
        if start_q is not None:
            ax_ws.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (W)")
        if goal_q is not None:
            ax_ws.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (W)")

        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # Arm im Workspace
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal im WORKSPACE anzeigen (Endeffektor + gestrichelte Pose)
        base_xy = getattr(cc, "base", (0.0, 0.0))

        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=base_xy)
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")


# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode1_seed_gauss",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "seedSigma": 4.0,
    "seedTries": 5,
}


# Clearance zentral setzen
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[2]
cc = b.collisionChecker

# Start & Ziel FEST als C-space Winkel (Radiant!)
start_q = [-2.0, -1.0]
goal_q  = [2.0, 1.0]

# Setzen + prüfen (Parameter bleiben fix)
set_benchmark_start_goal(b, start=start_q, goal=goal_q, check=True)

# Fix übernehmen (nichts mehr überschreiben)
start_list = b.startList
goal_list  = b.goalList

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())
print("EnvLimits:", cc.getEnvironmentLimits())
print("Start:", start_list[0], "collision?", cc.pointInCollision(start_list[0]))
print("Goal :", goal_list[0],  "collision?", cc.pointInCollision(goal_list[0]))

# Start/Goal override (hilft Mode4, schadet nicht)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

path_q = path_positions_from_graph(planner.graph, path_nodes)

ani = animate_workspace_and_cspace(
    b, planner, path_q,
    q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
    ws_limits=(-3, 3, -3, 3),
    robot_kind="arm",
    interval_ms=30,
    densify_step=0.03,
    arm_L1=getattr(cc, "L1", 1.6),
    arm_L2=getattr(cc, "L2", 1.2),
    show_clearance=False,
    start_q=start_q,
    goal_q=goal_q
)

display(HTML(ani.to_jshtml()))


### Benchmark: Planar Passage

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Passage
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,     # zeigt inflated obstacles
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten (für saubere Animation)
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Pasage")
    ax_q.set_title("Configuration Space")

    # Workspace obstacles (und optional Sicherheitszone)
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Path im C-space (Graph lebt in C-space!)
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    if robot_kind == "point":
        # Point robot im Workspace (C-space == Workspace)
        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)  # Pfad im Workspace

        # Start/Goal im Workspace (direkt x,y)
        start_w = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_w  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]
        if start_w is not None:
            ax_ws.scatter([start_w[0]], [start_w[1]], s=90, marker="o", label="Start")
        if goal_w is not None:
            ax_ws.scatter([goal_w[0]],  [goal_w[1]],  s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # ---------------------------
        # Arm im Workspace + Start/Goal anzeigen
        # ---------------------------

        # Roboter als 2 Liniensegmente
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal aus Benchmark (C-space Winkel)
        start_q = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_q  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]

        # Start/Goal im Workspace darstellen:
        # - sinnvoll: Endeffektorpunkt + optional gestrichelte Pose
        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")

# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode1_seed_gauss",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "seedSigma": 4.0,
    "seedTries": 5,
}


# Clearance zentral setzen (wie du es wolltest)
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[3]

# Start & Ziel festlegen (NICHT verändern)
set_benchmark_start_goal(b, start=[math.pi, 0.0], goal=[0.0, 0.0], check=True)

cc = b.collisionChecker

print("Start collision?", cc.pointInCollision(getattr(b, "startList", [b.start])[0]))
print("Goal  collision?", cc.pointInCollision(getattr(b, "goalList",  [b.goal])[0]))

start_list, goal_list = ensure_valid_start_goal(b, cc, max_tries=20000)

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())

# Falls Goal kollidiert: neues Goal sampeln (für Planar sinnvoll)
if cc.pointInCollision(goal_list[0]):
    print("\nGoal ist in Kollision -> suche kollisionsfreies Goal ...")
    qG = sample_free_angles(cc, max_tries=20000)
    b.goalList = [qG]
    b.goal = qG
    goal_list = [qG]
    print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

# Sinnvoll: Start/Goal override (hilft Mode4)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

G = planner.graph
start_key = "start" if "start" in G.nodes else None
goal_key  = "goal"  if "goal"  in G.nodes else None

print("\n--- GRAPH DIAG ---")
print("Graph nodes:", G.number_of_nodes())
print("Graph edges:", G.number_of_edges())
print("Has 'start':", start_key is not None, "| Has 'goal':", goal_key is not None)
if start_key is not None:
    print("deg(start):", G.degree[start_key])
if goal_key is not None:
    print("deg(goal) :", G.degree[goal_key])
if start_key is not None and goal_key is not None:
    print("nx.has_path(start,goal):", nx.has_path(G, start_key, goal_key))
    comps = list(nx.connected_components(G))
    print("Connected components:", len(comps))
    s_comp = next((i for i, c in enumerate(comps) if start_key in c), None)
    g_comp = next((i for i, c in enumerate(comps) if goal_key in c), None)
    print("start component:", s_comp, "| goal component:", g_comp)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

if not ok:
    print("Hinweis: Wenn success=False, erhöhe testweise maxIterations oder updateRoadmapSize,")
    print("oder prüfe, ob Start/Goal wirklich in derselben freien Komponente liegen.")

path_q = path_positions_from_graph(planner.graph, path_nodes)

if isinstance(cc, PlanarRobot2DoFCollisionChecker):
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="arm",
        interval_ms=30,
        densify_step=0.03,
        arm_L1=getattr(cc, "L1", 1.6),
        arm_L2=getattr(cc, "L2", 1.2),
        show_clearance=False,
    )
else:
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-3, 3), (-3, 3)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="point",
        interval_ms=30,
        densify_step=0.03,
        show_clearance=True,
    )

display(HTML(ani.to_jshtml()))


## Sampling-Mode: Mode2_Seed_Distance

Die Animationen des 2-DoF-Planarroboters mit Mode 2 zeigen im Konfigurationsraum eine Roadmapstruktur, die visuell stark der Baseline und Mode 1 ähnelt. Sowohl im Planar Easy- als auch im Planar Passage-Benchmark sind keine klar abgegrenzten lokalen Cluster oder eine gezielte Verdichtung entlang bestimmter Konfigurationsraumregionen erkennbar. Die Knoten verteilen sich weiterhin breit im Konfigurationsraum, und der resultierende Pfad nutzt die durch die Geometrie vorgegebenen freien Korridore.

Insgesamt zeigen die Animationen, dass auch Mode 2 beim 2-DoF-Planarroboter keinen klar erkennbaren geometrischen Vorteil gegenüber der Baseline bietet. Unterschiede zwischen den Sampling-Modi sind im Arbeits- und Konfigurationsraum visuell kaum unterscheidbar und werden maßgeblich von der Konfigurationsraumgeometrie dominiert.

### Benchmark: Planar Easy

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Easy
# ============================================================

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,
    start_q=None,
    goal_q=None
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Easy")
    ax_q.set_title("Configuration Space")

    # Obstacles im Workspace
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Pfad im C-space
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space (bewegter Punkt)
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    # Start/Goal im C-space anzeigen
    if start_q is not None:
        ax_q.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (C)")
    if goal_q is not None:
        ax_q.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (C)")

    if robot_kind == "point":
        # Point robot: Workspace == C-space
        if start_q is not None:
            ax_ws.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (W)")
        if goal_q is not None:
            ax_ws.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (W)")

        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # Arm im Workspace
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal im WORKSPACE anzeigen (Endeffektor + gestrichelte Pose)
        base_xy = getattr(cc, "base", (0.0, 0.0))

        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=base_xy)
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")


# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode2_seed_dist",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "seedMaxStep": 5.0,
    "seedBeta": 0.9,
    "seedTries": 5,
}


# Clearance zentral setzen
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[2]
cc = b.collisionChecker

# Start & Ziel FEST als C-space Winkel (Radiant!)
start_q = [-2.0, -1.0]
goal_q  = [2.0, 1.0]

# Setzen + prüfen (Parameter bleiben fix)
set_benchmark_start_goal(b, start=start_q, goal=goal_q, check=True)

# Fix übernehmen (nichts mehr überschreiben)
start_list = b.startList
goal_list  = b.goalList

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())
print("EnvLimits:", cc.getEnvironmentLimits())
print("Start:", start_list[0], "collision?", cc.pointInCollision(start_list[0]))
print("Goal :", goal_list[0],  "collision?", cc.pointInCollision(goal_list[0]))

# Start/Goal override (hilft Mode4, schadet nicht)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

path_q = path_positions_from_graph(planner.graph, path_nodes)

ani = animate_workspace_and_cspace(
    b, planner, path_q,
    q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
    ws_limits=(-3, 3, -3, 3),
    robot_kind="arm",
    interval_ms=30,
    densify_step=0.03,
    arm_L1=getattr(cc, "L1", 1.6),
    arm_L2=getattr(cc, "L2", 1.2),
    show_clearance=False,
    start_q=start_q,
    goal_q=goal_q
)

display(HTML(ani.to_jshtml()))


### Benchmark: Planar Passage

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Passage
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,     # zeigt inflated obstacles
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten (für saubere Animation)
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Pasage")
    ax_q.set_title("Configuration Space")

    # Workspace obstacles (und optional Sicherheitszone)
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Path im C-space (Graph lebt in C-space!)
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    if robot_kind == "point":
        # Point robot im Workspace (C-space == Workspace)
        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)  # Pfad im Workspace

        # Start/Goal im Workspace (direkt x,y)
        start_w = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_w  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]
        if start_w is not None:
            ax_ws.scatter([start_w[0]], [start_w[1]], s=90, marker="o", label="Start")
        if goal_w is not None:
            ax_ws.scatter([goal_w[0]],  [goal_w[1]],  s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # ---------------------------
        # Arm im Workspace + Start/Goal anzeigen
        # ---------------------------

        # Roboter als 2 Liniensegmente
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal aus Benchmark (C-space Winkel)
        start_q = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_q  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]

        # Start/Goal im Workspace darstellen:
        # - sinnvoll: Endeffektorpunkt + optional gestrichelte Pose
        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")

# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode2_seed_dist",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "seedMaxStep": 5.0,
    "seedBeta": 0.9,
    "seedTries": 5,
}


# Clearance zentral setzen (wie du es wolltest)
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[3]

# Start & Ziel festlegen (NICHT verändern)
set_benchmark_start_goal(b, start=[math.pi, 0.0], goal=[0.0, 0.0], check=True)

cc = b.collisionChecker

print("Start collision?", cc.pointInCollision(getattr(b, "startList", [b.start])[0]))
print("Goal  collision?", cc.pointInCollision(getattr(b, "goalList",  [b.goal])[0]))

start_list, goal_list = ensure_valid_start_goal(b, cc, max_tries=20000)

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())

# Falls Goal kollidiert: neues Goal sampeln (für Planar sinnvoll)
if cc.pointInCollision(goal_list[0]):
    print("\nGoal ist in Kollision -> suche kollisionsfreies Goal ...")
    qG = sample_free_angles(cc, max_tries=20000)
    b.goalList = [qG]
    b.goal = qG
    goal_list = [qG]
    print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

# Sinnvoll: Start/Goal override (hilft Mode4)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

G = planner.graph
start_key = "start" if "start" in G.nodes else None
goal_key  = "goal"  if "goal"  in G.nodes else None

print("\n--- GRAPH DIAG ---")
print("Graph nodes:", G.number_of_nodes())
print("Graph edges:", G.number_of_edges())
print("Has 'start':", start_key is not None, "| Has 'goal':", goal_key is not None)
if start_key is not None:
    print("deg(start):", G.degree[start_key])
if goal_key is not None:
    print("deg(goal) :", G.degree[goal_key])
if start_key is not None and goal_key is not None:
    print("nx.has_path(start,goal):", nx.has_path(G, start_key, goal_key))
    comps = list(nx.connected_components(G))
    print("Connected components:", len(comps))
    s_comp = next((i for i, c in enumerate(comps) if start_key in c), None)
    g_comp = next((i for i, c in enumerate(comps) if goal_key in c), None)
    print("start component:", s_comp, "| goal component:", g_comp)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

if not ok:
    print("Hinweis: Wenn success=False, erhöhe testweise maxIterations oder updateRoadmapSize,")
    print("oder prüfe, ob Start/Goal wirklich in derselben freien Komponente liegen.")

path_q = path_positions_from_graph(planner.graph, path_nodes)

if isinstance(cc, PlanarRobot2DoFCollisionChecker):
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="arm",
        interval_ms=30,
        densify_step=0.03,
        arm_L1=getattr(cc, "L1", 1.6),
        arm_L2=getattr(cc, "L2", 1.2),
        show_clearance=False,
    )
else:
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-3, 3), (-3, 3)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="point",
        interval_ms=30,
        densify_step=0.03,
        show_clearance=True,
    )

display(HTML(ani.to_jshtml()))


## Sampling-Mode: Mode3_Max_Min

Die Animationen von Mode 3 unterscheiden sich von den übrigen Sampling-Modi vor allem durch die deutlich geringere Knotendichte im Konfigurationsraum. Insbesondere im Planar-Passage-Benchmark zeigt sich auch bei Mode 3 eine weitgehend gleichmäßige Abdeckung des Konfigurationsraums. Der Unterschied zu Baseline liegt weniger in der räumlichen Verteilung der Knoten als vielmehr in der Art ihrer Auswahl, wodurch Redundanzen reduziert werden, ohne die globale Abdeckung sichtbar zu verändern.

### Benchmark: Planar Easy

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Easy
# ============================================================

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,
    start_q=None,
    goal_q=None
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Easy")
    ax_q.set_title("Configuration Space")

    # Obstacles im Workspace
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Pfad im C-space
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space (bewegter Punkt)
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    # Start/Goal im C-space anzeigen
    if start_q is not None:
        ax_q.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (C)")
    if goal_q is not None:
        ax_q.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (C)")

    if robot_kind == "point":
        # Point robot: Workspace == C-space
        if start_q is not None:
            ax_ws.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (W)")
        if goal_q is not None:
            ax_ws.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (W)")

        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # Arm im Workspace
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal im WORKSPACE anzeigen (Endeffektor + gestrichelte Pose)
        base_xy = getattr(cc, "base", (0.0, 0.0))

        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=base_xy)
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")


# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode3_max_min",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "dispersionCandidates": 30,
}


# Clearance zentral setzen
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[2]
cc = b.collisionChecker

# Start & Ziel FEST als C-space Winkel (Radiant!)
start_q = [-2.0, -1.0]
goal_q  = [2.0, 1.0]

# Setzen + prüfen (Parameter bleiben fix)
set_benchmark_start_goal(b, start=start_q, goal=goal_q, check=True)

# Fix übernehmen (nichts mehr überschreiben)
start_list = b.startList
goal_list  = b.goalList

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())
print("EnvLimits:", cc.getEnvironmentLimits())
print("Start:", start_list[0], "collision?", cc.pointInCollision(start_list[0]))
print("Goal :", goal_list[0],  "collision?", cc.pointInCollision(goal_list[0]))

# Start/Goal override (hilft Mode4, schadet nicht)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

path_q = path_positions_from_graph(planner.graph, path_nodes)

ani = animate_workspace_and_cspace(
    b, planner, path_q,
    q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
    ws_limits=(-3, 3, -3, 3),
    robot_kind="arm",
    interval_ms=30,
    densify_step=0.03,
    arm_L1=getattr(cc, "L1", 1.6),
    arm_L2=getattr(cc, "L2", 1.2),
    show_clearance=False,
    start_q=start_q,
    goal_q=goal_q
)

display(HTML(ani.to_jshtml()))


### Benchmark: Planar Passage

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Passage
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,     # zeigt inflated obstacles
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten (für saubere Animation)
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Pasage")
    ax_q.set_title("Configuration Space")

    # Workspace obstacles (und optional Sicherheitszone)
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Path im C-space (Graph lebt in C-space!)
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    if robot_kind == "point":
        # Point robot im Workspace (C-space == Workspace)
        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)  # Pfad im Workspace

        # Start/Goal im Workspace (direkt x,y)
        start_w = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_w  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]
        if start_w is not None:
            ax_ws.scatter([start_w[0]], [start_w[1]], s=90, marker="o", label="Start")
        if goal_w is not None:
            ax_ws.scatter([goal_w[0]],  [goal_w[1]],  s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # ---------------------------
        # Arm im Workspace + Start/Goal anzeigen
        # ---------------------------

        # Roboter als 2 Liniensegmente
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal aus Benchmark (C-space Winkel)
        start_q = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_q  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]

        # Start/Goal im Workspace darstellen:
        # - sinnvoll: Endeffektorpunkt + optional gestrichelte Pose
        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")

# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode3_max_min",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "dispersionCandidates": 30,
}


# Clearance zentral setzen (wie du es wolltest)
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[3]

# Start & Ziel festlegen (NICHT verändern)
set_benchmark_start_goal(b, start=[math.pi, 0.0], goal=[0.0, 0.0], check=True)

cc = b.collisionChecker

print("Start collision?", cc.pointInCollision(getattr(b, "startList", [b.start])[0]))
print("Goal  collision?", cc.pointInCollision(getattr(b, "goalList",  [b.goal])[0]))

start_list, goal_list = ensure_valid_start_goal(b, cc, max_tries=20000)

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())

# Falls Goal kollidiert: neues Goal sampeln (für Planar sinnvoll)
if cc.pointInCollision(goal_list[0]):
    print("\nGoal ist in Kollision -> suche kollisionsfreies Goal ...")
    qG = sample_free_angles(cc, max_tries=20000)
    b.goalList = [qG]
    b.goal = qG
    goal_list = [qG]
    print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

# Sinnvoll: Start/Goal override (hilft Mode4)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

G = planner.graph
start_key = "start" if "start" in G.nodes else None
goal_key  = "goal"  if "goal"  in G.nodes else None

print("\n--- GRAPH DIAG ---")
print("Graph nodes:", G.number_of_nodes())
print("Graph edges:", G.number_of_edges())
print("Has 'start':", start_key is not None, "| Has 'goal':", goal_key is not None)
if start_key is not None:
    print("deg(start):", G.degree[start_key])
if goal_key is not None:
    print("deg(goal) :", G.degree[goal_key])
if start_key is not None and goal_key is not None:
    print("nx.has_path(start,goal):", nx.has_path(G, start_key, goal_key))
    comps = list(nx.connected_components(G))
    print("Connected components:", len(comps))
    s_comp = next((i for i, c in enumerate(comps) if start_key in c), None)
    g_comp = next((i for i, c in enumerate(comps) if goal_key in c), None)
    print("start component:", s_comp, "| goal component:", g_comp)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

if not ok:
    print("Hinweis: Wenn success=False, erhöhe testweise maxIterations oder updateRoadmapSize,")
    print("oder prüfe, ob Start/Goal wirklich in derselben freien Komponente liegen.")

path_q = path_positions_from_graph(planner.graph, path_nodes)

if isinstance(cc, PlanarRobot2DoFCollisionChecker):
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="arm",
        interval_ms=30,
        densify_step=0.03,
        arm_L1=getattr(cc, "L1", 1.6),
        arm_L2=getattr(cc, "L2", 1.2),
        show_clearance=False,
    )
else:
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-3, 3), (-3, 3)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="point",
        interval_ms=30,
        densify_step=0.03,
        show_clearance=True,
    )

display(HTML(ani.to_jshtml()))


## Sampling-Mode: Mode4_Start_Goal_Corridor

Die Animationen von Mode 4 zeigen beim 2-DoF-Planarroboter keine ausgeprägte strukturelle Abweichung gegenüber Baseline, Mode 1 oder Mode 2 im Konfigurationsraum. Insbesondere im Planar-Passage-Benchmark ist eine weitgehend gleichmäßige Verteilung der Knoten im C-Space zu erkennen. Eine klare Fokussierung entlang einer Start-Ziel-Achse, wie sie im kartesischen Punktroboter sichtbar war, überträgt sich hier nicht direkt auf den Winkelraum. 

### Benchmark: Planar Easy

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Easy
# ============================================================

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,
    start_q=None,
    goal_q=None
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Easy")
    ax_q.set_title("Configuration Space")

    # Obstacles im Workspace
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Pfad im C-space
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space (bewegter Punkt)
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    # Start/Goal im C-space anzeigen
    if start_q is not None:
        ax_q.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (C)")
    if goal_q is not None:
        ax_q.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (C)")

    if robot_kind == "point":
        # Point robot: Workspace == C-space
        if start_q is not None:
            ax_ws.scatter([start_q[0]], [start_q[1]], s=90, marker="o", label="Start (W)")
        if goal_q is not None:
            ax_ws.scatter([goal_q[0]], [goal_q[1]], s=90, marker="X", label="Goal (W)")

        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # Arm im Workspace
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal im WORKSPACE anzeigen (Endeffektor + gestrichelte Pose)
        base_xy = getattr(cc, "base", (0.0, 0.0))

        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=base_xy)
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")
        ax_q.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=base_xy)
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")


# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode4_start_goal_corr",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "corridorSigma": 1.8,
    "corridorAlongSigma": 0.45,
    "corridorTries": 6,
}


# Clearance zentral setzen
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[2]
cc = b.collisionChecker

# Start & Ziel FEST als C-space Winkel (Radiant!)
start_q = [-2.0, -1.0]
goal_q  = [2.0, 1.0]

# Setzen + prüfen (Parameter bleiben fix)
set_benchmark_start_goal(b, start=start_q, goal=goal_q, check=True)

# Fix übernehmen (nichts mehr überschreiben)
start_list = b.startList
goal_list  = b.goalList

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())
print("EnvLimits:", cc.getEnvironmentLimits())
print("Start:", start_list[0], "collision?", cc.pointInCollision(start_list[0]))
print("Goal :", goal_list[0],  "collision?", cc.pointInCollision(goal_list[0]))

# Start/Goal override (hilft Mode4, schadet nicht)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

path_q = path_positions_from_graph(planner.graph, path_nodes)

ani = animate_workspace_and_cspace(
    b, planner, path_q,
    q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
    ws_limits=(-3, 3, -3, 3),
    robot_kind="arm",
    interval_ms=30,
    densify_step=0.03,
    arm_L1=getattr(cc, "L1", 1.6),
    arm_L2=getattr(cc, "L2", 1.2),
    show_clearance=False,
    start_q=start_q,
    goal_q=goal_q
)

display(HTML(ani.to_jshtml()))


### Benchmark: Planar Passage

In [ ]:
# ============================================================
# Animation + Run: Benchmark: Planar Passage
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

def densify_path(path_xy, step=0.05):
    if path_xy is None or len(path_xy) < 2:
        return np.asarray(path_xy) if path_xy is not None else np.zeros((0, 2))
    pts = np.asarray(path_xy, dtype=float)
    dense = [pts[0]]
    for i in range(len(pts) - 1):
        a, b = pts[i], pts[i + 1]
        dist = float(np.linalg.norm(b - a))
        n = max(1, int(math.ceil(dist / step)))
        for k in range(1, n + 1):
            t = k / n
            dense.append((1 - t) * a + t * b)
    return np.asarray(dense)


def _graph_pos(G):
    return nx.get_node_attributes(G, "pos")


def _draw_polygon(ax, poly, alpha=0.25):
    if poly is None:
        return
    if getattr(poly, "is_empty", False):
        return
    if poly.geom_type == "Polygon":
        x, y = poly.exterior.xy
        ax.fill(x, y, alpha=alpha)
    elif poly.geom_type == "MultiPolygon":
        for p in poly.geoms:
            x, y = p.exterior.xy
            ax.fill(x, y, alpha=alpha)


def _draw_obstacles(ax, obstacles, alpha=0.25):
    if obstacles is None:
        return
    for obs in obstacles:
        _draw_polygon(ax, obs, alpha=alpha)


def _draw_obstacles_inflated(ax, obstacles, clearance, alpha=0.10):
    """
    Visualisiert die Sicherheitszone (Obstacle buffer(clearance)).
    Nur Visualisierung; die echte Kollision macht der CollisionChecker.
    """
    if obstacles is None:
        return
    clearance = float(clearance)
    if clearance <= 0:
        return
    for obs in obstacles:
        infl = obs.buffer(clearance)
        _draw_polygon(ax, infl, alpha=alpha)


def _plot_roadmap(ax, G, edge_alpha=0.12, node_size=18):
    pos = _graph_pos(G)
    for u, v in G.edges():
        if u in pos and v in pos:
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            ax.plot(x, y, linewidth=1, alpha=edge_alpha)
    xs = [pos[n][0] for n in pos]
    ys = [pos[n][1] for n in pos]
    ax.scatter(xs, ys, s=node_size)


def _plot_path(ax, path_xy, lw=3, alpha=1.0):
    if path_xy is None or len(path_xy) < 2:
        return
    p = np.asarray(path_xy, float)
    ax.plot(p[:, 0], p[:, 1], linewidth=lw, alpha=alpha)


def _fk_2r(q, L1, L2, base=(0.0, 0.0)):
    t1, t2 = float(q[0]), float(q[1])
    x0, y0 = base
    x1 = x0 + L1 * math.cos(t1)
    y1 = y0 + L1 * math.sin(t1)
    x2 = x1 + L2 * math.cos(t1 + t2)
    y2 = y1 + L2 * math.sin(t1 + t2)
    return (x0, y0), (x1, y1), (x2, y2)


def animate_workspace_and_cspace(
    benchmark,
    planner,
    path_q,
    q_limits,
    ws_limits,
    robot_kind="arm",        # "arm" oder "point"
    interval_ms=40,
    densify_step=0.03,
    arm_L1=1.6,
    arm_L2=1.2,
    show_clearance=False,     # zeigt inflated obstacles
):
    cc = benchmark.collisionChecker
    obstacles = getattr(cc, "obstacles", None)
    clearance = float(getattr(cc, "min_clearance", 0.0) or 0.0)

    # Pfad verdichten (für saubere Animation)
    path_q = densify_path(path_q, step=densify_step)

    fig, (ax_ws, ax_q) = plt.subplots(1, 2, figsize=(12, 5))
    ax_ws.set_aspect("equal", adjustable="box")

    xmin, xmax, ymin, ymax = ws_limits
    ax_ws.set_xlim(xmin, xmax)
    ax_ws.set_ylim(ymin, ymax)

    (q1min, q1max), (q2min, q2max) = q_limits
    ax_q.set_xlim(q1min, q1max)
    ax_q.set_ylim(q2min, q2max)
    ax_q.set_aspect("equal", adjustable="box")

    ax_ws.set_title("Workspace: Planar Pasage")
    ax_q.set_title("Configuration Space")

    # Workspace obstacles (und optional Sicherheitszone)
    _draw_obstacles(ax_ws, obstacles, alpha=0.25)
    if show_clearance and clearance > 0.0:
        _draw_obstacles_inflated(ax_ws, obstacles, clearance, alpha=0.10)

    # Roadmap & Path im C-space (Graph lebt in C-space!)
    _plot_roadmap(ax_q, planner.graph, edge_alpha=0.10, node_size=14)
    _plot_path(ax_q, path_q, lw=3, alpha=1.0)

    # Marker in C-space
    dot_q, = ax_q.plot([], [], marker="o", markersize=7, linestyle="None")

    if robot_kind == "point":
        # Point robot im Workspace (C-space == Workspace)
        dot_ws, = ax_ws.plot([], [], marker="o", markersize=8, linestyle="None")
        _plot_path(ax_ws, path_q, lw=3, alpha=0.6)  # Pfad im Workspace

        # Start/Goal im Workspace (direkt x,y)
        start_w = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_w  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]
        if start_w is not None:
            ax_ws.scatter([start_w[0]], [start_w[1]], s=90, marker="o", label="Start")
        if goal_w is not None:
            ax_ws.scatter([goal_w[0]],  [goal_w[1]],  s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            dot_ws.set_data([], [])
            return (dot_q, dot_ws)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])
            dot_ws.set_data([q[0]], [q[1]])
            return (dot_q, dot_ws)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani

    else:
        # ---------------------------
        # Arm im Workspace + Start/Goal anzeigen
        # ---------------------------

        # Roboter als 2 Liniensegmente
        link1_line, = ax_ws.plot([], [], linewidth=4)
        link2_line, = ax_ws.plot([], [], linewidth=4)
        ee_dot,     = ax_ws.plot([], [], marker="o", markersize=6, linestyle="None")

        # Start/Goal aus Benchmark (C-space Winkel)
        start_q = getattr(benchmark, "startList", [getattr(benchmark, "start", None)])[0]
        goal_q  = getattr(benchmark, "goalList",  [getattr(benchmark, "goal", None)])[0]

        # Start/Goal im Workspace darstellen:
        # - sinnvoll: Endeffektorpunkt + optional gestrichelte Pose
        if start_q is not None:
            p0s, p1s, p2s = _fk_2r(start_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0s[0], p1s[0]], [p0s[1], p1s[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1s[0], p2s[0]], [p1s[1], p2s[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2s[0]], [p2s[1]], s=90, marker="o", label="Start")

        if goal_q is not None:
            p0g, p1g, p2g = _fk_2r(goal_q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            #ax_ws.plot([p0g[0], p1g[0]], [p0g[1], p1g[1]], linestyle="--", linewidth=2)
            #ax_ws.plot([p1g[0], p2g[0]], [p1g[1], p2g[1]], linestyle="--", linewidth=2)
            ax_ws.scatter([p2g[0]], [p2g[1]], s=90, marker="X", label="Goal")

        ax_ws.legend(loc="upper right")

        def init():
            dot_q.set_data([], [])
            link1_line.set_data([], [])
            link2_line.set_data([], [])
            ee_dot.set_data([], [])
            return (dot_q, link1_line, link2_line, ee_dot)

        def update(i):
            q = path_q[i]
            dot_q.set_data([q[0]], [q[1]])

            p0, p1, p2 = _fk_2r(q, arm_L1, arm_L2, base=getattr(cc, "base", (0.0, 0.0)))
            link1_line.set_data([p0[0], p1[0]], [p0[1], p1[1]])
            link2_line.set_data([p1[0], p2[0]], [p1[1], p2[1]])
            ee_dot.set_data([p2[0]], [p2[1]])
            return (dot_q, link1_line, link2_line, ee_dot)

        ani = FuncAnimation(fig, update, frames=len(path_q), init_func=init,
                            interval=interval_ms, blit=True)
        plt.close(fig)
        return ani


def set_benchmark_start_goal(b, start, goal, check=True):
    b.startList = [list(start)]
    b.goalList  = [list(goal)]
    b.start = list(start)
    b.goal  = list(goal)

    if check:
        cc = b.collisionChecker
        if cc.pointInCollision(b.startList[0]):
            raise ValueError(f"{b.name}: Start kollidiert: {b.startList[0]}")
        if cc.pointInCollision(b.goalList[0]):
            raise ValueError(f"{b.name}: Goal kollidiert: {b.goalList[0]}")

# ============================================================
# RUN
# ============================================================

# Enhancement-Strategie:
config = {
    "enhanceMode": "mode4_start_goal_corr",
    "initialRoadmapSize": 2,
    "updateRoadmapSize":  3,
    "kNearest":           4,
    "maxIterations":      60,
    "corridorSigma": 1.8,
    "corridorAlongSigma": 0.45,
    "corridorTries": 6,
}


# Clearance zentral setzen (wie du es wolltest)
ROBOT_RADIUS  = 0.1
SAFETY_MARGIN = 0.05

benchmarks_2b = build_2b_benchmarks_from_task1style(
    robot_radius=ROBOT_RADIUS,
    safety_margin=SAFETY_MARGIN
)

# 2: Planar Easy, 3: Planar Passage
b = benchmarks_2b[3]

# Start & Ziel festlegen (NICHT verändern)
set_benchmark_start_goal(b, start=[math.pi, 0.0], goal=[0.0, 0.0], check=True)

cc = b.collisionChecker

print("Start collision?", cc.pointInCollision(getattr(b, "startList", [b.start])[0]))
print("Goal  collision?", cc.pointInCollision(getattr(b, "goalList",  [b.goal])[0]))

start_list, goal_list = ensure_valid_start_goal(b, cc, max_tries=20000)

print("Benchmark:", getattr(b, "name", "unnamed"))
print("Dim:", cc.getDim())

# Falls Goal kollidiert: neues Goal sampeln (für Planar sinnvoll)
if cc.pointInCollision(goal_list[0]):
    print("\nGoal ist in Kollision -> suche kollisionsfreies Goal ...")
    qG = sample_free_angles(cc, max_tries=20000)
    b.goalList = [qG]
    b.goal = qG
    goal_list = [qG]
    print("Neues Goal:", qG, "collision?", cc.pointInCollision(qG))

# Sinnvoll: Start/Goal override (hilft Mode4)
config["start_override"] = start_list[0]
config["goal_override"]  = goal_list[0]

planner = EnhancedLazyPRM(cc)
path_nodes = planner.planPath(start_list, goal_list, config)

G = planner.graph
start_key = "start" if "start" in G.nodes else None
goal_key  = "goal"  if "goal"  in G.nodes else None

print("\n--- GRAPH DIAG ---")
print("Graph nodes:", G.number_of_nodes())
print("Graph edges:", G.number_of_edges())
print("Has 'start':", start_key is not None, "| Has 'goal':", goal_key is not None)
if start_key is not None:
    print("deg(start):", G.degree[start_key])
if goal_key is not None:
    print("deg(goal) :", G.degree[goal_key])
if start_key is not None and goal_key is not None:
    print("nx.has_path(start,goal):", nx.has_path(G, start_key, goal_key))
    comps = list(nx.connected_components(G))
    print("Connected components:", len(comps))
    s_comp = next((i for i, c in enumerate(comps) if start_key in c), None)
    g_comp = next((i for i, c in enumerate(comps) if goal_key in c), None)
    print("start component:", s_comp, "| goal component:", g_comp)

ok = bool(path_nodes) and len(path_nodes) >= 2
print("\nSuccess:", ok, "| pathlen:", len(path_nodes) if path_nodes else 0)

if not ok:
    print("Hinweis: Wenn success=False, erhöhe testweise maxIterations oder updateRoadmapSize,")
    print("oder prüfe, ob Start/Goal wirklich in derselben freien Komponente liegen.")

path_q = path_positions_from_graph(planner.graph, path_nodes)

if isinstance(cc, PlanarRobot2DoFCollisionChecker):
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-math.pi, math.pi), (-math.pi, math.pi)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="arm",
        interval_ms=30,
        densify_step=0.03,
        arm_L1=getattr(cc, "L1", 1.6),
        arm_L2=getattr(cc, "L2", 1.2),
        show_clearance=False,
    )
else:
    ani = animate_workspace_and_cspace(
        b, planner, path_q,
        q_limits=[(-3, 3), (-3, 3)],
        ws_limits=(-3, 3, -3, 3),
        robot_kind="point",
        interval_ms=30,
        densify_step=0.03,
        show_clearance=True,
    )

display(HTML(ani.to_jshtml()))


# Fazit

Ziel dieser Arbeit ist es, den Lazy-PRM-Ansatz mit unterschiedlichen Sampling-Strategien systematisch zu analysieren und deren Einfluss auf Planungseffizienz, Pfadqualität und Robustheit zu untersuchen. Die Ergebnisse zeigen deutlich, dass Lazy-PRM gegenüber klassischen PRM-Varianten eine signifikante Reduktion der notwendigen Kollisionschecks ermöglicht, ohne die grundsätzliche Lösbarkeit der Planungsprobleme einzuschränken. Gleichzeitig wird sichtbar, dass die Wahl der Sampling-Strategie einen maßgeblichen Einfluss auf Rechenzeit, Pfadlänge und Stabilität der Planung besitzt.

Im Vergleich der einzelnen Modi lassen sich klare Charakteristika identifizieren. Die seedbasierten Verfahren (Mode 1 und Mode 2) verfolgen einen stark lokal verdichtenden Ansatz und erzeugen häufig sehr dichte Roadmaps in der Umgebung bestehender Knoten. Dies kann in strukturierten Szenarien vorteilhaft sein, geht jedoch mit erhöhtem Rechenaufwand und einer hohen Anzahl an Kollisionsprüfungen einher. Mode 3 verfolgt eine global explorierende Max–Min-Strategie, die den Raum mit vergleichsweise wenigen, aber weit voneinander entfernten Knoten abdeckt. Zudem zeigt diese Enhancementstrategie in vielen Benchmarks qualitativ gute Pfade, jedoch eine erhöhte Varianz und eine geringere Robustheit gegenüber ungünstigen Konfigurationen. Mode 4 zeigt in einfachen oder moderat strukturierten Szenarien ein stabiles Verhalten, liefert jedoch keinen konsistenten Vorteil gegenüber der Baseline. In komplexen oder topologisch irreführenden Umgebungen kann die start-ziel-gerichtete Samplingstrategie sogar nachteilig wirken, da sie relevante Umgehungsstrukturen nicht ausreichend exploriert.

Der Vergleich zwischen 2-DoF-Punktroboter und 2-DoF-Planarroboter verdeutlicht die zunehmende Komplexität realer Planungsprobleme. Während der Punktroboter einen anschaulichen, didaktisch gut nachvollziehbaren Zugang zur Wirkung der Sampling-Strategien erlaubt, treten beim Planarroboter die rechentechnischen Herausforderungen klar in den Vordergrund. Der Konfigurationsraum ist nicht mehr kartesisch, Kollisions- und Kantenprüfungen sind deutlich teurer, und Unterschiede zwischen den Sampling-Modi manifestieren sich weniger in der Geometrie der Roadmap als vielmehr in Rechenzeit und Kollisionschecks. Gerade in diesem Szenario entfaltet der Lazy-PRM-Ansatz seinen größten Nutzen.

Abschließend ist festzuhalten, dass die vorgestellten Verfahren trotz ihrer Leistungsfähigkeit klaren Grenzen unterliegen. Die Ergebnisse sind stark parameterabhängig, eine Garantie auf optimale Pfade besteht nicht, und einzelne Strategien reagieren empfindlich auf Benchmark-spezifische Strukturen. Gleichzeitig bietet der Ansatz vielfältige Erweiterungsmöglichkeiten, etwa durch adaptive Parametrisierung, alternative Heuristiken oder die Übertragung auf Systeme mit höherer Freiheitsgradzahl. Insgesamt zeigt die Arbeit, dass Lazy-PRM in Kombination mit geeigneten Sampling-Strategien ein leistungsfähiges und flexibles Werkzeug für die probabilistische Pfadplanung darstellt.